In [118]:
# ======================================
# PART 1: Import Library
# ======================================

from scapy.all import IP, IPv6, TCP, Ether, Padding, wrpcap, raw, rdpcap , load_contrib
from scapy.contrib.bgp import *
from scapy.utils import PcapReader
from scipy.stats import pareto, weibull_min
import datetime
import time
import random
import os
import csv
import struct


In [119]:
# ======================================
# PART 2: Delay Distribution
# ======================================

def gauss_delay(mu_ms, sigma_ms):
    """Generate delay using Gaussian distribution."""
    d = max(0, random.gauss(mu_ms, sigma_ms) / 1000.0)
    time.sleep(d)
    return d

def pareto_delay(shape, scale_ms):
    """Generate delay using Pareto distribution."""
    d = max(0, pareto.rvs(shape, scale=scale_ms, size=1)[0] / 1000.0)
    time.sleep(d)
    return d

def weibull_delay(shape, scale_ms):
    """Generate delay using Weibull distribution."""
    d = max(0, weibull_min.rvs(shape, scale=scale_ms, size=1)[0] / 1000.0)
    time.sleep(d)
    return d

def apply_delay(is_attack=False, distribution='pareto'):
    """Apply appropriate delay based on traffic type and distribution."""
    if distribution == 'gauss':
        if is_attack:
            return gauss_delay(mu_ms=15, sigma_ms=5)
        else:
            return gauss_delay(mu_ms=10, sigma_ms=2)
            
    elif distribution == 'weibull':
        if is_attack:
            return weibull_delay(shape=1.0, scale_ms=20)
        else:
            return weibull_delay(shape=1.5, scale_ms=10)
            
    else:  # default to pareto
        if is_attack:
            return pareto_delay(shape=1.8, scale_ms=15)
        else:
            return pareto_delay(shape=2.5, scale_ms=8)

# Global variable to control which distribution to use
DELAY_DISTRIBUTION = 'pareto'  # Options: 'gauss', 'pareto', 'weibull'

In [120]:
# ======================================
# PART 3: AS Topology Generation
# ======================================

def generate_as_topology(num_tier1=2, num_tier2=3, num_tier3=4, ixp_content=2):
    """Generate a BGP topology with hierarchical AS structure"""
    # Generate AS numbers for each tier
    as_numbers = {
        "tier1": random.sample(range(1000, 5000), num_tier1),
        "tier2": random.sample(range(10000, 20000), num_tier2),
        "tier3": random.sample(range(30000, 50000), num_tier3),
        "ixp_content": random.sample(range(50000, 65000), ixp_content)
    }
    
    # Initialize topology structure
    topology = {}
    
    # Create basic structure for all ASes
    all_asns = []
    for tier, asn_list in as_numbers.items():
        tier_level = int(tier.replace("tier", "")) if "tier" in tier else 4
        for asn in asn_list:
            all_asns.append(asn)
            topology[asn] = {
                "tier": tier_level,
                "neighbors": [],
                "relationships": {}
            }
    
    # Connect Tier 1s to each other (full mesh)
    for i, asn1 in enumerate(as_numbers["tier1"]):
        for asn2 in as_numbers["tier1"][i+1:]:
            topology[asn1]["neighbors"].append(asn2)
            topology[asn2]["neighbors"].append(asn1)
            # Peer-to-peer relationship
            topology[asn1]["relationships"][asn2] = "peer"
            topology[asn2]["relationships"][asn1] = "peer"
    
    # Connect Tier 2s to Tier 1s (provider-customer)
    for asn2 in as_numbers["tier2"]:
        # Each Tier 2 connects to at least one Tier 1
        num_providers = random.randint(1, len(as_numbers["tier1"]))
        providers = random.sample(as_numbers["tier1"], num_providers)
        
        for asn1 in providers:
            topology[asn2]["neighbors"].append(asn1)
            topology[asn1]["neighbors"].append(asn2)
            # Provider-customer relationship
            topology[asn2]["relationships"][asn1] = "provider"
            topology[asn1]["relationships"][asn2] = "customer"
    
    # Connect some Tier 2s to each other (peer-to-peer)
    for i, asn1 in enumerate(as_numbers["tier2"]):
        for asn2 in random.sample(as_numbers["tier2"][i+1:], 
                                 min(len(as_numbers["tier2"])-i-1, random.randint(0, 2))):
            if asn2 not in topology[asn1]["neighbors"]:
                topology[asn1]["neighbors"].append(asn2)
                topology[asn2]["neighbors"].append(asn1)
                # Peer-to-peer relationship
                topology[asn1]["relationships"][asn2] = "peer"
                topology[asn2]["relationships"][asn1] = "peer"
    
    # Connect Tier 3s to Tier 2s (provider-customer)
    for asn3 in as_numbers["tier3"]:
        # Each Tier 3 connects to at least one Tier 2
        num_providers = random.randint(1, min(2, len(as_numbers["tier2"])))
        providers = random.sample(as_numbers["tier2"], num_providers)
        
        for asn2 in providers:
            topology[asn3]["neighbors"].append(asn2)
            topology[asn2]["neighbors"].append(asn3)
            # Provider-customer relationship
            topology[asn3]["relationships"][asn2] = "provider"
            topology[asn2]["relationships"][asn3] = "customer"
    
    # Connect IXP to multiple ASes
    if len(as_numbers["ixp_content"]) > 0:
        ixp_asn = as_numbers["ixp_content"][0]
        # Connect IXP to all Tier 2s and some Tier 3s
        for asn in as_numbers["tier2"] + random.sample(as_numbers["tier3"], 
                                                      min(2, len(as_numbers["tier3"]))):
            topology[ixp_asn]["neighbors"].append(asn)
            topology[asn]["neighbors"].append(ixp_asn)
            # IXP relationship (peer)
            topology[ixp_asn]["relationships"][asn] = "peer"
            topology[asn]["relationships"][ixp_asn] = "peer"
    
    # Connect Content Provider to multiple ASes
    if len(as_numbers["ixp_content"]) > 1:
        content_asn = as_numbers["ixp_content"][1]
        # Connect Content to some Tier 1s and Tier 2s
        for asn in random.sample(as_numbers["tier1"], 1) + random.sample(as_numbers["tier2"], 
                                                                        min(2, len(as_numbers["tier2"]))):
            topology[content_asn]["neighbors"].append(asn)
            topology[asn]["neighbors"].append(content_asn)
            # Content provider relationship (customer of transit providers)
            topology[content_asn]["relationships"][asn] = "provider"
            topology[asn]["relationships"][content_asn] = "customer"
    
    # Select a main source and destination AS for our focus (for attack scenarios)
    # We'll use two Tier 3 ASes for this
    if len(as_numbers["tier3"]) >= 2:
        main_src_as = as_numbers["tier3"][0]
        main_dst_as = as_numbers["tier3"][1]
        
        # Make sure these two ASes are connected
        if main_dst_as not in topology[main_src_as]["neighbors"]:
            topology[main_src_as]["neighbors"].append(main_dst_as)
            topology[main_dst_as]["neighbors"].append(main_src_as)
            # Set up a peer-to-peer relationship
            topology[main_src_as]["relationships"][main_dst_as] = "peer"
            topology[main_dst_as]["relationships"][main_src_as] = "peer"
    else:
        # Fallback if we don't have at least 2 Tier 3 ASes
        main_src_as = all_asns[0]
        main_dst_as = all_asns[1]
    
    return topology, as_numbers, main_src_as, main_dst_as

# Generate our topology
topology, as_numbers, main_src_as, main_dst_as = generate_as_topology()

# Display topology information
print(f"Generated AS topology with {len(topology)} ASes:")
for tier, asns in as_numbers.items():
    print(f"  {tier.capitalize()}: {', '.join(map(str, asns))}")

print(f"\nMain AS pair for attack scenarios: AS{main_src_as} and AS{main_dst_as}")

print("\nNeighbor Relationships:")
for asn, info in topology.items():
    print(f"  AS{asn} (Tier {info['tier']}):")
    for neighbor in info['neighbors']:
        rel = info['relationships'].get(neighbor, "unknown")
        print(f"    - AS{neighbor} ({rel})")

Generated AS topology with 11 ASes:
  Tier1: 3010, 2430
  Tier2: 15376, 16170, 16741
  Tier3: 43289, 37250, 39618, 46797
  Ixp_content: 62558, 60859

Main AS pair for attack scenarios: AS43289 and AS37250

Neighbor Relationships:
  AS3010 (Tier 1):
    - AS2430 (peer)
    - AS16170 (customer)
    - AS16741 (customer)
    - AS60859 (customer)
  AS2430 (Tier 1):
    - AS3010 (peer)
    - AS15376 (customer)
    - AS16170 (customer)
    - AS16741 (customer)
  AS15376 (Tier 2):
    - AS2430 (provider)
    - AS16741 (peer)
    - AS43289 (customer)
    - AS37250 (customer)
    - AS39618 (customer)
    - AS46797 (customer)
    - AS62558 (peer)
    - AS60859 (customer)
  AS16170 (Tier 2):
    - AS2430 (provider)
    - AS3010 (provider)
    - AS16741 (peer)
    - AS62558 (peer)
  AS16741 (Tier 2):
    - AS2430 (provider)
    - AS3010 (provider)
    - AS15376 (peer)
    - AS16170 (peer)
    - AS43289 (customer)
    - AS37250 (customer)
    - AS39618 (customer)
    - AS46797 (customer)
    - AS625

In [121]:
# ======================================
# PART 4: IP and Interface Allocation
# ======================================

# Special prefixes for hijack scenarios - these will always be the same
PREDEFINED_PREFIXES = [
    "203.0.113.0/24",   # Primary prefix to be hijacked
    "198.51.100.0/24",  # Secondary prefix
    "192.0.2.0/24"      # Third prefix for other scenarios
]

def allocate_ip_addresses(topology, as_numbers, main_src_as, main_dst_as):
    """Allocate IP addresses to ASes and their interfaces"""
    ip_allocations = {}
    
    # Assign router IDs and prefix blocks to each AS
    for tier, asn_list in as_numbers.items():
        for asn in asn_list:
            # Generate router ID logic
            if tier == "tier1":
                octet1, octet2 = 100, random.randint(64, 127)
            elif tier == "tier2":
                octet1, octet2 = 172, random.randint(16, 31)
            elif tier == "tier3":
                octet1, octet2 = 192, 168
            else:
                octet1, octet2 = 10, random.randint(0, 255)
            
            octet3 = random.randint(0, 255)
            router_id = f"{octet1}.{octet2}.{octet3}.1"
            
            # Generate prefixes to announce based on AS
            announced_prefixes = []
            
            # Special case: For our main source AS, use predefined prefixes
            if asn == main_src_as:
                # Source AS gets the predefined prefixes for hijack scenarios
                announced_prefixes = PREDEFINED_PREFIXES.copy()
            else:
                # All other ASes get random prefixes
                if tier == "tier1":
                    # Tier 1s announce large blocks (/16)
                    for _ in range(random.randint(1, 2)):
                        prefix = f"203.{random.randint(0, 254)}.0.0/16"  # Avoid 203.0.113.0
                        if prefix != "203.0.113.0/16":  # Avoid overlapping with hijack prefix
                            announced_prefixes.append(prefix)
                elif tier == "tier2":
                    # Tier 2s announce medium blocks (/24)
                    for _ in range(random.randint(1, 3)):
                        prefix = f"198.51.{random.randint(0, 99)}.0/24"  # Avoid 198.51.100.0
                        if prefix != "198.51.100.0/24":  # Avoid overlapping with secondary prefix
                            announced_prefixes.append(prefix)
                elif tier == "tier3":
                    # Tier 3s announce specific blocks
                    for _ in range(random.randint(1, 2)):
                        third = random.randint(3, 255)  # Avoid 192.0.2.0
                        prefix = f"192.0.{third}.0/24"
                        if prefix != "192.0.2.0/24":  # Avoid overlapping with third prefix
                            announced_prefixes.append(prefix)
                else:
                    # Content providers
                    prefix = f"198.18.{random.randint(0, 255)}.0/24"
                    announced_prefixes.append(prefix)
                
                # Make sure we have at least one prefix
                if not announced_prefixes:
                    announced_prefixes.append(f"172.{random.randint(20, 30)}.{random.randint(0, 255)}.0/24")
            
            # Create entry for this AS
            ip_allocations[asn] = {
                "router_id": router_id,
                "announced_prefixes": announced_prefixes,
                "interfaces": {}
            }
    
    # Allocate interface IPs for peering links
    for asn, info in topology.items():
        for neighbor in info["neighbors"]:
            # Skip if we've already allocated this peering link
            if neighbor in ip_allocations[asn]["interfaces"]:
                continue
            
            # Generate a /30 network for this peering (point-to-point)
            link_net1 = random.randint(0, 255)
            link_net2 = random.randint(0, 255)
            link_net3 = random.randint(0, 63) * 4  # Ensure multiple of 4 for /30
            
            # Assign .1 to lower ASN and .2 to higher ASN
            if asn < neighbor:
                ip_allocations[asn]["interfaces"][neighbor] = f"10.{link_net1}.{link_net2}.{link_net3+1}"
                ip_allocations[neighbor]["interfaces"][asn] = f"10.{link_net1}.{link_net2}.{link_net3+2}"
            else:
                ip_allocations[asn]["interfaces"][neighbor] = f"10.{link_net1}.{link_net2}.{link_net3+2}"
                ip_allocations[neighbor]["interfaces"][asn] = f"10.{link_net1}.{link_net2}.{link_net3+1}"
    
    return ip_allocations

# Allocate IP addresses for our topology
ip_allocations = allocate_ip_addresses(topology, as_numbers, main_src_as, main_dst_as)

# Display IP allocation information
print("IP Allocations:")
for asn, info in ip_allocations.items():
    print(f"  AS{asn}:")
    print(f"    Router ID: {info['router_id']}")
    print(f"    Announced Prefixes: {', '.join(info['announced_prefixes'])}")
    print(f"    Interfaces:")
    for neighbor, ip in info["interfaces"].items():
        print(f"      - To AS{neighbor}: {ip}")
    print()

# Make sure our main ASNs are properly connected
print(f"Main AS pair for attack scenarios: AS{main_src_as} and AS{main_dst_as}")
if main_dst_as in topology[main_src_as]["neighbors"]:
    print(f"  ✅ AS{main_src_as} and AS{main_dst_as} are neighbors in the topology")
else:
    print(f"  ❌ AS{main_src_as} and AS{main_dst_as} are not direct neighbors, establishing connection")
    # Ensure these ASes are connected
    topology[main_src_as]["neighbors"].append(main_dst_as)
    topology[main_dst_as]["neighbors"].append(main_src_as)
    topology[main_src_as]["relationships"][main_dst_as] = "peer"
    topology[main_dst_as]["relationships"][main_src_as] = "peer"
    
    # Allocate interface IPs if needed
    if main_dst_as not in ip_allocations[main_src_as]["interfaces"]:
        link_net1 = random.randint(0, 255)
        link_net2 = random.randint(0, 255)
        link_net3 = random.randint(0, 63) * 4
        ip_allocations[main_src_as]["interfaces"][main_dst_as] = f"10.{link_net1}.{link_net2}.{link_net3+1}"
        ip_allocations[main_dst_as]["interfaces"][main_src_as] = f"10.{link_net1}.{link_net2}.{link_net3+2}"

# Print prefixes for the main source AS (for hijack scenarios)
print(f"\nPrefixes for main source AS{main_src_as} (for hijack scenarios):")
for prefix in ip_allocations[main_src_as]["announced_prefixes"]:
    print(f"  - {prefix}")


# IP ID ranges for different traffic types
NORMAL_TRAFFIC_ID_RANGE = (0x03E8, 0x7527)      # Regular BGP updates, keepalives
PREFIX_HIJACK_ID_RANGE = (0x7530, 0x9C3F)      # Prefix hijacking attacks
PATH_MANIP_ID_RANGE = (0x9C40, 0xC34F)         # Path manipulation attacks
DOS_ATTACK_ID_RANGE = (0xC350, 0xEA5F)   # More-specific prefix hijacking
ROUTE_LEAK_ID_RANGE = (0xEA60, 0xFFFF)         # Route leaks


# Document this mapping in a dictionary for reference
ATTACK_TYPE_MAPPING = {
    "normal": NORMAL_TRAFFIC_ID_RANGE,
    "prefix_hijack": PREFIX_HIJACK_ID_RANGE,
    "path_manipulation": PATH_MANIP_ID_RANGE,
    "DOS_ATTACK_ID_RANGE": DOS_ATTACK_ID_RANGE, 
    "route_leak": ROUTE_LEAK_ID_RANGE
}


IP Allocations:
  AS3010:
    Router ID: 100.89.241.1
    Announced Prefixes: 203.234.0.0/16, 203.182.0.0/16
    Interfaces:
      - To AS2430: 10.5.123.246
      - To AS16170: 10.77.241.217
      - To AS16741: 10.74.214.5
      - To AS60859: 10.1.82.117

  AS2430:
    Router ID: 100.69.79.1
    Announced Prefixes: 203.176.0.0/16
    Interfaces:
      - To AS3010: 10.5.123.245
      - To AS15376: 10.237.122.225
      - To AS16170: 10.38.240.97
      - To AS16741: 10.242.138.1

  AS15376:
    Router ID: 172.23.129.1
    Announced Prefixes: 198.51.34.0/24
    Interfaces:
      - To AS2430: 10.237.122.226
      - To AS16741: 10.18.178.185
      - To AS43289: 10.134.185.25
      - To AS37250: 10.71.254.229
      - To AS39618: 10.229.204.129
      - To AS46797: 10.111.121.197
      - To AS62558: 10.179.83.237
      - To AS60859: 10.64.101.105

  AS16170:
    Router ID: 172.24.110.1
    Announced Prefixes: 198.51.45.0/24, 198.51.51.0/24, 198.51.65.0/24
    Interfaces:
      - To AS3010: 10.7

In [122]:
# ======================================
# PART 5: Generate BGP Sessions for All ASes
# ======================================

pkts = []  # Clear the global packets list before generating new traffic

def generate_all_bgp_sessions(topology, ip_allocations):
    """Generate BGP sessions for all AS pairs in the topology"""
    
    # Initialize session dictionaries
    bgp_sessions_ipv4 = {}
    bgp_sessions_ipv6 = {}

    # Track sequence numbers for each AS pair
    seq_numbers_v4 = {}
    seq_numbers_v6 = {}
    all_packets = []

    # For each AS in the topology
    for asn, info in topology.items():
        # For each neighbor of this AS
        for neighbor in info["neighbors"]:
            # Skip if we've already processed this pair (avoid duplicates)
            if (asn, neighbor) in seq_numbers_v4:
                continue
            
            print(f"Creating BGP session between AS{asn} and AS{neighbor}...")
            
            # Get IPv4 parameters for this BGP session
            src_ipv4 = ip_allocations[asn]["interfaces"][neighbor]
            dst_ipv4 = ip_allocations[neighbor]["interfaces"][asn]
            src_router_id = ip_allocations[asn]["router_id"]
            dst_router_id = ip_allocations[neighbor]["router_id"]
            
            # Generate IPv6 addresses based on IPv4
            ipv4_parts = src_ipv4.split('.')
            src_ipv6 = f"2001:db8:{int(ipv4_parts[2]):x}:{int(ipv4_parts[3]):x}::1"
            
            ipv4_parts = dst_ipv4.split('.')
            dst_ipv6 = f"2001:db8:{int(ipv4_parts[2]):x}:{int(ipv4_parts[3]):x}::2"
            
            # Generate MAC addresses
            src_mac = "00:" + ":".join([f"{random.randint(0, 255):02x}" for _ in range(5)])
            dst_mac = "00:" + ":".join([f"{random.randint(0, 255):02x}" for _ in range(5)])
            
            # Generate port numbers
            src_port = random.randint(30000, 65000)
            dst_port = 179  # Standard BGP port
            
            
            # Initialize sequence numbers
            seq_a_v4 = random.randint(1000, 10000)
            seq_b_v4 = random.randint(1000, 10000)
            seq_a_v6 = random.randint(1000, 10000)
            seq_b_v6 = random.randint(1000, 10000)
            
            seq_numbers_v4[(asn, neighbor)] = (seq_a_v4, seq_b_v4)
            seq_numbers_v6[(asn, neighbor)] = (seq_a_v6, seq_b_v6)
            

            # List to store packets for this session
            session_packets_v4 = []
            session_packets_v6 = []

            # TCP options
            tcp_options = [('MSS', 1460)]
            
            #================================================
            # IPv4 BGP Session
            #================================================
            
            src_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
            dst_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
            # TCP 3-way handshake for IPv4 session
            # SYN packet - Src to Dst (Client to BGP Server)
            syn_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=src_port, dport=dst_port, flags="S", seq=seq_a_v4, window=16384, options=tcp_options)
            if len(syn_pkt) < 60:
                pad_len = 60 - len(syn_pkt)
                syn_pkt = syn_pkt/Padding(load=b'\x00' * pad_len)
            all_packets.append(syn_pkt)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
            
            # SYN-ACK packet - Dst to Src (BGP Server to Client)
            synack_pkt = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dst_port, dport=src_port, flags="SA", seq=seq_b_v4, ack=seq_a_v4+1, window=16384, options=tcp_options)
            if len(synack_pkt) < 60:
                pad_len = 60 - len(synack_pkt)
                synack_pkt = synack_pkt/Padding(load=b'\x00' * pad_len)
            all_packets.append(synack_pkt)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
            
            # ACK packet - Src to Dst
            ack_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+1)/TCP(sport=src_port, dport=dst_port, flags="A", seq=seq_a_v4+1, ack=seq_b_v4+1, window=16384)
            if len(ack_pkt) < 60:
                pad_len = 60 - len(ack_pkt)
                ack_pkt = ack_pkt/Padding(load=b'\x00' * pad_len)
            all_packets.append(ack_pkt)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
            
            # Create BGP capabilities for OPEN message
            # 1. Multiprotocol IPv4 Unicast capability
            mp_ipv4_cap = BGPCapMultiprotocol(code=1, length=4, afi=1, safi=1)
            
            # 2. Multiprotocol IPv6 Unicast capability
            mp_ipv6_cap = BGPCapMultiprotocol(code=1, length=4, afi=2, safi=1)
            
            # 3. Route Refresh Capability (Cisco)
            rr_cisco = BGPCapGeneric(code=128, length=0)
            
            # 4. Route Refresh standard capability
            rr_std = BGPCapGeneric(code=2, length=0)
            
            # 5. Enhanced route refresh capability
            err_cap = BGPCapGeneric(code=70, length=0)
            
            # 6. Support for 4-octet AS capability for source
            as4_cap_src = BGPCapFourBytesASN(code=65, length=4, asn=asn)
            
            # 7. Support for 4-octet AS capability for destination
            as4_cap_dst = BGPCapFourBytesASN(code=65, length=4, asn=neighbor)
            
            # Create BGP Optional Parameters with correct field names
            opt_params_src = [
                BGPOptParam(param_type=2, param_length=len(mp_ipv4_cap), param_value=mp_ipv4_cap),
                BGPOptParam(param_type=2, param_length=len(mp_ipv6_cap), param_value=mp_ipv6_cap),
                BGPOptParam(param_type=2, param_length=len(rr_cisco), param_value=rr_cisco),
                BGPOptParam(param_type=2, param_length=len(rr_std), param_value=rr_std),
                BGPOptParam(param_type=2, param_length=len(err_cap), param_value=err_cap),
                BGPOptParam(param_type=2, param_length=len(as4_cap_src), param_value=as4_cap_src)
            ]
            
            # Create separate set for destination with its own AS number
            opt_params_dst = opt_params_src.copy()
            # Replace the last parameter (AS4) with destination AS
            opt_params_dst[-1] = BGPOptParam(
                param_type=2, 
                param_length=len(as4_cap_dst),
                param_value=as4_cap_dst
            )
            
            # BGP OPEN from src over IPv4
            open_a_v4 = BGPHeader(type=1)/BGPOpen(
                version=4, 
                my_as=asn, 
                hold_time=180,
                bgp_id=src_router_id,
                opt_param_len=None,
                opt_params=opt_params_src
            )
            
            open_a_pkt_v4 = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+2)/TCP(sport=src_port, dport=dst_port, flags="PA", seq=ack_pkt[TCP].seq, ack=ack_pkt[TCP].ack, window=16384)/open_a_v4
            if len(open_a_pkt_v4) < 60:
                pad_len = 60 - len(open_a_pkt_v4)
                open_a_pkt_v4 = open_a_pkt_v4/Padding(load=b'\x00' * pad_len)
            all_packets.append(open_a_pkt_v4)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
            seq_a_v4 += len(open_a_v4)
            
            # BGP OPEN from dst over IPv4
            open_b_v4 = BGPHeader(type=1)/BGPOpen(
                version=4, 
                my_as=neighbor, 
                hold_time=180,
                bgp_id=dst_router_id,
                opt_param_len=None,
                opt_params=opt_params_dst
            )
            
            open_b_pkt_v4 = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id+1)/TCP(sport=dst_port, dport=src_port, flags="PA", seq=synack_pkt[TCP].seq+1, ack=seq_a_v4, window=16384)/open_b_v4
            if len(open_b_pkt_v4) < 60:
                pad_len = 60 - len(open_b_pkt_v4)
                open_b_pkt_v4 = open_b_pkt_v4/Padding(load=b'\x00' * pad_len)
            all_packets.append(open_b_pkt_v4)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
            seq_b_v4 += len(open_b_v4)
            
            # KEEPALIVE from src over IPv4
            keep_a_v4 = BGPKeepAlive()
            keep_a_pkt_v4 = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+3)/TCP(sport=src_port, dport=dst_port, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/keep_a_v4
            if len(keep_a_pkt_v4) < 60:
                pad_len = 60 - len(keep_a_pkt_v4)
                keep_a_pkt_v4 = keep_a_pkt_v4/Padding(load=b'\x00' * pad_len)
            all_packets.append(keep_a_pkt_v4)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
            seq_a_v4 += len(keep_a_v4)
            
            # KEEPALIVE from dst over IPv4
            keep_b_v4 = BGPKeepAlive()
            keep_b_pkt_v4 = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id+2)/TCP(sport=dst_port, dport=src_port, flags="PA", seq=seq_b_v4, ack=seq_a_v4, window=16384)/keep_b_v4
            if len(keep_b_pkt_v4) < 60:
                pad_len = 60 - len(keep_b_pkt_v4)
                keep_b_pkt_v4 = keep_b_pkt_v4/Padding(load=b'\x00' * pad_len)
            all_packets.append(keep_b_pkt_v4)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
            seq_b_v4 += len(keep_b_v4)
            
            # Prepare common path attributes for advertisements
            # 1. ORIGIN attribute (mandatory)
            origin = BGPPathAttr(type_flags=0x40, type_code=1)
            origin.attribute = BGPPAOrigin(origin=0)  # IGP = 0
            
            # 2. AS_PATH attribute (mandatory)
            as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
            
            # Create a proper AS_PATH segment
            as_path_segment = BGPPAASPath()
            # Create a segment with an AS_SEQUENCE containing asn
            segment = BGPPAASPath.ASPathSegment(
                segment_type=2,  # AS_SEQUENCE
                segment_length=1,
                segment_value=[asn]
            )
            # Add the segment to the AS_PATH
            as_path_segment.segments = [segment]
            as_path_attr.attribute = as_path_segment
            
            # 3. NEXT_HOP attribute for direct IPv4 advertisement (mandatory)
            next_hop_attr_v4 = BGPPathAttr(type_flags=0x40, type_code=3)
            next_hop_attr_v4.attribute = BGPPANextHop(next_hop=src_ipv4)
            
            # 4. MULTI_EXIT_DISC attribute (MED)
            med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
            med_attr.attribute = BGPPAMultiExitDisc(med=100)  # A typical MED value
            
            # 5. LOCAL_PREF attribute
            local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
            local_pref_attr.attribute = BGPPALocalPref(local_pref=200)
            
            # 6. COMMUNITIES attribute
            communities_list = []
            communities_list.append(BGPPACommunity(community=0xFFFFFF01))  # NO_EXPORT
            communities_list.append(BGPPACommunity(community=asn<<16|200))  # asn:200
            
            communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
            communities_attr.attribute = communities_list
            
            # IPv4 UPDATE
            for prefix in ip_allocations[asn]["announced_prefixes"]:
                # IPv4 advertisement directly in NLRI over IPv4 transport
                ipv4_update_v4 = BGPHeader(type=2)/BGPUpdate()
                
                # Set path attributes for IPv4 announcement
                ipv4_update_v4.path_attr = [
                    origin,               # ORIGIN
                    as_path_attr,         # AS_PATH
                    next_hop_attr_v4,     # NEXT_HOP
                    med_attr,             # MED
                    local_pref_attr,      # LOCAL_PREF
                    communities_attr      # COMMUNITIES
                ]
                
                # Add IPv4 NLRI
                ipv4_update_v4.nlri.append(BGPNLRI_IPv4(prefix=prefix))
                
                # Send IPv4 UPDATE
                ipv4_update_pkt_v4 = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+4)/TCP(sport=src_port, dport=dst_port, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/ipv4_update_v4
                if len(ipv4_update_pkt_v4) < 60:
                    pad_len = 60 - len(ipv4_update_pkt_v4)
                    ipv4_update_pkt_v4 = ipv4_update_pkt_v4/Padding(load=b'\x00' * pad_len)
                all_packets.append(ipv4_update_pkt_v4)
                apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
                seq_a_v4 += len(ipv4_update_v4)
                
                # ACK for update
                ack_update_pkt_v4 = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id+3)/TCP(sport=dst_port, dport=src_port, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
                if len(ack_update_pkt_v4) < 60:
                    pad_len = 60 - len(ack_update_pkt_v4)
                    ack_update_pkt_v4 = ack_update_pkt_v4/Padding(load=b'\x00' * pad_len)
                all_packets.append(ack_update_pkt_v4)
                apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)

                # Store session information in the dictionaries
                session_key = (asn, neighbor)
                bgp_sessions_ipv4[session_key] = {
                    "src_ipv4": src_ipv4,
                    "dst_ipv4": dst_ipv4,
                    "src_mac": src_mac,
                    "dst_mac": dst_mac,
                    "seq_a": seq_a_v4,
                    "seq_b": seq_b_v4,
                    "sport": src_port,
                    "dport": dst_port,
                    "packets": session_packets_v4
                }

            
            #================================================
            # IPv6 BGP Session
            #================================================
            
            # TCP 3-way handshake for IPv6 session
            seq_a_v6 = random.randint(2000, 3000)  # Different initial sequence numbers
            seq_b_v6 = random.randint(6000, 7000)  # to distinguish from IPv4 session
            
            # SYN packet - Src to Dst
            syn_pkt_v6 = Ether(src=src_mac, dst=dst_mac)/IPv6(src=src_ipv6, dst=dst_ipv6, hlim=64)/TCP(sport=src_port+1, dport=dst_port, flags="S", seq=seq_a_v6, window=16384, options=tcp_options)
            if len(syn_pkt_v6) < 60:
                pad_len = 60 - len(syn_pkt_v6)
                syn_pkt_v6 = syn_pkt_v6/Padding(load=b'\x00' * pad_len)
            all_packets.append(syn_pkt_v6)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
            
            # SYN-ACK packet - Dst to Src
            synack_pkt_v6 = Ether(src=dst_mac, dst=src_mac)/IPv6(src=dst_ipv6, dst=src_ipv6, hlim=64)/TCP(sport=dst_port, dport=src_port+1, flags="SA", seq=seq_b_v6, ack=seq_a_v6+1, window=16384, options=tcp_options)
            if len(synack_pkt_v6) < 60:
                pad_len = 60 - len(synack_pkt_v6)
                synack_pkt_v6 = synack_pkt_v6/Padding(load=b'\x00' * pad_len)
            all_packets.append(synack_pkt_v6)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
            
            # ACK packet - Src to Dst
            ack_pkt_v6 = Ether(src=src_mac, dst=dst_mac)/IPv6(src=src_ipv6, dst=dst_ipv6, hlim=64)/TCP(sport=src_port+1, dport=dst_port, flags="A", seq=seq_a_v6+1, ack=seq_b_v6+1, window=16384)
            if len(ack_pkt_v6) < 60:
                pad_len = 60 - len(ack_pkt_v6)
                ack_pkt_v6 = ack_pkt_v6/Padding(load=b'\x00' * pad_len)
            all_packets.append(ack_pkt_v6)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
            
            # BGP OPEN from src over IPv6
            open_a_v6 = BGPHeader(type=1)/BGPOpen(
                version=4, 
                my_as=asn, 
                hold_time=180,
                bgp_id=src_router_id,  # BGP ID is always IPv4 format even in IPv6 sessions
                opt_param_len=None,
                opt_params=opt_params_src
            )
            
            open_a_pkt_v6 = Ether(src=src_mac, dst=dst_mac)/IPv6(src=src_ipv6, dst=dst_ipv6, hlim=64)/TCP(sport=src_port+1, dport=dst_port, flags="PA", seq=ack_pkt_v6[TCP].seq, ack=ack_pkt_v6[TCP].ack, window=16384)/open_a_v6
            if len(open_a_pkt_v6) < 60:
                pad_len = 60 - len(open_a_pkt_v6)
                open_a_pkt_v6 = open_a_pkt_v6/Padding(load=b'\x00' * pad_len)
            all_packets.append(open_a_pkt_v6)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
            seq_a_v6 += len(open_a_v6)
            
            # BGP OPEN from dst over IPv6
            open_b_v6 = BGPHeader(type=1)/BGPOpen(
                version=4, 
                my_as=neighbor, 
                hold_time=180,
                bgp_id=dst_router_id,
                opt_param_len=None,
                opt_params=opt_params_dst
            )
            
            open_b_pkt_v6 = Ether(src=dst_mac, dst=src_mac)/IPv6(src=dst_ipv6, dst=src_ipv6, hlim=64)/TCP(sport=dst_port, dport=src_port+1, flags="PA", seq=synack_pkt_v6[TCP].seq+1, ack=seq_a_v6, window=16384)/open_b_v6
            if len(open_b_pkt_v6) < 60:
                pad_len = 60 - len(open_b_pkt_v6)
                open_b_pkt_v6 = open_b_pkt_v6/Padding(load=b'\x00' * pad_len)
            all_packets.append(open_b_pkt_v6)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
            seq_b_v6 += len(open_b_v6)
            
            # KEEPALIVE from src over IPv6
            keep_a_v6 = BGPKeepAlive()
            keep_a_pkt_v6 = Ether(src=src_mac, dst=dst_mac)/IPv6(src=src_ipv6, dst=dst_ipv6, hlim=64)/TCP(sport=src_port+1, dport=dst_port, flags="PA", seq=seq_a_v6, ack=seq_b_v6, window=16384)/keep_a_v6
            if len(keep_a_pkt_v6) < 60:
                pad_len = 60 - len(keep_a_pkt_v6)
                keep_a_pkt_v6 = keep_a_pkt_v6/Padding(load=b'\x00' * pad_len)
            all_packets.append(keep_a_pkt_v6)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
            seq_a_v6 += len(keep_a_v6)  
            
            # KEEPALIVE from dst over IPv6
            keep_b_v6 = BGPKeepAlive()
            keep_b_pkt_v6 = Ether(src=dst_mac, dst=src_mac)/IPv6(src=dst_ipv6, dst=src_ipv6, hlim=64)/TCP(sport=dst_port, dport=src_port+1, flags="PA", seq=seq_b_v6, ack=seq_a_v6, window=16384)/keep_b_v6
            if len(keep_b_pkt_v6) < 60:
                pad_len = 60 - len(keep_b_pkt_v6)
                keep_b_pkt_v6 = keep_b_pkt_v6/Padding(load=b'\x00' * pad_len)
            all_packets.append(keep_b_pkt_v6)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
            seq_b_v6 += len(keep_b_v6)
            
            # Generate IPv6 prefixes from IPv4 prefixes
            ipv6_prefixes = []
            for prefix in ip_allocations[asn]["announced_prefixes"]:
                ip_part, mask = prefix.split('/')
                octets = ip_part.split('.')
                ipv6_prefix = f"2001:db8:{int(octets[2]):x}:{int(octets[3]):x}::/64"
                ipv6_prefixes.append(ipv6_prefix)
            
            # Prepare IPv6 prefixes for MP_REACH_NLRI
            ipv6_nlri_objs = []
            for prefix in ipv6_prefixes:
                ipv6_nlri_objs.append(BGPNLRI_IPv6(prefix=prefix))
                
            # IPv6 advertisement via MP_REACH_NLRI over IPv6 transport
            ipv6_update_v6 = BGPHeader(type=2)/BGPUpdate()
            
            # MP_REACH_NLRI attribute for IPv6
            mp_reach_attr_v6 = BGPPathAttr(type_flags=0x80, type_code=14)
            
            # Create the MP_REACH_NLRI attribute for IPv6
            mp_reach_v6 = BGPPAMPReachNLRI(
                afi=2,                # IPv6 = 2
                safi=1,               # Unicast = 1
                nh_addr_len=16,       # IPv6 address length
                nh_v6_addr=src_ipv6,  # IPv6 next hop
                reserved=0,
                nlri=ipv6_nlri_objs   # IPv6 prefixes
            )
            
            mp_reach_attr_v6.attribute = mp_reach_v6
            
            # Combine path attributes for IPv6 update
            ipv6_update_v6.path_attr = [
                origin,               # ORIGIN
                as_path_attr,         # AS_PATH
                med_attr,             # MED
                local_pref_attr,      # LOCAL_PREF
                communities_attr,     # COMMUNITIES
                mp_reach_attr_v6      # MP_REACH_NLRI (always last)
            ]
            
            # Send IPv6 UPDATE over IPv6 session
            ipv6_update_pkt_v6 = Ether(src=src_mac, dst=dst_mac)/IPv6(src=src_ipv6, dst=dst_ipv6, hlim=64)/TCP(sport=src_port+1, dport=dst_port, flags="PA", seq=seq_a_v6, ack=seq_b_v6, window=16384)/ipv6_update_v6
            if len(ipv6_update_pkt_v6) < 60:
                pad_len = 60 - len(ipv6_update_pkt_v6)
                ipv6_update_pkt_v6 = ipv6_update_pkt_v6/Padding(load=b'\x00' * pad_len)
            all_packets.append(ipv6_update_pkt_v6)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
            seq_a_v6 += len(ipv6_update_v6)
            
            # ACK for IPv6 UPDATE
            ack_ipv6_update = Ether(src=dst_mac, dst=src_mac)/IPv6(src=dst_ipv6, dst=src_ipv6, hlim=64)/TCP(sport=dst_port, dport=src_port+1, flags="A", seq=seq_b_v6, ack=seq_a_v6, window=16384)
            if len(ack_ipv6_update) < 60:
                pad_len = 60 - len(ack_ipv6_update)
                ack_ipv6_update = ack_ipv6_update/Padding(load=b'\x00' * pad_len)
            all_packets.append(ack_ipv6_update)
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
            bgp_sessions_ipv6[session_key] = {
                "src_ipv6": src_ipv6,
                "dst_ipv6": dst_ipv6,
                "src_mac": src_mac,
                "dst_mac": dst_mac,
                "seq_a": seq_a_v6,
                "seq_b": seq_b_v6,
                "sport": src_port,
                "dport": dst_port,
                "packets": session_packets_v6
            }


            print(f"  Created BGP sessions between AS{asn} and AS{neighbor} (IPv4: {src_ipv4}<->{dst_ipv4}, IPv6: {src_ipv6}<->{dst_ipv6})")
            # Add all packets to the overall list
            all_packets.extend(session_packets_v4)
            all_packets.extend(session_packets_v6)

    print(f"Generated {len(all_packets)} BGP packets for {len(topology)} ASes")
    #return all_packets
    return bgp_sessions_ipv4, bgp_sessions_ipv6

# Generate BGP sessions for all AS pairs
print("Generating BGP sessions (IPv4 and IPv6) for all ASes in the topology...")
#normal_traffic = generate_all_bgp_sessions(topology, ip_allocations)
bgp_sessions_ipv4, bgp_sessions_ipv6 = generate_all_bgp_sessions(topology, ip_allocations)

# Add packets to the global pkts list for compatibility with your existing code
#pkts.extend(normal_traffic)
#wrpcap("/Users/shadimoteali/PhD/BGP_Traffic_Generation/pcaps/bgp_topology.pcap", pkts)
# Add these to the global variables
global_bgp_sessions_ipv4 = bgp_sessions_ipv4  # Make accessible globally
global_bgp_sessions_ipv6 = bgp_sessions_ipv6  # Make accessible globally
# Now extend the pkts list
normal_traffic = []
for session_packets in bgp_sessions_ipv4.values():
    normal_traffic.extend(session_packets["packets"])
for session_packets in bgp_sessions_ipv6.values():
    normal_traffic.extend(session_packets["packets"])
pkts.extend(normal_traffic)

# Optionally, print a summary
print(f"Created {len(bgp_sessions_ipv4)} IPv4 BGP sessions")
print(f"Created {len(bgp_sessions_ipv6)} IPv6 BGP sessions")
print(f"Added {len(normal_traffic)} session establishment packets")

# Print statistics
print(f"\nNormal Traffic Statistics:")
print(f"  Total packets generated: {len(normal_traffic)}")
print(f"  Total packets in global 'pkts' list: {len(pkts)}")

# Identify our main ASes for attack scenarios
print(f"\nMain AS pair for attack scenarios: AS{main_src_as} and AS{main_dst_as}")
print(f"  Source Router ID: {ip_allocations[main_src_as]['router_id']}")
print(f"  Destination Router ID: {ip_allocations[main_dst_as]['router_id']}")
print(f"  Source IPv4: {ip_allocations[main_src_as]['interfaces'][main_dst_as]}")
print(f"  Target Prefixes:")
for prefix in ip_allocations[main_src_as]['announced_prefixes']:
    print(f"    - {prefix}")

Generating BGP sessions (IPv4 and IPv6) for all ASes in the topology...
Creating BGP session between AS3010 and AS2430...
  Created BGP sessions between AS3010 and AS2430 (IPv4: 10.5.123.246<->10.5.123.245, IPv6: 2001:db8:7b:f6::1<->2001:db8:7b:f5::2)
Creating BGP session between AS3010 and AS16170...
  Created BGP sessions between AS3010 and AS16170 (IPv4: 10.77.241.217<->10.77.241.218, IPv6: 2001:db8:f1:d9::1<->2001:db8:f1:da::2)
Creating BGP session between AS3010 and AS16741...
  Created BGP sessions between AS3010 and AS16741 (IPv4: 10.74.214.5<->10.74.214.6, IPv6: 2001:db8:d6:5::1<->2001:db8:d6:6::2)
Creating BGP session between AS3010 and AS60859...
  Created BGP sessions between AS3010 and AS60859 (IPv4: 10.1.82.117<->10.1.82.118, IPv6: 2001:db8:52:75::1<->2001:db8:52:76::2)
Creating BGP session between AS2430 and AS3010...
  Created BGP sessions between AS2430 and AS3010 (IPv4: 10.5.123.245<->10.5.123.246, IPv6: 2001:db8:7b:f5::1<->2001:db8:7b:f6::2)
Creating BGP session betwe

In [123]:
# ======================================
# PART 6: Realistic BGP Update Scenarios
# ======================================
print("[+] Generating realistic BGP updates for IPv4 session...")

# ------------ Scenario 1: ORIGIN Change (IGP → INCOMPLETE → IGP) ------------
print("[*] Generating ORIGIN change scenario (Extended)...")

# Get the target prefix from the main source AS
target_prefix = random.choice(ip_allocations[main_src_as]["announced_prefixes"])
print(f"  Using prefix {target_prefix} for ORIGIN Change scenario")

# Get session parameters from the main AS pair
# Retrieve session parameters from the session dictionaries
session_key = (main_src_as, main_dst_as)
if session_key in global_bgp_sessions_ipv4:
    # Get IPv4 session info
    session_info = global_bgp_sessions_ipv4[session_key]
    src_ipv4 = session_info["src_ipv4"]
    dst_ipv4 = session_info["dst_ipv4"]
    src_mac = session_info["src_mac"]
    dst_mac = session_info["dst_mac"]
    sport = session_info["sport"]
    dport = session_info["dport"]
    seq_a_v4 = session_info["seq_a"]  # Start with stored sequence numbers
    seq_b_v4 = session_info["seq_b"]
else:
    print(f"  Warning: No established session found for AS{main_src_as}-AS{main_dst_as}")
    print(f"  Available session keys: {list(global_bgp_sessions_ipv4.keys())}")

# Use the SAME MAC addresses from previous session - DO NOT generate new ones
# Use the SAME sequence numbers - DO NOT arbitrarily add offsets
# Just increment src_ip_id for new packets
src_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
dst_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])

# Create path attributes for this update
# AS_PATH attribute
as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
as_path_segment = BGPPAASPath()
segment = BGPPAASPath.ASPathSegment(
    segment_type=2,  # AS_SEQUENCE
    segment_length=1,
    segment_value=[main_src_as]
)
as_path_segment.segments = [segment]
as_path_attr.attribute = as_path_segment

# NEXT_HOP attribute
next_hop_attr_v4 = BGPPathAttr(type_flags=0x40, type_code=3)
next_hop_attr_v4.attribute = BGPPANextHop(next_hop=src_ipv4)

# Create standard attributes for consistency
med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
med_attr.attribute = BGPPAMultiExitDisc(med=100)

local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
local_pref_attr.attribute = BGPPALocalPref(local_pref=200)

# COMMUNITIES attribute
communities_list = []
communities_list.append(BGPPACommunity(community=0xFFFFFF01))  # NO_EXPORT
communities_list.append(BGPPACommunity(community=main_src_as<<16|200))
communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
communities_attr.attribute = communities_list

# Step 1: Change ORIGIN to INCOMPLETE (2)
origin_change_update = BGPHeader(type=2)/BGPUpdate()

# Create origin attribute with INCOMPLETE (2)
origin_incomplete = BGPPathAttr(type_flags=0x40, type_code=1)
origin_incomplete.attribute = BGPPAOrigin(origin=2)  # INCOMPLETE = 2

# Use same prefix but with changed ORIGIN
origin_change_update.path_attr = [
    origin_incomplete,    # Changed from IGP (0) to INCOMPLETE (2)
    as_path_attr,
    next_hop_attr_v4,
    med_attr,
    local_pref_attr,
    communities_attr,
]

origin_change_update.nlri.append(BGPNLRI_IPv4(prefix=target_prefix))

# Send ORIGIN change update over IPv4
origin_change_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/origin_change_update
if len(origin_change_pkt) < 60:
    pad_len = 60 - len(origin_change_pkt)
    origin_change_pkt = origin_change_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(origin_change_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_a_v4 += len(origin_change_update)

# ACK for ORIGIN change
origin_change_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(origin_change_ack) < 60:
    pad_len = 60 - len(origin_change_ack)
    origin_change_ack = origin_change_ack/Padding(load=b'\x00' * pad_len)
pkts.append(origin_change_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
dst_ip_id += 1  # Increment for next packet

# EXTENSION 1: Add a KEEPALIVE message exchange
keepalive_msg = BGPHeader(type=4)  # Type 4 = KEEPALIVE
keepalive_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+1)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/keepalive_msg
if len(keepalive_pkt) < 60:
    pad_len = 60 - len(keepalive_pkt)
    keepalive_pkt = keepalive_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(keepalive_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
seq_a_v4 += len(keepalive_msg)

# ACK for KEEPALIVE
keepalive_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id+1)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(keepalive_ack) < 60:
    pad_len = 60 - len(keepalive_ack)
    keepalive_ack = keepalive_ack/Padding(load=b'\x00' * pad_len)
pkts.append(keepalive_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
dst_ip_id += 1

# EXTENSION 2: Change to EGP (1) instead of directly back to IGP
origin_egp = BGPPathAttr(type_flags=0x40, type_code=1)
origin_egp.attribute = BGPPAOrigin(origin=1)  # EGP = 1

origin_egp_update = BGPHeader(type=2)/BGPUpdate()
origin_egp_update.path_attr = [
    origin_egp,           # Changed from INCOMPLETE (2) to EGP (1)
    as_path_attr,
    next_hop_attr_v4,
    med_attr,
    local_pref_attr,
    communities_attr,
]

origin_egp_update.nlri.append(BGPNLRI_IPv4(prefix=target_prefix))

# Send EGP ORIGIN change over IPv4
origin_egp_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+2)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/origin_egp_update
if len(origin_egp_pkt) < 60:
    pad_len = 60 - len(origin_egp_pkt)
    origin_egp_pkt = origin_egp_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(origin_egp_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_a_v4 += len(origin_egp_update)

# ACK for EGP ORIGIN change
origin_egp_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id+2)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(origin_egp_ack) < 60:
    pad_len = 60 - len(origin_egp_ack)
    origin_egp_ack = origin_egp_ack/Padding(load=b'\x00' * pad_len)
pkts.append(origin_egp_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
dst_ip_id += 1

# Step 3: Change back to IGP (0)
origin_igp = BGPPathAttr(type_flags=0x40, type_code=1)
origin_igp.attribute = BGPPAOrigin(origin=0)  # IGP = 0

origin_igp_update = BGPHeader(type=2)/BGPUpdate()
origin_igp_update.path_attr = [
    origin_igp,           # Changed from EGP (1) to IGP (0)
    as_path_attr,
    next_hop_attr_v4,
    med_attr,
    local_pref_attr,
    communities_attr,
]

origin_igp_update.nlri.append(BGPNLRI_IPv4(prefix=target_prefix))

# Send IGP ORIGIN change over IPv4
origin_igp_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+3)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/origin_igp_update
if len(origin_igp_pkt) < 60:
    pad_len = 60 - len(origin_igp_pkt)
    origin_igp_pkt = origin_igp_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(origin_igp_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_a_v4 += len(origin_igp_update)

# ACK for IGP ORIGIN change
origin_igp_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id+3)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(origin_igp_ack) < 60:
    pad_len = 60 - len(origin_igp_ack)
    origin_igp_ack = origin_igp_ack/Padding(load=b'\x00' * pad_len)
pkts.append(origin_igp_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
dst_ip_id += 1

# EXTENSION 3: Add a multi-prefix update with additional prefixes
# Get additional prefixes
additional_prefixes = []
for prefix in ip_allocations[main_src_as]["announced_prefixes"]:
    if prefix != target_prefix:
        additional_prefixes.append(prefix)
        if len(additional_prefixes) >= 2:  # Get 2 additional prefixes
            break

if additional_prefixes:  # Only proceed if we have additional prefixes
    multi_prefix_update = BGPHeader(type=2)/BGPUpdate()
    multi_prefix_update.path_attr = [
        origin_igp,
        as_path_attr,
        next_hop_attr_v4,
        med_attr,
        local_pref_attr,
        communities_attr,
    ]
    
    # Add multiple prefixes to NLRI
    for prefix in additional_prefixes:
        multi_prefix_update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
    
    # Send multi-prefix update
    multi_prefix_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+4)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/multi_prefix_update
    if len(multi_prefix_pkt) < 60:
        pad_len = 60 - len(multi_prefix_pkt)
        multi_prefix_pkt = multi_prefix_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(multi_prefix_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    seq_a_v4 += len(multi_prefix_update)
    
    # ACK for multi-prefix update
    multi_prefix_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id+4)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
    if len(multi_prefix_ack) < 60:
        pad_len = 60 - len(multi_prefix_ack)
        multi_prefix_ack = multi_prefix_ack/Padding(load=b'\x00' * pad_len)
    pkts.append(multi_prefix_ack)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    dst_ip_id += 1

# EXTENSION 4: Add a withdrawal and re-announcement sequence
# Create withdrawal message
withdraw_update = BGPHeader(type=2)/BGPUpdate()
withdraw_update.withdrawn_routes = [BGPNLRI_IPv4(prefix=target_prefix)]

# Send withdrawal
withdraw_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+5)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/withdraw_update
if len(withdraw_pkt) < 60:
    pad_len = 60 - len(withdraw_pkt)
    withdraw_pkt = withdraw_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(withdraw_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
seq_a_v4 += len(withdraw_update)

# ACK for withdrawal
withdraw_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id+5)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(withdraw_ack) < 60:
    pad_len = 60 - len(withdraw_ack)
    withdraw_ack = withdraw_ack/Padding(load=b'\x00' * pad_len)
pkts.append(withdraw_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
dst_ip_id += 1

# Re-announce the prefix again
reannounce_update = BGPHeader(type=2)/BGPUpdate()
reannounce_update.path_attr = [
    origin_igp,
    as_path_attr,
    next_hop_attr_v4,
    med_attr,
    local_pref_attr,
    communities_attr,
]
reannounce_update.nlri.append(BGPNLRI_IPv4(prefix=target_prefix))

reannounce_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+6)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/reannounce_update
if len(reannounce_pkt) < 60:
    pad_len = 60 - len(reannounce_pkt)
    reannounce_pkt = reannounce_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(reannounce_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
seq_a_v4 += len(reannounce_update)

# ACK for reannounce
reannounce_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id+6)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(reannounce_ack) < 60:
    pad_len = 60 - len(reannounce_ack)
    reannounce_ack = reannounce_ack/Padding(load=b'\x00' * pad_len)
pkts.append(reannounce_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
dst_ip_id += 1

# EXTENSION 5: Add a multi-attribute change
new_local_pref = BGPPathAttr(type_flags=0x40, type_code=5)
new_local_pref.attribute = BGPPALocalPref(local_pref=300)  # Changed from 200

multi_attr_update = BGPHeader(type=2)/BGPUpdate()
multi_attr_update.path_attr = [
    origin_igp,
    as_path_attr,
    next_hop_attr_v4,
    new_local_pref,        # Changed LOCAL_PREF
    communities_attr,
]

multi_attr_update.nlri.append(BGPNLRI_IPv4(prefix=target_prefix))

# Send multi-attribute update
multi_attr_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+7)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/multi_attr_update
if len(multi_attr_pkt) < 60:
    pad_len = 60 - len(multi_attr_pkt)
    multi_attr_pkt = multi_attr_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(multi_attr_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
seq_a_v4 += len(multi_attr_update)

# ACK for multi-attribute update
multi_attr_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id+7)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(multi_attr_ack) < 60:
    pad_len = 60 - len(multi_attr_ack)
    multi_attr_ack = multi_attr_ack/Padding(load=b'\x00' * pad_len)
pkts.append(multi_attr_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
dst_ip_id += 1

# Final KEEPALIVE to complete the extended sequence
final_keepalive_msg = BGPHeader(type=4)  # Type 4 = KEEPALIVE
final_keepalive_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+8)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/final_keepalive_msg
if len(final_keepalive_pkt) < 60:
    pad_len = 60 - len(final_keepalive_pkt)
    final_keepalive_pkt = final_keepalive_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(final_keepalive_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
seq_a_v4 += len(final_keepalive_msg)

# ACK for final KEEPALIVE
final_keepalive_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id+8)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(final_keepalive_ack) < 60:
    pad_len = 60 - len(final_keepalive_ack)
    final_keepalive_ack = final_keepalive_ack/Padding(load=b'\x00' * pad_len)
pkts.append(final_keepalive_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)

# Update session sequence numbers for future reference
global_bgp_sessions_ipv4[session_key]["seq_a"] = seq_a_v4
global_bgp_sessions_ipv4[session_key]["seq_b"] = seq_b_v4

print(f"  Added {len(pkts)} packets for extended ORIGIN change scenario")
# ------------ Scenario 2: AS_PATH Modifications (Path Prepending) ------------
print("[*] Generating extended AS_PATH modification scenario...")

# Get multiple prefixes from the main source AS
import random
available_prefixes = ip_allocations[main_src_as]["announced_prefixes"]
if len(available_prefixes) >= 5:
    selected_prefixes = random.sample(available_prefixes, 5)
else:
    # If not enough prefixes available, duplicate some
    selected_prefixes = available_prefixes * (5 // len(available_prefixes) + 1)
    selected_prefixes = selected_prefixes[:5]

target_prefix = selected_prefixes[0]
print(f"  Using prefix {target_prefix} and others for AS_PATH modification scenario")

# Get session parameters from the main AS pair
session_key = (main_src_as, main_dst_as)
if session_key in global_bgp_sessions_ipv4:
    # Get IPv4 session info
    session_info = global_bgp_sessions_ipv4[session_key]
    src_ipv4 = session_info["src_ipv4"]
    dst_ipv4 = session_info["dst_ipv4"]
    src_mac = session_info["src_mac"]
    dst_mac = session_info["dst_mac"]
    sport = session_info["sport"]
    dport = session_info["dport"]
    seq_a_v4 = session_info["seq_a"]
    seq_b_v4 = session_info["seq_b"]
else:
    print(f"  Warning: No established session found for AS{main_src_as}-AS{main_dst_as}")
    src_ipv4 = ip_allocations[main_src_as]["interfaces"][main_dst_as]
    dst_ipv4 = ip_allocations[main_dst_as]["interfaces"][main_src_as]
    # Generate consistent MAC addresses
    src_mac = "00:00:%02x:%02x:%02x:%02x" % tuple(map(int, src_ipv4.split('.')))
    dst_mac = "00:00:%02x:%02x:%02x:%02x" % tuple(map(int, dst_ipv4.split('.')))
    sport = random.randint(30000, 65000)
    dport = BGP_PORT
    seq_a_v4 = random.randint(1000000, 2000000)
    seq_b_v4 = random.randint(3000000, 4000000)

# Use randomized IP ID values
src_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
dst_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])

# Helper function to create a base BGP update with common attributes
def create_base_update(origin_value=0):
    # Ensure origin attribute is defined
    origin = BGPPathAttr(type_flags=0x40, type_code=1)
    origin.attribute = BGPPAOrigin(origin=origin_value)  # IGP=0, EGP=1, INCOMPLETE=2

    # NEXT_HOP attribute
    next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
    next_hop_attr.attribute = BGPPANextHop(next_hop=src_ipv4)

    # MED attribute
    med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
    med_attr.attribute = BGPPAMultiExitDisc(med=100)

    # LOCAL_PREF attribute
    local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
    local_pref_attr.attribute = BGPPALocalPref(local_pref=200)

    # COMMUNITIES attribute
    communities_list = []
    communities_list.append(BGPPACommunity(community=0xFFFFFF01))  # NO_EXPORT
    communities_list.append(BGPPACommunity(community=main_src_as<<16|200))
    communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
    communities_attr.attribute = communities_list

    return origin, next_hop_attr, med_attr, local_pref_attr, communities_attr

# Helper function to create an AS_PATH attribute
def create_as_path(path_segments):
    as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
    as_path_segment = BGPPAASPath()
    as_path_segment.segments = path_segments
    as_path_attr.attribute = as_path_segment
    return as_path_attr

# Helper function to create an AS_SEQUENCE segment
def create_as_sequence(as_numbers):
    return BGPPAASPath.ASPathSegment(
        segment_type=2,  # AS_SEQUENCE
        segment_length=len(as_numbers),
        segment_value=as_numbers
    )

# Helper function to create an AS_SET segment
def create_as_set(as_numbers):
    return BGPPAASPath.ASPathSegment(
        segment_type=1,  # AS_SET
        segment_length=len(as_numbers),
        segment_value=as_numbers
    )

# Helper function to send a BGP update and receive ACK
def send_update_and_ack(update, src_id, dst_id, description=""):
    global seq_a_v4, seq_b_v4, pkts
    
    # Send update
    update_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/update
    if len(update_pkt) < 60:
        pad_len = 60 - len(update_pkt)
        update_pkt = update_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(update_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    seq_a_v4 += len(update)
    
    # Send ACK
    ack_pkt = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
    if len(ack_pkt) < 60:
        pad_len = 60 - len(ack_pkt)
        ack_pkt = ack_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(ack_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    
    if description:
        print(f"    - Sent {description}")
    
    return src_id + 1, dst_id + 1

# Helper function to send a KEEPALIVE and receive ACK
def send_keepalive_and_ack(src_id, dst_id):
    global seq_a_v4, seq_b_v4, pkts
    
    # Create KEEPALIVE
    keepalive = BGPHeader(type=4)  # KEEPALIVE type=4
    
    # Send KEEPALIVE
    keepalive_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/keepalive
    if len(keepalive_pkt) < 60:
        pad_len = 60 - len(keepalive_pkt)
        keepalive_pkt = keepalive_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(keepalive_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    seq_a_v4 += len(keepalive)
    
    # Send ACK for KEEPALIVE
    keepalive_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
    if len(keepalive_ack) < 60:
        pad_len = 60 - len(keepalive_ack)
        keepalive_ack = keepalive_ack/Padding(load=b'\x00' * pad_len)
    pkts.append(keepalive_ack)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    
    return src_id + 1, dst_id + 1

# List to collect nearby ASNs for more realistic AS paths
neighboring_asns = [main_src_as - 1, main_src_as + 1, main_src_as + 100, main_src_as + 200]
distant_asns = [main_src_as + 1000, main_src_as + 2000, main_src_as + 3000, main_src_as + 4000]

# Add some KEEPALIVE exchanges to start
print("  Generating initial KEEPALIVE exchanges...")
for i in range(4):
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 1: Basic AS path prepending variations (15 packets)
print("  Generating basic AS path prepending variations...")
for prepend_count in range(1, 8):  # Generate updates with 1-7 prepends
    # Create basic attributes
    origin, next_hop_attr, med_attr, local_pref_attr, communities_attr = create_base_update()
    
    # Create AS_PATH with varying degrees of prepending
    prepended_as = [main_src_as] * prepend_count
    as_path_segment = create_as_sequence(prepended_as)
    as_path_attr = create_as_path([as_path_segment])
    
    # Create and send update
    update = BGPHeader(type=2)/BGPUpdate()
    update.path_attr = [origin, as_path_attr, next_hop_attr, med_attr, local_pref_attr, communities_attr]
    update.nlri.append(BGPNLRI_IPv4(prefix=target_prefix))
    
    src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id, 
                                              f"update with {prepend_count}x prepending")
    
    # Add a KEEPALIVE after every second update
    if prepend_count % 2 == 0:
        src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 2: Multiple prefixes with AS path variations (25 packets)
print("  Announcing multiple prefixes with AS path variations...")
for prefix_index, prefix in enumerate(selected_prefixes):
    # Create AS paths with various transit ASNs
    for transit_count in range(1, 4):  # Generate paths with 1-3 transit ASes
        # Create basic attributes
        origin, next_hop_attr, med_attr, local_pref_attr, communities_attr = create_base_update()
        
        # Create a path with the main AS plus some transit ASes
        path_values = [main_src_as]
        for i in range(transit_count):
            path_values.append(neighboring_asns[i % len(neighboring_asns)])
        
        as_path_segment = create_as_sequence(path_values)
        as_path_attr = create_as_path([as_path_segment])
        
        # Create and send update
        update = BGPHeader(type=2)/BGPUpdate()
        update.path_attr = [origin, as_path_attr, next_hop_attr, med_attr, local_pref_attr, communities_attr]
        update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id, 
                                                 f"update for prefix {prefix} with {transit_count} transit ASes")
    
    # Add a KEEPALIVE after each prefix
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 3: AS_SET variations (20 packets)
print("  Generating AS_SET variations...")
for set_size in range(2, 5):  # Sets with 2-4 ASes
    for prefix_index, prefix in enumerate(selected_prefixes[:3]):  # Use first 3 prefixes
        # Create basic attributes
        origin, next_hop_attr, med_attr, local_pref_attr, communities_attr = create_base_update()
        
        # Create a path with a sequence and a set
        sequence_segment = create_as_sequence([main_src_as, neighboring_asns[0]])
        
        # Create AS_SET with varying ASes
        set_asns = []
        for i in range(set_size):
            set_asns.append(distant_asns[i % len(distant_asns)])
        set_segment = create_as_set(set_asns)
        
        # Create AS_PATH with both segments
        as_path_attr = create_as_path([sequence_segment, set_segment])
        
        # Create and send update
        update = BGPHeader(type=2)/BGPUpdate()
        update.path_attr = [origin, as_path_attr, next_hop_attr, med_attr, local_pref_attr, communities_attr]
        update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id, 
                                                 f"update with AS_SET size {set_size} for prefix {prefix}")
    
    # Add a KEEPALIVE after each set size
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 4: Withdrawals and re-announcements with different paths (20 packets)
print("  Generating withdrawals and re-announcements...")
for prefix_index, prefix in enumerate(selected_prefixes[:5]):  # Use all 5 prefixes
    # Create withdrawal update
    withdrawal = BGPHeader(type=2)/BGPUpdate()
    withdrawal.withdrawn_routes = [BGPNLRI_IPv4(prefix=prefix)]
    
    src_ip_id, dst_ip_id = send_update_and_ack(withdrawal, src_ip_id, dst_ip_id, 
                                             f"withdrawal of prefix {prefix}")
    
    # Re-announce with a different path
    origin, next_hop_attr, med_attr, local_pref_attr, communities_attr = create_base_update()
    
    # Create a path with some prepending and a transit AS
    as_path_segment = create_as_sequence([main_src_as, main_src_as, neighboring_asns[prefix_index % len(neighboring_asns)]])
    as_path_attr = create_as_path([as_path_segment])
    
    # Create and send re-announcement
    reannounce = BGPHeader(type=2)/BGPUpdate()
    reannounce.path_attr = [origin, as_path_attr, next_hop_attr, med_attr, local_pref_attr, communities_attr]
    reannounce.nlri.append(BGPNLRI_IPv4(prefix=prefix))
    
    src_ip_id, dst_ip_id = send_update_and_ack(reannounce, src_ip_id, dst_ip_id, 
                                             f"re-announcement of prefix {prefix}")

# SECTION 5: Complex combinations of AS_SEQUENCE and AS_SET (20 packets)
print("  Generating complex AS path combinations...")

# Generate some complex path segments
complex_segments = [
    # Complex path 1: Multiple sequences
    [
        create_as_sequence([main_src_as, main_src_as]),
        create_as_sequence([neighboring_asns[0], neighboring_asns[1]]),
        create_as_set([distant_asns[0], distant_asns[1]])
    ],
    # Complex path 2: Sequence + set + sequence
    [
        create_as_sequence([main_src_as]),
        create_as_set([neighboring_asns[0], neighboring_asns[2]]),
        create_as_sequence([distant_asns[0]])
    ],
    # Complex path 3: Multiple sets
    [
        create_as_sequence([main_src_as]),
        create_as_set([neighboring_asns[1]]),
        create_as_set([distant_asns[0], distant_asns[2]])
    ],
    # Complex path 4: Prepended + set
    [
        create_as_sequence([main_src_as, main_src_as, main_src_as]),
        create_as_set([neighboring_asns[0], neighboring_asns[3], distant_asns[1]])
    ]
]

for complex_index, segments in enumerate(complex_segments):
    for prefix_index, prefix in enumerate(selected_prefixes):
        # Create basic attributes
        origin, next_hop_attr, med_attr, local_pref_attr, communities_attr = create_base_update()
        
        # Create complex AS_PATH
        as_path_attr = create_as_path(segments)
        
        # Create and send update
        update = BGPHeader(type=2)/BGPUpdate()
        update.path_attr = [origin, as_path_attr, next_hop_attr, med_attr, local_pref_attr, communities_attr]
        update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id, 
                                                 f"complex path type {complex_index+1} for prefix {prefix}")
        
    # Add a KEEPALIVE after each complex path type
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 6: Combined attribute changes with AS path modifications (20 packets)
print("  Generating combined attribute changes...")
for med_value in [100]:
    for local_pref_value in [100, 200, 300]:
        # Skip some combinations to avoid too many packets
        if med_value == 100 and local_pref_value == 200:
            continue
            
        # Create basic attributes with varying MED and LOCAL_PREF
        origin = BGPPathAttr(type_flags=0x40, type_code=1)
        origin.attribute = BGPPAOrigin(origin=0)  # IGP=0
        
        next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
        next_hop_attr.attribute = BGPPANextHop(next_hop=src_ipv4)
        
        # Custom MED
        med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
        med_attr.attribute = BGPPAMultiExitDisc(med=med_value)
        
        # Custom LOCAL_PREF
        local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
        local_pref_attr.attribute = BGPPALocalPref(local_pref=local_pref_value)
        
        # Standard communities
        communities_list = []
        communities_list.append(BGPPACommunity(community=0xFFFFFF01))  # NO_EXPORT
        communities_list.append(BGPPACommunity(community=main_src_as<<16|200))
        communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
        communities_attr.attribute = communities_list
        
        # Create AS_PATH (different for each combination)
        prepend_count = (med_value // 50) + (local_pref_value // 100)  # 2-5 prepends
        prepended_as = [main_src_as] * prepend_count
        as_path_segment = create_as_sequence(prepended_as)
        as_path_attr = create_as_path([as_path_segment])
        
        # Select a prefix based on attributes
        prefix_index = (med_value // 50 + local_pref_value // 100) % len(selected_prefixes)
        prefix = selected_prefixes[prefix_index]
        
        # Create and send update
        update = BGPHeader(type=2)/BGPUpdate()
        update.path_attr = [origin, as_path_attr, next_hop_attr, med_attr, local_pref_attr, communities_attr]
        update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id, 
                                                 f"combined update (MED={med_value}, LOCAL_PREF={local_pref_value})")

# SECTION 7: Aggregation scenarios (20 packets)
print("  Generating aggregation scenarios...")

# Create common aggregation attributes
atomic_aggr_attr = BGPPathAttr(type_flags=0x40, type_code=6)
atomic_aggr_attr.attribute = BGPPAAtomicAggregate()

aggregator_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=7)
aggregator_attr.attribute = BGPPAAggregator(aggregator_asn=main_src_as, 
                                          speaker_address=ip_allocations[main_src_as]["router_id"])

for prefix in selected_prefixes:
    # Create a more specific prefix from the current prefix
    ip, mask = prefix.split('/')
    mask = int(mask)
    
    # Create one more specific prefix (e.g., /24 -> /25)
    if mask < 24:
        more_specific = f"{ip}/{mask+1}"
        
        # First announce the more specific prefix
        origin, next_hop_attr, med_attr, local_pref_attr, communities_attr = create_base_update()
        
        # Path for more specific prefix
        as_path_segment = create_as_sequence([main_src_as, neighboring_asns[0]])
        as_path_attr = create_as_path([as_path_segment])
        
        # Announce more specific
        more_specific_update = BGPHeader(type=2)/BGPUpdate()
        more_specific_update.path_attr = [origin, as_path_attr, next_hop_attr, med_attr, local_pref_attr, communities_attr]
        more_specific_update.nlri.append(BGPNLRI_IPv4(prefix=more_specific))
        
        src_ip_id, dst_ip_id = send_update_and_ack(more_specific_update, src_ip_id, dst_ip_id, 
                                                 f"more specific prefix {more_specific}")
        
        # Now announce the aggregate prefix
        # Use AS_SET for the aggregation
        sequence_segment = create_as_sequence([main_src_as])
        set_segment = create_as_set([main_src_as, neighboring_asns[0]])
        aggregate_path_attr = create_as_path([sequence_segment, set_segment])
        
        # Announce aggregate
        aggregate_update = BGPHeader(type=2)/BGPUpdate()
        aggregate_update.path_attr = [
            origin, 
            aggregate_path_attr, 
            next_hop_attr, 
            med_attr, 
            local_pref_attr,
            atomic_aggr_attr,
            aggregator_attr,
            communities_attr
        ]
        aggregate_update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        src_ip_id, dst_ip_id = send_update_and_ack(aggregate_update, src_ip_id, dst_ip_id, 
                                                 f"aggregate prefix {prefix}")
    
    # Add a KEEPALIVE after each aggregation pair
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# Final sequence of KEEPALIVEs (10 packets)
print("  Generating final KEEPALIVE exchanges...")
for i in range(5):
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# Update session sequence numbers for future reference
if session_key in global_bgp_sessions_ipv4:
    global_bgp_sessions_ipv4[session_key]["seq_a"] = seq_a_v4
    global_bgp_sessions_ipv4[session_key]["seq_b"] = seq_b_v4

print(f"  Added {len(pkts)} packets for extended AS_PATH modification scenario")


# ------------ Scenario 3: NEXT_HOP Changes------------
print("[*] Generating extended NEXT_HOP change scenario...")

# Get multiple prefixes from the main source AS
import random
available_prefixes = ip_allocations[main_src_as]["announced_prefixes"]
if len(available_prefixes) >= 5:
    selected_prefixes = random.sample(available_prefixes, 5)
else:
    # If not enough prefixes available, duplicate some
    selected_prefixes = available_prefixes * (5 // len(available_prefixes) + 1)
    selected_prefixes = selected_prefixes[:5]

target_prefix = selected_prefixes[0]
print(f"  Using prefix {target_prefix} and others for NEXT_HOP change scenario")

# Get session parameters from the main AS pair
session_key = (main_src_as, main_dst_as)
if session_key in global_bgp_sessions_ipv4:
    # Get IPv4 session info
    session_info = global_bgp_sessions_ipv4[session_key]
    src_ipv4 = session_info["src_ipv4"]
    dst_ipv4 = session_info["dst_ipv4"]
    src_mac = session_info["src_mac"]
    dst_mac = session_info["dst_mac"]
    sport = session_info["sport"]
    dport = session_info["dport"]
    seq_a_v4 = session_info["seq_a"]
    seq_b_v4 = session_info["seq_b"]
else:
    print(f"  Warning: No established session found for AS{main_src_as}-AS{main_dst_as}")
    src_ipv4 = ip_allocations[main_src_as]["interfaces"][main_dst_as]
    dst_ipv4 = ip_allocations[main_dst_as]["interfaces"][main_src_as]
    # Generate consistent MAC addresses
    src_mac = "00:00:%02x:%02x:%02x:%02x" % tuple(map(int, src_ipv4.split('.')))
    dst_mac = "00:00:%02x:%02x:%02x:%02x" % tuple(map(int, dst_ipv4.split('.')))
    sport = random.randint(30000, 65000)
    dport = BGP_PORT
    seq_a_v4 = random.randint(1000000, 2000000)
    seq_b_v4 = random.randint(3000000, 4000000)

# Use randomized IP ID values
src_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
dst_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])

# Helper function to create a base BGP update with common attributes
def create_base_update(origin_value=0):
    # Ensure origin attribute is defined
    origin = BGPPathAttr(type_flags=0x40, type_code=1)
    origin.attribute = BGPPAOrigin(origin=origin_value)  # IGP=0, EGP=1, INCOMPLETE=2

    # Regular AS_PATH attribute
    as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
    as_path_segment = BGPPAASPath()
    segment = BGPPAASPath.ASPathSegment(
        segment_type=2,  # AS_SEQUENCE
        segment_length=1,
        segment_value=[main_src_as]
    )
    as_path_segment.segments = [segment]
    as_path_attr.attribute = as_path_segment

    # LOCAL_PREF attribute - we're keeping this constant as per user feedback
    local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
    local_pref_attr.attribute = BGPPALocalPref(local_pref=200)

    return origin, as_path_attr, local_pref_attr

# Helper function to send a BGP update and receive ACK
def send_update_and_ack(update, src_id, dst_id, description=""):
    global seq_a_v4, seq_b_v4, pkts
    
    # Send update
    update_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/update
    if len(update_pkt) < 60:
        pad_len = 60 - len(update_pkt)
        update_pkt = update_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(update_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    seq_a_v4 += len(update)
    
    # Send ACK
    ack_pkt = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
    if len(ack_pkt) < 60:
        pad_len = 60 - len(ack_pkt)
        ack_pkt = ack_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(ack_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    
    if description:
        print(f"    - Sent {description}")
    
    return src_id + 1, dst_id + 1

# Helper function to send a KEEPALIVE and receive ACK
def send_keepalive_and_ack(src_id, dst_id):
    global seq_a_v4, seq_b_v4, pkts
    
    # Create KEEPALIVE
    keepalive = BGPHeader(type=4)  # KEEPALIVE type=4
    
    # Send KEEPALIVE
    keepalive_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/keepalive
    if len(keepalive_pkt) < 60:
        pad_len = 60 - len(keepalive_pkt)
        keepalive_pkt = keepalive_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(keepalive_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    seq_a_v4 += len(keepalive)
    
    # Send ACK for KEEPALIVE
    keepalive_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
    if len(keepalive_ack) < 60:
        pad_len = 60 - len(keepalive_ack)
        keepalive_ack = keepalive_ack/Padding(load=b'\x00' * pad_len)
    pkts.append(keepalive_ack)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    
    return src_id + 1, dst_id + 1

# Generate a pool of next hop addresses for various scenarios
next_hop_pool = []

# 1. Add the router ID as a next hop
next_hop_pool.append(ip_allocations[main_src_as]["router_id"])
print(f"  Added router ID {ip_allocations[main_src_as]['router_id']} to next hop pool")

# 2. Add a few other interfaces from the same AS (if available)
for neighbor_as, interface_ip in ip_allocations[main_src_as]["interfaces"].items():
    if neighbor_as != main_dst_as:  # Skip the interface to the destination AS
        next_hop_pool.append(interface_ip)
        print(f"  Added interface IP {interface_ip} to next hop pool")
        if len(next_hop_pool) >= 3:
            break  # Limit to 3 additional interfaces

# 3. Add third-party next hops from other ASes
third_party_next_hops = []
for asn in topology:
    if asn != main_src_as and asn != main_dst_as:
        third_party_next_hops.append(ip_allocations[asn]["router_id"])
        print(f"  Added third-party next hop {ip_allocations[asn]['router_id']} from AS{asn}")
        if len(third_party_next_hops) >= 3:
            break  # Limit to 3 third-party ASes

# 4. Create some private IP addresses for potentially invalid next hops
private_next_hops = [
    "10.0.0.1", "10.1.1.1", "10.2.2.2",
    "172.16.0.1", "172.16.1.1",
    "192.168.0.1", "192.168.1.1"
]

# Add a few KEEPALIVE exchanges to start
print("  Generating initial KEEPALIVE exchanges...")
for i in range(4):
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 1: Basic next hop changes (25 packets)
print("  Generating basic next hop changes...")
for prefix in selected_prefixes:
    for next_hop in next_hop_pool:
        # Create basic attributes
        origin, as_path_attr, local_pref_attr = create_base_update()
        
        # Create next hop attribute
        next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
        next_hop_attr.attribute = BGPPANextHop(next_hop=next_hop)
        
        # Create update
        update = BGPHeader(type=2)/BGPUpdate()
        update.path_attr = [origin, as_path_attr, next_hop_attr, local_pref_attr]
        update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        # Send update and get ACK
        src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id, 
                                                 f"update for prefix {prefix} with next hop {next_hop}")
    
    # Add a KEEPALIVE after each prefix
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 2: Third-party next hops (15 packets)
print("  Generating third-party next hop updates...")
for prefix in selected_prefixes[:3]:  # Use first 3 prefixes
    for next_hop in third_party_next_hops:
        # Create basic attributes
        origin, as_path_attr, local_pref_attr = create_base_update()
        
        # Create next hop attribute with third-party next hop
        next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
        next_hop_attr.attribute = BGPPANextHop(next_hop=next_hop)
        
        # Create update
        update = BGPHeader(type=2)/BGPUpdate()
        update.path_attr = [origin, as_path_attr, next_hop_attr, local_pref_attr]
        update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        # Send update and get ACK
        src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id, 
                                                 f"update for prefix {prefix} with third-party next hop {next_hop}")
    
    # Add a KEEPALIVE after each prefix
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)


# SECTION 4: Multiple prefixes with same next hop (20 packets)
print("  Generating multi-prefix updates with same next hop...")
for next_hop in next_hop_pool + third_party_next_hops[:1]:  # Use all normal next hops and 1 third-party
    # Create update with multiple prefixes
    multi_prefix_update = BGPHeader(type=2)/BGPUpdate()
    
    # Create basic attributes
    origin, as_path_attr, local_pref_attr = create_base_update()
    
    # Create next hop attribute
    next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
    next_hop_attr.attribute = BGPPANextHop(next_hop=next_hop)
    
    # Add path attributes
    multi_prefix_update.path_attr = [origin, as_path_attr, next_hop_attr, local_pref_attr]
    
    # Add multiple prefixes
    for prefix in selected_prefixes:
        multi_prefix_update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
    
    # Send update and get ACK
    src_ip_id, dst_ip_id = send_update_and_ack(multi_prefix_update, src_ip_id, dst_ip_id, 
                                             f"multi-prefix update with next hop {next_hop}")
    
    # Add a KEEPALIVE after each update
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 5: Withdrawals and re-announcements with different next hops (25 packets)
print("  Generating withdrawals and re-announcements with different next hops...")
for prefix in selected_prefixes:
    # Create withdrawal
    withdrawal = BGPHeader(type=2)/BGPUpdate()
    withdrawal.withdrawn_routes = [BGPNLRI_IPv4(prefix=prefix)]
    
    # Send withdrawal and get ACK
    src_ip_id, dst_ip_id = send_update_and_ack(withdrawal, src_ip_id, dst_ip_id, 
                                             f"withdrawal of prefix {prefix}")
    
    # Re-announce with different next hop
    for next_hop in next_hop_pool[:1]:  # Just use the first next hop from pool
        # Create basic attributes
        origin, as_path_attr, local_pref_attr = create_base_update()
        
        # Create next hop attribute
        next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
        next_hop_attr.attribute = BGPPANextHop(next_hop=next_hop)
        
        # Create update
        reannounce = BGPHeader(type=2)/BGPUpdate()
        reannounce.path_attr = [origin, as_path_attr, next_hop_attr, local_pref_attr]
        reannounce.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        # Send re-announcement and get ACK
        src_ip_id, dst_ip_id = send_update_and_ack(reannounce, src_ip_id, dst_ip_id, 
                                                 f"re-announcement of {prefix} with next hop {next_hop}")
        
        # Withdraw again
        src_ip_id, dst_ip_id = send_update_and_ack(withdrawal, src_ip_id, dst_ip_id, 
                                                 f"second withdrawal of prefix {prefix}")
        
        # Re-announce with a different next hop
        for third_party_next_hop in third_party_next_hops[:1]:  # Use first third-party next hop
            # Create next hop attribute
            third_party_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
            third_party_hop_attr.attribute = BGPPANextHop(next_hop=third_party_next_hop)
            
            # Create update
            reannounce2 = BGPHeader(type=2)/BGPUpdate()
            reannounce2.path_attr = [origin, as_path_attr, third_party_hop_attr, local_pref_attr]
            reannounce2.nlri.append(BGPNLRI_IPv4(prefix=prefix))
            
            # Send second re-announcement and get ACK
            src_ip_id, dst_ip_id = send_update_and_ack(reannounce2, src_ip_id, dst_ip_id, 
                                                     f"second re-announcement of {prefix} with third-party next hop")
    
    # Add a KEEPALIVE after each prefix cycle
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 6: AS Path variations with different next hops (15 packets)
print("  Generating AS path variations with different next hops...")
# Create a few different AS paths
as_paths = [
    [main_src_as],  # No prepending
    [main_src_as, main_src_as],  # Single prepend
    [main_src_as, main_src_as, main_src_as]  # Double prepend
]

for as_path in as_paths:
    for next_hop in next_hop_pool[:1] + third_party_next_hops[:1]:  # One normal + one third party
        # Create basic attributes
        origin = BGPPathAttr(type_flags=0x40, type_code=1)
        origin.attribute = BGPPAOrigin(origin=0)  # IGP
        
        # Create custom AS path
        as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
        as_path_segment = BGPPAASPath()
        segment = BGPPAASPath.ASPathSegment(
            segment_type=2,  # AS_SEQUENCE
            segment_length=len(as_path),
            segment_value=as_path
        )
        as_path_segment.segments = [segment]
        as_path_attr.attribute = as_path_segment
        
        # Create next hop attribute
        next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
        next_hop_attr.attribute = BGPPANextHop(next_hop=next_hop)
        
        # Create LOCAL_PREF
        local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
        local_pref_attr.attribute = BGPPALocalPref(local_pref=200)
        
        # Create update
        update = BGPHeader(type=2)/BGPUpdate()
        update.path_attr = [origin, as_path_attr, next_hop_attr, local_pref_attr]
        
        # Add prefixes - use first 3 prefixes
        for prefix in selected_prefixes[:3]:
            update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        # Send update and get ACK
        src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id, 
                                                 f"AS path {as_path} with next hop {next_hop}")
    
    # Add a KEEPALIVE after each AS path
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 7: NEXT_HOP with different ORIGIN values (15 packets)
print("  Generating next hop changes with different ORIGIN values...")
for origin_value in [0, 1, 2]:  # IGP, EGP, INCOMPLETE
    for next_hop in next_hop_pool[:1] + third_party_next_hops[:1]:  # One normal + one third party
        # Create basic attributes with varying ORIGIN
        origin = BGPPathAttr(type_flags=0x40, type_code=1)
        origin.attribute = BGPPAOrigin(origin=origin_value)
        
        # Create standard AS path
        as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
        as_path_segment = BGPPAASPath()
        segment = BGPPAASPath.ASPathSegment(
            segment_type=2,
            segment_length=1,
            segment_value=[main_src_as]
        )
        as_path_segment.segments = [segment]
        as_path_attr.attribute = as_path_segment
        
        # Create next hop attribute
        next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
        next_hop_attr.attribute = BGPPANextHop(next_hop=next_hop)
        
        # Create LOCAL_PREF
        local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
        local_pref_attr.attribute = BGPPALocalPref(local_pref=200)
        
        # Create update
        update = BGPHeader(type=2)/BGPUpdate()
        update.path_attr = [origin, as_path_attr, next_hop_attr, local_pref_attr]
        
        # Add prefixes - use first 2 prefixes
        for prefix in selected_prefixes[:2]:
            update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        # Send update and get ACK
        origin_names = {0: "IGP", 1: "EGP", 2: "INCOMPLETE"}
        src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id, 
                                                 f"ORIGIN {origin_names[origin_value]} with next hop {next_hop}")
    
    # Add a KEEPALIVE after each origin type
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# Final sequence of KEEPALIVEs (10 packets)
print("  Generating final KEEPALIVE exchanges...")
for i in range(5):
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# Update session sequence numbers for future reference
if session_key in global_bgp_sessions_ipv4:
    global_bgp_sessions_ipv4[session_key]["seq_a"] = seq_a_v4
    global_bgp_sessions_ipv4[session_key]["seq_b"] = seq_b_v4

print(f"  Added {len(pkts)} packets for extended NEXT_HOP change scenario")


# ------------ Scenario 4: LOCAL_PREF Modifications ------------
print("[*] Generating extended LOCAL_PREF modification scenario...")

# Get multiple prefixes from the main source AS
import random
available_prefixes = ip_allocations[main_src_as]["announced_prefixes"]
if len(available_prefixes) >= 5:
    selected_prefixes = random.sample(available_prefixes, 5)
else:
    # If not enough prefixes available, duplicate some
    selected_prefixes = available_prefixes * (5 // len(available_prefixes) + 1)
    selected_prefixes = selected_prefixes[:5]

target_prefix = selected_prefixes[0]
print(f"  Using prefix {target_prefix} and others for LOCAL_PREF modification scenario")

# Get session parameters from the main AS pair
session_key = (main_src_as, main_dst_as)
if session_key in global_bgp_sessions_ipv4:
    # Get IPv4 session info
    session_info = global_bgp_sessions_ipv4[session_key]
    src_ipv4 = session_info["src_ipv4"]
    dst_ipv4 = session_info["dst_ipv4"]
    src_mac = session_info["src_mac"]
    dst_mac = session_info["dst_mac"]
    sport = session_info["sport"]
    dport = session_info["dport"]
    seq_a_v4 = session_info["seq_a"]
    seq_b_v4 = session_info["seq_b"]
else:
    print(f"  Warning: No established session found for AS{main_src_as}-AS{main_dst_as}")
    src_ipv4 = ip_allocations[main_src_as]["interfaces"][main_dst_as]
    dst_ipv4 = ip_allocations[main_dst_as]["interfaces"][main_src_as]
    # Generate consistent MAC addresses
    src_mac = "00:00:%02x:%02x:%02x:%02x" % tuple(map(int, src_ipv4.split('.')))
    dst_mac = "00:00:%02x:%02x:%02x:%02x" % tuple(map(int, dst_ipv4.split('.')))
    sport = random.randint(30000, 65000)
    dport = BGP_PORT
    seq_a_v4 = random.randint(1000000, 2000000)
    seq_b_v4 = random.randint(3000000, 4000000)

# Use randomized IP ID values
src_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
dst_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])

# Helper function to send a BGP update and receive ACK
def send_update_and_ack(update, src_id, dst_id, description=""):
    global seq_a_v4, seq_b_v4, pkts
    
    # Send update
    update_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/update
    if len(update_pkt) < 60:
        pad_len = 60 - len(update_pkt)
        update_pkt = update_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(update_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    seq_a_v4 += len(update)
    
    # Send ACK
    ack_pkt = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
    if len(ack_pkt) < 60:
        pad_len = 60 - len(ack_pkt)
        ack_pkt = ack_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(ack_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    
    if description:
        print(f"    - Sent {description}")
    
    return src_id + 1, dst_id + 1

# Helper function to send a KEEPALIVE and receive ACK
def send_keepalive_and_ack(src_id, dst_id):
    global seq_a_v4, seq_b_v4, pkts
    
    # Create KEEPALIVE
    keepalive = BGPHeader(type=4)  # KEEPALIVE type=4
    
    # Send KEEPALIVE
    keepalive_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/keepalive
    if len(keepalive_pkt) < 60:
        pad_len = 60 - len(keepalive_pkt)
        keepalive_pkt = keepalive_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(keepalive_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    seq_a_v4 += len(keepalive)
    
    # Send ACK for KEEPALIVE
    keepalive_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
    if len(keepalive_ack) < 60:
        pad_len = 60 - len(keepalive_ack)
        keepalive_ack = keepalive_ack/Padding(load=b'\x00' * pad_len)
    pkts.append(keepalive_ack)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    
    return src_id + 1, dst_id + 1

# Create standard attributes that stay the same for all updates
origin = BGPPathAttr(type_flags=0x40, type_code=1)
origin.attribute = BGPPAOrigin(origin=0)

as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
as_path_segment = BGPPAASPath()
segment = BGPPAASPath.ASPathSegment(
    segment_type=2,  # AS_SEQUENCE
    segment_length=1,
    segment_value=[main_src_as]
)
as_path_segment.segments = [segment]
as_path_attr.attribute = as_path_segment

next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
next_hop_attr.attribute = BGPPANextHop(next_hop=src_ipv4)

med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
med_attr.attribute = BGPPAMultiExitDisc(med=100)

communities_list = []
communities_list.append(BGPPACommunity(community=0xFFFFFF01))  # NO_EXPORT
communities_list.append(BGPPACommunity(community=main_src_as<<16|200))
communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
communities_attr.attribute = communities_list

# Add a few KEEPALIVEs to start
print("  Sending initial KEEPALIVE messages...")
for i in range(3):
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 1: Gradually changing LOCAL_PREF values for a single prefix
print("  Generating gradual LOCAL_PREF changes for prefix", target_prefix)

# Create a list of LOCAL_PREF values to use
local_pref_values = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

# Send updates for each LOCAL_PREF value
for local_pref_value in local_pref_values:
    # Create LOCAL_PREF attribute
    local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
    local_pref_attr.attribute = BGPPALocalPref(local_pref=local_pref_value)
    
    # Create update
    update = BGPHeader(type=2)/BGPUpdate()
    update.path_attr = [
        origin,
        as_path_attr,
        next_hop_attr,
        med_attr,
        local_pref_attr,
        communities_attr,
    ]
    update.nlri.append(BGPNLRI_IPv4(prefix=target_prefix))
    
    # Send update and get ACK
    src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id,
                                             f"update for prefix {target_prefix} with LOCAL_PREF {local_pref_value}")
    
    # Add a KEEPALIVE message after every 3 updates
    if local_pref_values.index(local_pref_value) % 3 == 2:
        src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 2: Announcing multiple prefixes with different LOCAL_PREF values
print("  Announcing multiple prefixes with different LOCAL_PREF values...")
for prefix in selected_prefixes:
    # Choose different LOCAL_PREF values for each prefix
    for local_pref_value in [100, 200, 300, 400]:
        # Create LOCAL_PREF attribute
        local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
        local_pref_attr.attribute = BGPPALocalPref(local_pref=local_pref_value)
        
        # Create update
        update = BGPHeader(type=2)/BGPUpdate()
        update.path_attr = [
            origin,
            as_path_attr,
            next_hop_attr,
            med_attr,
            local_pref_attr,
            communities_attr,
        ]
        update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        # Send update and get ACK
        src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id,
                                                 f"update for prefix {prefix} with LOCAL_PREF {local_pref_value}")
    
    # Add a KEEPALIVE after each prefix
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 3: Announcing multiple prefixes with the same LOCAL_PREF in a single update
print("  Announcing multiple prefixes with the same LOCAL_PREF in single updates...")
for local_pref_value in [150, 250, 350]:
    # Create LOCAL_PREF attribute
    local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
    local_pref_attr.attribute = BGPPALocalPref(local_pref=local_pref_value)
    
    # Create update with multiple prefixes
    update = BGPHeader(type=2)/BGPUpdate()
    update.path_attr = [
        origin,
        as_path_attr,
        next_hop_attr,
        med_attr,
        local_pref_attr,
        communities_attr,
    ]
    
    # Add all prefixes to this update
    for prefix in selected_prefixes:
        update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
    
    # Send update and get ACK
    src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id,
                                             f"multi-prefix update with LOCAL_PREF {local_pref_value}")
    
    # Add a KEEPALIVE after each multi-prefix update
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 4: Withdraw and re-announce with different LOCAL_PREF values
print("  Generating withdrawals and re-announcements with different LOCAL_PREF values...")
for prefix in selected_prefixes:
    # First send a withdrawal
    withdrawal = BGPHeader(type=2)/BGPUpdate()
    withdrawal.withdrawn_routes = [BGPNLRI_IPv4(prefix=prefix)]
    
    # Send withdrawal and get ACK
    src_ip_id, dst_ip_id = send_update_and_ack(withdrawal, src_ip_id, dst_ip_id,
                                             f"withdrawal of prefix {prefix}")
    
    # Then re-announce with a high LOCAL_PREF
    high_local_pref = BGPPathAttr(type_flags=0x40, type_code=5)
    high_local_pref.attribute = BGPPALocalPref(local_pref=400)
    
    high_update = BGPHeader(type=2)/BGPUpdate()
    high_update.path_attr = [
        origin,
        as_path_attr,
        next_hop_attr,
        med_attr,
        high_local_pref,
        communities_attr,
    ]
    high_update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
    
    # Send high LOCAL_PREF update and get ACK
    src_ip_id, dst_ip_id = send_update_and_ack(high_update, src_ip_id, dst_ip_id,
                                             f"re-announcement of prefix {prefix} with LOCAL_PREF 400")
    
    # Then withdraw again
    src_ip_id, dst_ip_id = send_update_and_ack(withdrawal, src_ip_id, dst_ip_id,
                                             f"second withdrawal of prefix {prefix}")
    
    # Then re-announce with a low LOCAL_PREF
    low_local_pref = BGPPathAttr(type_flags=0x40, type_code=5)
    low_local_pref.attribute = BGPPALocalPref(local_pref=50)
    
    low_update = BGPHeader(type=2)/BGPUpdate()
    low_update.path_attr = [
        origin,
        as_path_attr,
        next_hop_attr,
        med_attr,
        low_local_pref,
        communities_attr,
    ]
    low_update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
    
    # Send low LOCAL_PREF update and get ACK
    src_ip_id, dst_ip_id = send_update_and_ack(low_update, src_ip_id, dst_ip_id,
                                             f"re-announcement of prefix {prefix} with LOCAL_PREF 50")
    
    # Add a KEEPALIVE after each cycle
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 5: LOCAL_PREF with AS path variations
print("  Generating LOCAL_PREF with AS path variations...")

# Create different AS paths
as_paths = [
    [main_src_as],                      # No prepending
    [main_src_as, main_src_as],         # 1x prepend
    [main_src_as, main_src_as, main_src_as],  # 2x prepend
]

for as_path_values in as_paths:
    for local_pref_value in [100, 300]:
        # Create custom AS_PATH
        custom_as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
        custom_as_path_segment = BGPPAASPath()
        custom_segment = BGPPAASPath.ASPathSegment(
            segment_type=2,  # AS_SEQUENCE
            segment_length=len(as_path_values),
            segment_value=as_path_values
        )
        custom_as_path_segment.segments = [custom_segment]
        custom_as_path_attr.attribute = custom_as_path_segment
        
        # Create LOCAL_PREF attribute
        local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
        local_pref_attr.attribute = BGPPALocalPref(local_pref=local_pref_value)
        
        # Create update
        update = BGPHeader(type=2)/BGPUpdate()
        update.path_attr = [
            origin,
            custom_as_path_attr,
            next_hop_attr,
            med_attr,
            local_pref_attr,
            communities_attr,
        ]
        
        # Add selected prefixes
        for prefix in selected_prefixes[:2]:  # Use first 2 prefixes
            update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        # Send update and get ACK
        prepend_count = len(as_path_values) - 1
        src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id,
                                                 f"update with {prepend_count}x prepend and LOCAL_PREF {local_pref_value}")
    
    # Add a KEEPALIVE after each AS path
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 6: LOCAL_PREF oscillation to simulate route flapping
print("  Generating LOCAL_PREF oscillation...")

# Oscillate between high and low LOCAL_PREF values
local_pref_oscillation = [500, 100, 500, 100, 500, 100]

for cycle, local_pref_value in enumerate(local_pref_oscillation):
    # Create LOCAL_PREF attribute
    local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
    local_pref_attr.attribute = BGPPALocalPref(local_pref=local_pref_value)
    
    # Create update
    update = BGPHeader(type=2)/BGPUpdate()
    update.path_attr = [
        origin,
        as_path_attr,
        next_hop_attr,
        med_attr,
        local_pref_attr,
        communities_attr,
    ]
    
    # Add a prefix - alternate between two prefixes
    prefix_index = cycle % 2
    update.nlri.append(BGPNLRI_IPv4(prefix=selected_prefixes[prefix_index]))
    
    # Send update and get ACK
    src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id,
                                             f"oscillation cycle {cycle+1} with LOCAL_PREF {local_pref_value}")
    
    # Add a KEEPALIVE after every second cycle
    if cycle % 2 == 1:
        src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 7: LOCAL_PREF with different ORIGIN values
print("  Generating LOCAL_PREF with different ORIGIN values...")

for origin_value in [0, 1, 2]:  # IGP, EGP, INCOMPLETE
    for local_pref_value in [150, 350]:
        # Create ORIGIN attribute
        custom_origin = BGPPathAttr(type_flags=0x40, type_code=1)
        custom_origin.attribute = BGPPAOrigin(origin=origin_value)
        
        # Create LOCAL_PREF attribute
        local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
        local_pref_attr.attribute = BGPPALocalPref(local_pref=local_pref_value)
        
        # Create update
        update = BGPHeader(type=2)/BGPUpdate()
        update.path_attr = [
            custom_origin,
            as_path_attr,
            next_hop_attr,
            med_attr,
            local_pref_attr,
            communities_attr,
        ]
        
        # Add all prefixes to this update
        for prefix in selected_prefixes[:2]:  # Use first 2 prefixes
            update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        # Send update and get ACK
        origin_names = {0: "IGP", 1: "EGP", 2: "INCOMPLETE"}
        src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id,
                                                 f"update with ORIGIN {origin_names[origin_value]} and LOCAL_PREF {local_pref_value}")
    
    # Add a KEEPALIVE after each origin type
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# Add some final KEEPALIVEs
print("  Sending final KEEPALIVE messages...")
for i in range(3):
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# Update session sequence numbers for future reference
if session_key in global_bgp_sessions_ipv4:
    global_bgp_sessions_ipv4[session_key]["seq_a"] = seq_a_v4
    global_bgp_sessions_ipv4[session_key]["seq_b"] = seq_b_v4

print(f"  Added {len(pkts)} packets for extended LOCAL_PREF modification scenario")


# ------------ Scenario 5: Community Changes ------------
print("[*] Generating COMMUNITIES change scenario...")

# Get a random prefix from the main source AS
import random
target_prefix = random.choice(ip_allocations[main_src_as]["announced_prefixes"])
print(f"  Using prefix {target_prefix} for COMMUNITIES change scenario")

# Get session parameters from the main AS pair
src_ipv4 = ip_allocations[main_src_as]["interfaces"][main_dst_as]
dst_ipv4 = ip_allocations[main_dst_as]["interfaces"][main_src_as]

src_ip_id += 1
# Check we're still in range
if src_ip_id > NORMAL_TRAFFIC_ID_RANGE[1]:
    src_ip_id = NORMAL_TRAFFIC_ID_RANGE[0]

dst_ip_id += 1
# Check we're still in range
if dst_ip_id > NORMAL_TRAFFIC_ID_RANGE[1]:
    dst_ip_id = NORMAL_TRAFFIC_ID_RANGE[0]

# Ensure origin attribute is defined (IGP = 0)
origin = BGPPathAttr(type_flags=0x40, type_code=1)
origin.attribute = BGPPAOrigin(origin=0)

# Regular AS_PATH attribute
as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
as_path_segment = BGPPAASPath()
segment = BGPPAASPath.ASPathSegment(
    segment_type=2,  # AS_SEQUENCE
    segment_length=1,
    segment_value=[main_src_as]
)
as_path_segment.segments = [segment]
as_path_attr.attribute = as_path_segment

# NEXT_HOP attribute
next_hop_attr_v4 = BGPPathAttr(type_flags=0x40, type_code=3)
next_hop_attr_v4.attribute = BGPPANextHop(next_hop=src_ipv4)

# MED attribute
med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
med_attr.attribute = BGPPAMultiExitDisc(med=100)

# LOCAL_PREF attribute
local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
local_pref_attr.attribute = BGPPALocalPref(local_pref=200)

# Create update with well-known communities
communities_change_update = BGPHeader(type=2)/BGPUpdate()

# Create a new communities list with well-known communities
new_communities_list = []
# Add well-known communities
new_communities_list.append(BGPPACommunity(community=0xFFFFFF02))  # NO_ADVERTISE
new_communities_list.append(BGPPACommunity(community=0xFFFFFF03))  # NO_EXPORT_SUBCONFED
# Add a custom community
new_communities_list.append(BGPPACommunity(community=main_src_as<<16|300))  # ASN:300

# Create the new communities path attribute
new_communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
new_communities_attr.attribute = new_communities_list

# Use the well-known communities with the prefix
communities_change_update.path_attr = [
    origin,               # IGP origin
    as_path_attr,         # Normal AS path
    next_hop_attr_v4,     # Original next hop
    med_attr,             # Original MED
    local_pref_attr,      # Original LOCAL_PREF
    new_communities_attr, # Well-known communities
]

# Add the prefix to the update
communities_change_update.nlri.append(BGPNLRI_IPv4(prefix=target_prefix))

# Send communities change update
communities_change_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/communities_change_update
if len(communities_change_pkt) < 60:
    pad_len = 60 - len(communities_change_pkt)
    communities_change_pkt = communities_change_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(communities_change_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_a_v4 += len(communities_change_update)

# ACK for communities change
communities_change_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(communities_change_ack) < 60:
    pad_len = 60 - len(communities_change_ack)
    communities_change_ack = communities_change_ack/Padding(load=b'\x00' * pad_len)
pkts.append(communities_change_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
dst_ip_id += 1  # Increment for next packet

# Create update with regional communities
regional_communities_update = BGPHeader(type=2)/BGPUpdate()

# Create a new communities list with regional communities
regional_communities_list = []
# Add regional communities (format: <ASN>:<region_code>)
regional_communities_list.append(BGPPACommunity(community=main_src_as<<16|1000))  # Region 1000
regional_communities_list.append(BGPPACommunity(community=main_src_as<<16|2000))  # Region 2000
# Add action communities (format: <ASN>:<action_code>)
regional_communities_list.append(BGPPACommunity(community=main_src_as<<16|100))   # Prepend once
regional_communities_list.append(BGPPACommunity(community=main_src_as<<16|3000))  # Don't advertise to peers

# Create the regional communities path attribute
regional_communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
regional_communities_attr.attribute = regional_communities_list

# Use the regional communities with the same prefix
regional_communities_update.path_attr = [
    origin,                  # IGP origin
    as_path_attr,            # Normal AS path
    next_hop_attr_v4,        # Original next hop
    med_attr,                # Original MED
    local_pref_attr,         # Original LOCAL_PREF
    regional_communities_attr, # Regional communities
]

# Add the same prefix to the update
regional_communities_update.nlri.append(BGPNLRI_IPv4(prefix=target_prefix))

# Send regional communities update
regional_communities_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+1)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/regional_communities_update
if len(regional_communities_pkt) < 60:
    pad_len = 60 - len(regional_communities_pkt)
    regional_communities_pkt = regional_communities_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(regional_communities_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_a_v4 += len(regional_communities_update)

# ACK for regional communities
regional_communities_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(regional_communities_ack) < 60:
    pad_len = 60 - len(regional_communities_ack)
    regional_communities_ack = regional_communities_ack/Padding(load=b'\x00' * pad_len)
pkts.append(regional_communities_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay

print(f"  Added {4} packets for COMMUNITIES change scenario")


# ------------ Scenario 6: Duplicate Announcements ------------

print("[*] Generating extended duplicate announcement scenario...")

# Get multiple random prefixes from the main source AS for duplicate announcements
import random
available_prefixes = ip_allocations[main_src_as]["announced_prefixes"]
if len(available_prefixes) >= 5:
    # Select 5 different prefixes if available
    selected_prefixes = random.sample(available_prefixes, 5)
else:
    # Use all available prefixes, potentially with repetition
    selected_prefixes = available_prefixes * (5 // len(available_prefixes) + 1)
    selected_prefixes = selected_prefixes[:5]

print(f"  Using {len(selected_prefixes)} prefixes for duplicate announcements")

# Get session parameters from the main AS pair
session_key = (main_src_as, main_dst_as)
if session_key in global_bgp_sessions_ipv4:
    # Get IPv4 session info
    session_info = global_bgp_sessions_ipv4[session_key]
    src_ipv4 = session_info["src_ipv4"]
    dst_ipv4 = session_info["dst_ipv4"]
    src_mac = session_info["src_mac"]
    dst_mac = session_info["dst_mac"]
    sport = session_info["sport"]
    dport = session_info["dport"]
    seq_a_v4 = session_info["seq_a"]
    seq_b_v4 = session_info["seq_b"]
else:
    print(f"  Warning: No established session found for AS{main_src_as}-AS{main_dst_as}")
    src_ipv4 = ip_allocations[main_src_as]["interfaces"][main_dst_as]
    dst_ipv4 = ip_allocations[main_dst_as]["interfaces"][main_src_as]
    # Generate consistent MAC addresses
    src_mac = "00:00:%02x:%02x:%02x:%02x" % tuple(map(int, src_ipv4.split('.')))
    dst_mac = "00:00:%02x:%02x:%02x:%02x" % tuple(map(int, dst_ipv4.split('.')))
    sport = random.randint(30000, 65000)
    dport = BGP_PORT
    seq_a_v4 = random.randint(1000000, 2000000)
    seq_b_v4 = random.randint(3000000, 4000000)

# Use randomized IP ID values
src_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
dst_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])

# Helper function to send a BGP update and receive ACK
def send_update_and_ack(update, src_id, dst_id, description=""):
    global seq_a_v4, seq_b_v4, pkts
    
    # Send update
    update_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/update
    if len(update_pkt) < 60:
        pad_len = 60 - len(update_pkt)
        update_pkt = update_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(update_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    seq_a_v4 += len(update)
    
    # Send ACK
    ack_pkt = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
    if len(ack_pkt) < 60:
        pad_len = 60 - len(ack_pkt)
        ack_pkt = ack_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(ack_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    
    if description:
        print(f"    - Sent {description}")
    
    return src_id + 1, dst_id + 1

# Helper function to send a KEEPALIVE and receive ACK
def send_keepalive_and_ack(src_id, dst_id):
    global seq_a_v4, seq_b_v4, pkts
    
    # Create KEEPALIVE
    keepalive = BGPHeader(type=4)  # KEEPALIVE type=4
    
    # Send KEEPALIVE
    keepalive_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/keepalive
    if len(keepalive_pkt) < 60:
        pad_len = 60 - len(keepalive_pkt)
        keepalive_pkt = keepalive_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(keepalive_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    seq_a_v4 += len(keepalive)
    
    # Send ACK for KEEPALIVE
    keepalive_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
    if len(keepalive_ack) < 60:
        pad_len = 60 - len(keepalive_ack)
        keepalive_ack = keepalive_ack/Padding(load=b'\x00' * pad_len)
    pkts.append(keepalive_ack)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    
    return src_id + 1, dst_id + 1

# Function to create standard BGP path attributes
def create_standard_attributes():
    # Ensure origin attribute is defined (IGP = 0)
    origin = BGPPathAttr(type_flags=0x40, type_code=1)
    origin.attribute = BGPPAOrigin(origin=0)
    
    # Regular AS_PATH attribute
    as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
    as_path_segment = BGPPAASPath()
    segment = BGPPAASPath.ASPathSegment(
        segment_type=2,  # AS_SEQUENCE
        segment_length=1,
        segment_value=[main_src_as]
    )
    as_path_segment.segments = [segment]
    as_path_attr.attribute = as_path_segment
    
    # NEXT_HOP attribute
    next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
    next_hop_attr.attribute = BGPPANextHop(next_hop=src_ipv4)
    
    # MED attribute
    med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
    med_attr.attribute = BGPPAMultiExitDisc(med=100)
    
    # LOCAL_PREF attribute
    local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
    local_pref_attr.attribute = BGPPALocalPref(local_pref=200)
    
    # COMMUNITIES attribute
    communities_list = []
    communities_list.append(BGPPACommunity(community=0xFFFFFF01))  # NO_EXPORT
    communities_list.append(BGPPACommunity(community=main_src_as<<16|200))
    communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
    communities_attr.attribute = communities_list
    
    return [origin, as_path_attr, next_hop_attr, med_attr, local_pref_attr, communities_attr]

# SECTION 1: Initial KEEPALIVE exchanges
print("  Sending initial KEEPALIVE messages...")
for i in range(3):
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 2: Realistic duplicate announcements for each prefix
print("  Generating realistic duplicate announcements for each prefix...")
for prefix in selected_prefixes:
    # Get standard attributes
    attributes = create_standard_attributes()
    
    # Send a realistic number of duplicates (2) for each prefix
    # This can occur in normal BGP operations due to implementation quirks or session resets
    for i in range(2):
        # Create identical update message
        duplicate_update = BGPHeader(type=2)/BGPUpdate()
        duplicate_update.path_attr = attributes.copy()
        duplicate_update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        # Send duplicate announcement and get ACK
        src_ip_id, dst_ip_id = send_update_and_ack(duplicate_update, src_ip_id, dst_ip_id,
                                                 f"duplicate {i+1}/2 for prefix {prefix}")
        
        # Add realistic delay between duplicates (more than standard delay)
        for _ in range(2):
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    
    # Add a KEEPALIVE after each prefix's duplicates
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 3: Duplicates with slight attribute variations
# This is realistic as different router implementations might reorder attributes
print("  Generating duplicates with slight attribute variations...")
for prefix in selected_prefixes:
    # Standard attributes
    origin = BGPPathAttr(type_flags=0x40, type_code=1)
    origin.attribute = BGPPAOrigin(origin=0)
    
    as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
    as_path_segment = BGPPAASPath()
    segment = BGPPAASPath.ASPathSegment(
        segment_type=2, segment_length=1, segment_value=[main_src_as]
    )
    as_path_segment.segments = [segment]
    as_path_attr.attribute = as_path_segment
    
    next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
    next_hop_attr.attribute = BGPPANextHop(next_hop=src_ipv4)
    
    med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
    med_attr.attribute = BGPPAMultiExitDisc(med=100)
    
    local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
    local_pref_attr.attribute = BGPPALocalPref(local_pref=200)
    
    # Communities attribute with different orders
    communities_list1 = []
    communities_list1.append(BGPPACommunity(community=0xFFFFFF01))
    communities_list1.append(BGPPACommunity(community=main_src_as<<16|200))
    communities_attr1 = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
    communities_attr1.attribute = communities_list1
    
    communities_list2 = []
    communities_list2.append(BGPPACommunity(community=main_src_as<<16|200))
    communities_list2.append(BGPPACommunity(community=0xFFFFFF01))
    communities_attr2 = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
    communities_attr2.attribute = communities_list2
    
    # Create 2 attribute orders (realistic variations)
    attribute_orders = [
        [origin, as_path_attr, next_hop_attr, med_attr, local_pref_attr, communities_attr1],
        [origin, as_path_attr, next_hop_attr, local_pref_attr, med_attr, communities_attr2]
    ]
    
    for order_idx, attributes in enumerate(attribute_orders):
        # Create update with reordered attributes
        reordered_update = BGPHeader(type=2)/BGPUpdate()
        reordered_update.path_attr = attributes
        reordered_update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
        
        # Send update and get ACK
        src_ip_id, dst_ip_id = send_update_and_ack(reordered_update, src_ip_id, dst_ip_id,
                                                 f"update with attribute variation {order_idx+1} for {prefix}")
    
    # Add some delay between prefixes
    for _ in range(2):
        apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)

# SECTION 4: Multi-prefix update followed by individual prefix updates
# This is realistic as route refreshes or policy changes can cause this pattern
print("  Generating multi-prefix update followed by individual updates...")
# Get standard attributes
attributes = create_standard_attributes()

# First, send a multi-prefix update
multi_prefix_update = BGPHeader(type=2)/BGPUpdate()
multi_prefix_update.path_attr = attributes
for prefix in selected_prefixes:
    multi_prefix_update.nlri.append(BGPNLRI_IPv4(prefix=prefix))

src_ip_id, dst_ip_id = send_update_and_ack(multi_prefix_update, src_ip_id, dst_ip_id,
                                         "multi-prefix update with all selected prefixes")

# Then, send individual updates for each prefix
for prefix in selected_prefixes:
    individual_update = BGPHeader(type=2)/BGPUpdate()
    individual_update.path_attr = attributes.copy()
    individual_update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
    
    src_ip_id, dst_ip_id = send_update_and_ack(individual_update, src_ip_id, dst_ip_id,
                                             f"individual update for prefix {prefix}")

# Add a KEEPALIVE after the sequence
src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 5: Realistic withdrawal and re-announcement cycles
# This happens in real networks during maintenance or failover events
print("  Generating realistic withdrawal and re-announcement cycles...")
for prefix in selected_prefixes:
    # Get standard attributes
    attributes = create_standard_attributes()
    
    # Create withdrawal
    withdrawal = BGPHeader(type=2)/BGPUpdate()
    withdrawal.withdrawn_routes = [BGPNLRI_IPv4(prefix=prefix)]
    
    # Create announcement
    announcement = BGPHeader(type=2)/BGPUpdate()
    announcement.path_attr = attributes
    announcement.nlri.append(BGPNLRI_IPv4(prefix=prefix))
    
    # Send withdrawal followed by re-announcement
    src_ip_id, dst_ip_id = send_update_and_ack(withdrawal, src_ip_id, dst_ip_id,
                                             f"withdrawal of prefix {prefix}")
    
    # Add realistic delay before re-announcement
    for _ in range(3):
        apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    
    # Send re-announcement
    src_ip_id, dst_ip_id = send_update_and_ack(announcement, src_ip_id, dst_ip_id,
                                             f"re-announcement of prefix {prefix}")
    
    # Occasionally (for 2 prefixes), repeat the cycle once more
    if selected_prefixes.index(prefix) < 2:
        # Add more delay
        for _ in range(2):
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
        
        # Send second withdrawal
        src_ip_id, dst_ip_id = send_update_and_ack(withdrawal, src_ip_id, dst_ip_id,
                                                 f"second withdrawal of prefix {prefix}")
        
        # Add delay before re-announcement
        for _ in range(3):
            apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
        
        # Send second re-announcement
        src_ip_id, dst_ip_id = send_update_and_ack(announcement, src_ip_id, dst_ip_id,
                                                 f"second re-announcement of prefix {prefix}")
    
    # Add a KEEPALIVE after each prefix cycle
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# SECTION 6: Router restart simulation (duplicate announcements after KEEPALIVE burst)
# This simulates a router restart where routes are re-advertised after session reestablishment
print("  Simulating router restart with duplicate announcements...")
# Send a few KEEPALIVEs to simulate session reestablishment
for i in range(3):
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# Now send the duplicate announcements for all prefixes
for prefix in selected_prefixes:
    # Get standard attributes
    attributes = create_standard_attributes()
    
    # Create update
    update = BGPHeader(type=2)/BGPUpdate()
    update.path_attr = attributes
    update.nlri.append(BGPNLRI_IPv4(prefix=prefix))
    
    # Send update
    src_ip_id, dst_ip_id = send_update_and_ack(update, src_ip_id, dst_ip_id,
                                             f"post-restart announcement of prefix {prefix}")

# Add some final KEEPALIVEs
print("  Sending final KEEPALIVE messages...")
for i in range(3):
    src_ip_id, dst_ip_id = send_keepalive_and_ack(src_ip_id, dst_ip_id)

# Update session sequence numbers for future reference
if session_key in global_bgp_sessions_ipv4:
    global_bgp_sessions_ipv4[session_key]["seq_a"] = seq_a_v4
    global_bgp_sessions_ipv4[session_key]["seq_b"] = seq_b_v4

print(f"  Added {len(pkts)} packets for extended duplicate announcement scenario")



# ------------ Scenario 7: AS Path Length Changes ------------
print("[*] Generating AS path length changes scenario...")

# Get a random prefix from the main source AS
import random
target_prefix = random.choice(ip_allocations[main_src_as]["announced_prefixes"])
print(f"  Using prefix {target_prefix} for AS path length changes scenario")

# Get session parameters from the main AS pair
src_ipv4 = ip_allocations[main_src_as]["interfaces"][main_dst_as]
dst_ipv4 = ip_allocations[main_dst_as]["interfaces"][main_src_as]

src_ip_id += 1
# Check we're still in range
if src_ip_id > NORMAL_TRAFFIC_ID_RANGE[1]:
    src_ip_id = NORMAL_TRAFFIC_ID_RANGE[0]

dst_ip_id += 1
# Check we're still in range
if dst_ip_id > NORMAL_TRAFFIC_ID_RANGE[1]:
    dst_ip_id = NORMAL_TRAFFIC_ID_RANGE[0]

# Find a path of real ASes from our topology
real_as_path = [main_src_as]
# Get a list of other ASes in our topology
other_ases = [asn for asn in topology.keys() if asn != main_src_as and asn != main_dst_as]
# Add some real ASes to our path (up to 4 if available)
if len(other_ases) > 0:
    path_ases = random.sample(other_ases, min(4, len(other_ases)))
    real_as_path.extend(path_ases)
    
print(f"  Using realistic AS path: {real_as_path}")

# Ensure origin attribute is defined (IGP = 0)
origin = BGPPathAttr(type_flags=0x40, type_code=1)
origin.attribute = BGPPAOrigin(origin=0)

# NEXT_HOP attribute
next_hop_attr_v4 = BGPPathAttr(type_flags=0x40, type_code=3)
next_hop_attr_v4.attribute = BGPPANextHop(next_hop=src_ipv4)

# MED attribute
med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
med_attr.attribute = BGPPAMultiExitDisc(med=100)

# LOCAL_PREF attribute
local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
local_pref_attr.attribute = BGPPALocalPref(local_pref=200)

# COMMUNITIES attribute
communities_list = []
communities_list.append(BGPPACommunity(community=0xFFFFFF01))  # NO_EXPORT
communities_list.append(BGPPACommunity(community=main_src_as<<16|200))
communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
communities_attr.attribute = communities_list

# Create update with longer AS path (using real ASes from our topology)
as_path_length_update = BGPHeader(type=2)/BGPUpdate()

# Create an AS_PATH with increased length
as_path_length_attr = BGPPathAttr(type_flags=0x40, type_code=2)

# Create a longer AS_PATH segment with real ASes
as_path_length_segment = BGPPAASPath()
length_segment = BGPPAASPath.ASPathSegment(
    segment_type=2,  # AS_SEQUENCE
    segment_length=len(real_as_path),
    segment_value=real_as_path  # Using actual ASes from our topology
)
# Add the segment to the AS_PATH
as_path_length_segment.segments = [length_segment]
as_path_length_attr.attribute = as_path_length_segment

# Use the longer AS_PATH with the prefix
as_path_length_update.path_attr = [
    origin,
    as_path_length_attr,
    next_hop_attr_v4,
    med_attr,
    local_pref_attr,
]

# Add the prefix to the update
as_path_length_update.nlri.append(BGPNLRI_IPv4(prefix=target_prefix))

# Send AS path length update
as_path_length_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/as_path_length_update
if len(as_path_length_pkt) < 60:
    pad_len = 60 - len(as_path_length_pkt)
    as_path_length_pkt = as_path_length_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(as_path_length_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_a_v4 += len(as_path_length_update)

# ACK for AS path length update
as_path_length_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(as_path_length_ack) < 60:
    pad_len = 60 - len(as_path_length_ack)
    as_path_length_ack = as_path_length_ack/Padding(load=b'\x00' * pad_len)
pkts.append(as_path_length_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
dst_ip_id += 1  # Increment for next packet

# Create update with shorter AS path (path length change)
short_path_update = BGPHeader(type=2)/BGPUpdate()

# Create an AS_PATH with decreased length
short_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)

# Create a shorter AS_PATH segment (direct path)
short_path_segment = BGPPAASPath()
# Create a segment with just one AS
short_segment = BGPPAASPath.ASPathSegment(
    segment_type=2,  # AS_SEQUENCE
    segment_length=1,
    segment_value=[main_src_as]  # Direct path
)
# Add the segment to the AS_PATH
short_path_segment.segments = [short_segment]
short_path_attr.attribute = short_path_segment

# Use the shorter AS_PATH with the same prefix
short_path_update.path_attr = [
    origin,
    short_path_attr,
    next_hop_attr_v4,
    med_attr,
    local_pref_attr,
]

# Add the same prefix to the update
short_path_update.nlri.append(BGPNLRI_IPv4(prefix=target_prefix))

# Send shorter AS path update
short_path_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+1)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/short_path_update
if len(short_path_pkt) < 60:
    pad_len = 60 - len(short_path_pkt)
    short_path_pkt = short_path_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(short_path_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_a_v4 += len(short_path_update)

# ACK for shorter AS path
short_path_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(short_path_ack) < 60:
    pad_len = 60 - len(short_path_ack)
    short_path_ack = short_path_ack/Padding(load=b'\x00' * pad_len)
pkts.append(short_path_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay

print(f"  Added {4} packets for AS path length changes scenario")


# ------------ Scenario 8: Route Aggregation/Summarization ------------
print("[*] Generating route aggregation/summarization scenario...")

# Get prefixes from the main source AS
prefixes = ip_allocations[main_src_as]["announced_prefixes"]
print(f"  Available prefixes for aggregation: {prefixes}")

# Get session parameters from the main AS pair
src_ipv4 = ip_allocations[main_src_as]["interfaces"][main_dst_as]
dst_ipv4 = ip_allocations[main_dst_as]["interfaces"][main_src_as]

src_ip_id += 1
# Check we're still in range
if src_ip_id > NORMAL_TRAFFIC_ID_RANGE[1]:
    src_ip_id = NORMAL_TRAFFIC_ID_RANGE[0]

dst_ip_id += 1
# Check we're still in range
if dst_ip_id > NORMAL_TRAFFIC_ID_RANGE[1]:
    dst_ip_id = NORMAL_TRAFFIC_ID_RANGE[0]

# Ensure origin attribute is defined (IGP = 0)
origin = BGPPathAttr(type_flags=0x40, type_code=1)
origin.attribute = BGPPAOrigin(origin=0)

# Regular AS_PATH attribute
as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
as_path_segment = BGPPAASPath()
segment = BGPPAASPath.ASPathSegment(
    segment_type=2,  # AS_SEQUENCE
    segment_length=1,
    segment_value=[main_src_as]
)
as_path_segment.segments = [segment]
as_path_attr.attribute = as_path_segment

# NEXT_HOP attribute
next_hop_attr_v4 = BGPPathAttr(type_flags=0x40, type_code=3)
next_hop_attr_v4.attribute = BGPPANextHop(next_hop=src_ipv4)

# MED attribute
med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
med_attr.attribute = BGPPAMultiExitDisc(med=100)

# LOCAL_PREF attribute
local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
local_pref_attr.attribute = BGPPALocalPref(local_pref=200)

# ATOMIC_AGGREGATE attribute
atomic_aggr_attr = BGPPathAttr(type_flags=0x40, type_code=6)
atomic_aggr_attr.attribute = BGPPAAtomicAggregate()

# AGGREGATOR attribute
aggregator_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=7)
aggregator_attr.attribute = BGPPAAggregator(aggregator_asn=main_src_as, 
                                          speaker_address=ip_allocations[main_src_as]["router_id"])

# COMMUNITIES attribute
communities_list = []
communities_list.append(BGPPACommunity(community=0xFFFFFF01))  # NO_EXPORT
communities_list.append(BGPPACommunity(community=main_src_as<<16|200))
communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
communities_attr.attribute = communities_list

# First, announce more specific prefixes
more_specific_packets = []

# For a selected prefix, announce two more specific routes
if len(prefixes) > 0:
    # Select the first prefix to derive more specific routes
    base_prefix = prefixes[0]
    print(f"  Using {base_prefix} as base for aggregation")
    
    # Parse the base prefix
    base_ip, base_mask = base_prefix.split('/')
    base_mask = int(base_mask)
    octets = base_ip.split('.')
    
    # Create two more specific prefixes (subnet if possible)
    more_specific_prefixes = []
    if base_mask < 24:
        # We can create two more specific prefixes
        more_specific_mask = base_mask + 1
        
        # First subnet
        more_specific1 = f"{octets[0]}.{octets[1]}.{octets[2]}.0/{more_specific_mask}"
        more_specific_prefixes.append(more_specific1)
        
        # Second subnet - increment the appropriate octet
        if more_specific_mask <= 8:
            # Modify first octet
            octets[0] = str(int(octets[0]) + 2**(8 - more_specific_mask))
        elif more_specific_mask <= 16:
            # Modify second octet
            octets[1] = str(int(octets[1]) + 2**(16 - more_specific_mask))
        elif more_specific_mask <= 24:
            # Modify third octet
            octets[2] = str(int(octets[2]) + 2**(24 - more_specific_mask))
        
        more_specific2 = f"{octets[0]}.{octets[1]}.{octets[2]}.0/{more_specific_mask}"
        more_specific_prefixes.append(more_specific2)
    else:
        # Base prefix is already /24 or more specific, just use it
        more_specific_prefixes.append(base_prefix)
    
    print(f"  Created more specific prefixes: {more_specific_prefixes}")
    
    # Announce more specific prefixes
    for i, specific_prefix in enumerate(more_specific_prefixes):
        specific_update = BGPHeader(type=2)/BGPUpdate()
        specific_update.path_attr = [
            origin,
            as_path_attr,
            next_hop_attr_v4,
            med_attr,
            local_pref_attr,
            communities_attr,
        ]
        specific_update.nlri.append(BGPNLRI_IPv4(prefix=specific_prefix))
        
        # Send more specific update
        specific_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/specific_update
        if len(specific_pkt) < 60:
            pad_len = 60 - len(specific_pkt)
            specific_pkt = specific_pkt/Padding(load=b'\x00' * pad_len)
        pkts.append(specific_pkt)
        more_specific_packets.append(specific_pkt)
        apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
        seq_a_v4 += len(specific_update)
        src_ip_id += 1
        
        # ACK for more specific update
        specific_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
        if len(specific_ack) < 60:
            pad_len = 60 - len(specific_ack)
            specific_ack = specific_ack/Padding(load=b'\x00' * pad_len)
        pkts.append(specific_ack)
        apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
        dst_ip_id += 1
    
    # Now announce the aggregated route
    aggregation_update = BGPHeader(type=2)/BGPUpdate()
    
    # Use attributes for route aggregation
    aggregation_update.path_attr = [
        origin,
        as_path_attr,
        next_hop_attr_v4,
        med_attr,
        local_pref_attr,
        atomic_aggr_attr,     # Indicate route aggregation
        aggregator_attr,      # Provide aggregator info
    ]
    
    # Add the original prefix as the aggregate
    aggregation_update.nlri.append(BGPNLRI_IPv4(prefix=base_prefix))
    
    # Send aggregation update
    aggregation_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/aggregation_update
    if len(aggregation_pkt) < 60:
        pad_len = 60 - len(aggregation_pkt)
        aggregation_pkt = aggregation_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(aggregation_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    seq_a_v4 += len(aggregation_update)
    
    # ACK for aggregation update
    aggregation_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
    if len(aggregation_ack) < 60:
        pad_len = 60 - len(aggregation_ack)
        aggregation_ack = aggregation_ack/Padding(load=b'\x00' * pad_len)
    pkts.append(aggregation_ack)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    
    print(f"  Added {len(more_specific_packets) + 2} packets for route aggregation scenario")
else:
    print("  ⚠️ No prefixes available for aggregation - skipping scenario")


# ------------ Scenario 9: IPv4/IPv6 Withdrawals Using MP_UNREACH_NLRI ------------
print("[*] Generating BGP route withdrawals (IPv4 direct and IPv6 MP_UNREACH_NLRI)...")

# Get session parameters from the main AS pair
src_ipv4 = ip_allocations[main_src_as]["interfaces"][main_dst_as]
dst_ipv4 = ip_allocations[main_dst_as]["interfaces"][main_src_as]

src_ip_id += 1
# Check we're still in range
if src_ip_id > NORMAL_TRAFFIC_ID_RANGE[1]:
    src_ip_id = NORMAL_TRAFFIC_ID_RANGE[0]

dst_ip_id += 1
# Check we're still in range
if dst_ip_id > NORMAL_TRAFFIC_ID_RANGE[1]:
    dst_ip_id = NORMAL_TRAFFIC_ID_RANGE[0]

# Get IPv4 prefixes to withdraw
import random
if len(ip_allocations[main_src_as]["announced_prefixes"]) >= 2:
    # Select 2 different prefixes if available
    ipv4_prefixes = random.sample(ip_allocations[main_src_as]["announced_prefixes"], 2)
else:
    # Use all available prefixes
    ipv4_prefixes = ip_allocations[main_src_as]["announced_prefixes"].copy()

print(f"  Withdrawing IPv4 prefixes: {ipv4_prefixes}")

# Generate corresponding IPv6 prefixes
ipv6_prefixes = []
for prefix in ipv4_prefixes:
    ip, mask = prefix.split('/')
    octets = ip.split('.')
    ipv6_prefix = f"2001:db8:{int(octets[2]):x}:{int(octets[3]):x}::/64"
    ipv6_prefixes.append(ipv6_prefix)

print(f"  Withdrawing IPv6 prefixes: {ipv6_prefixes}")

# 1. IPv4 Withdrawals (direct method)
# Create a new BGPUpdate instance directly (not layered on BGPHeader yet)
ipv4_withdraw = BGPUpdate()

# Create BGPNLRI_IPv4 objects for each prefix to withdraw
withdrawn_ipv4_objs = []
for prefix in ipv4_prefixes:
    withdrawn_ipv4_objs.append(BGPNLRI_IPv4(prefix=prefix))

# Set the withdrawn_routes field directly
ipv4_withdraw.withdrawn_routes = withdrawn_ipv4_objs
# Empty path attributes for a withdrawal
ipv4_withdraw.path_attr = []
# Empty NLRI for a withdrawal
ipv4_withdraw.nlri = []

# Now layer it with BGPHeader
ipv4_withdraw_msg = BGPHeader(type=2)/ipv4_withdraw

# Send IPv4 withdrawal
ipv4_withdraw_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/ipv4_withdraw_msg
if len(ipv4_withdraw_pkt) < 60:
    pad_len = 60 - len(ipv4_withdraw_pkt)
    ipv4_withdraw_pkt = ipv4_withdraw_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(ipv4_withdraw_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
seq_a_v4 += len(ipv4_withdraw_msg)

# ACK for IPv4 withdrawal
ipv4_withdraw_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(ipv4_withdraw_ack) < 60:
    pad_len = 60 - len(ipv4_withdraw_ack)
    ipv4_withdraw_ack = ipv4_withdraw_ack/Padding(load=b'\x00' * pad_len)
pkts.append(ipv4_withdraw_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
dst_ip_id += 1

# 2. IPv6 Withdrawals (MP_UNREACH_NLRI method)
mp_unreach_withdraw = BGPHeader(type=2)/BGPUpdate()

# Create the IPv6-specific part with withdrawn routes
withdrawn_ipv6_objs = []
for ipv6_prefix in ipv6_prefixes:
    withdrawn_ipv6_objs.append(BGPNLRI_IPv6(prefix=ipv6_prefix))

ipv6_specific = BGPPAMPUnreachNLRI_IPv6(withdrawn_routes=withdrawn_ipv6_objs)

# Create the MP_UNREACH_NLRI with proper structure
mp_unreach = BGPPAMPUnreachNLRI(afi=2, safi=1, afi_safi_specific=ipv6_specific)

# Create the attribute wrapper
mp_unreach_attr = BGPPathAttr(type_flags=0x80, type_code=15)
mp_unreach_attr.attribute = mp_unreach

# Add to UPDATE message
mp_unreach_withdraw.path_attr = [mp_unreach_attr]

# Send MP_UNREACH_NLRI update for IPv6
mp_unreach_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+1)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/mp_unreach_withdraw
if len(mp_unreach_pkt) < 60:
    pad_len = 60 - len(mp_unreach_pkt)
    mp_unreach_pkt = mp_unreach_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(mp_unreach_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
seq_a_v4 += len(mp_unreach_withdraw)

# ACK for MP_UNREACH_NLRI
mp_unreach_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(mp_unreach_ack) < 60:
    pad_len = 60 - len(mp_unreach_ack)
    mp_unreach_ack = mp_unreach_ack/Padding(load=b'\x00' * pad_len)
pkts.append(mp_unreach_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)

print(f"  Added {4} packets for BGP route withdrawals")

# ------------ Scenario 10: BGP NOTIFICATION Messages ------------
print("[*] Generating BGP NOTIFICATION messages scenarios...")

# Get session parameters from the main AS pair
src_ipv4 = ip_allocations[main_src_as]["interfaces"][main_dst_as]
dst_ipv4 = ip_allocations[main_dst_as]["interfaces"][main_src_as]

# For IPv6 notifications, get IPv6 addresses if available, or generate them
src_ipv6 = ip_allocations[main_src_as].get("ipv6_interfaces", {}).get(main_dst_as)
dst_ipv6 = ip_allocations[main_dst_as].get("ipv6_interfaces", {}).get(main_src_as)

# If IPv6 interfaces aren't available, generate them from the IPv4 addresses
if not src_ipv6:
    src_ipv6 = f"2001:db8:{main_src_as}::{main_dst_as}"
if not dst_ipv6:
    dst_ipv6 = f"2001:db8:{main_dst_as}::{main_src_as}"

src_ip_id += 1
# Check we're still in range
if src_ip_id > NORMAL_TRAFFIC_ID_RANGE[1]:
    src_ip_id = NORMAL_TRAFFIC_ID_RANGE[0]

dst_ip_id += 1
# Check we're still in range
if dst_ip_id > NORMAL_TRAFFIC_ID_RANGE[1]:
    dst_ip_id = NORMAL_TRAFFIC_ID_RANGE[0]

# 1. Message Header Error (Error Code 1)
print("  Generating Message Header Error (Code 1, Subcode 2 - Bad Message Length)")
notification_header = BGPHeader(type=3)/BGPNotification(error_code=1, error_subcode=2)  # Bad Message Length
notification_header_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/notification_header
pkts.append(notification_header_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_a_v4 += len(notification_header)

# ACK for Message Header Error
notification_header_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(notification_header_ack) < 60:
    pad_len = 60 - len(notification_header_ack)
    notification_header_ack = notification_header_ack/Padding(load=b'\x00' * pad_len)
pkts.append(notification_header_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
dst_ip_id += 1

# 2. OPEN Message Error (Error Code 2)
print("  Generating OPEN Message Error (Code 2, Subcode 2 - Bad Peer AS)")
notification_open = BGPHeader(type=3)/BGPNotification(error_code=2, error_subcode=2)  # Bad Peer AS
notification_open_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+1)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/notification_open
if len(notification_open_pkt) < 60:
    pad_len = 60 - len(notification_open_pkt)
    notification_open_pkt = notification_open_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(notification_open_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_a_v4 += len(notification_open)

# ACK for OPEN Message Error
notification_open_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(notification_open_ack) < 60:
    pad_len = 60 - len(notification_open_ack)
    notification_open_ack = notification_open_ack/Padding(load=b'\x00' * pad_len)
pkts.append(notification_open_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
dst_ip_id += 1

# 3. UPDATE Message Error (Error Code 3)
print("  Generating UPDATE Message Error (Code 3, Subcode 1 - Malformed Attribute List)")
notification_update = BGPHeader(type=3)/BGPNotification(error_code=3, error_subcode=1)  # Malformed Attribute List
notification_update_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+2)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/notification_update
if len(notification_update_pkt) < 60:
    pad_len = 60 - len(notification_update_pkt)
    notification_update_pkt = notification_update_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(notification_update_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_a_v4 += len(notification_update)

# ACK for UPDATE Message Error
notification_update_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(notification_update_ack) < 60:
    pad_len = 60 - len(notification_update_ack)
    notification_update_ack = notification_update_ack/Padding(load=b'\x00' * pad_len)
pkts.append(notification_update_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
dst_ip_id += 1

# 4. Hold Timer Expired (Error Code 4)
print("  Generating Hold Timer Expired (Code 4, Subcode 0)")
notification_hold = BGPHeader(type=3)/BGPNotification(error_code=4, error_subcode=0)  # Hold Timer Expired
notification_hold_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+3)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/notification_hold
if len(notification_hold_pkt) < 60:
    pad_len = 60 - len(notification_hold_pkt)
    notification_hold_pkt = notification_hold_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(notification_hold_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_a_v4 += len(notification_hold)

# ACK for Hold Timer Expired
notification_hold_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(notification_hold_ack) < 60:
    pad_len = 60 - len(notification_hold_ack)
    notification_hold_ack = notification_hold_ack/Padding(load=b'\x00' * pad_len)
pkts.append(notification_hold_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
dst_ip_id += 1

# 5. Finite State Machine Error (Error Code 5)
print("  Generating Finite State Machine Error (Code 5, Subcode 0)")
notification_fsm = BGPHeader(type=3)/BGPNotification(error_code=5, error_subcode=0)  # FSM Error
notification_fsm_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+4)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/notification_fsm
if len(notification_fsm_pkt) < 60:
    pad_len = 60 - len(notification_fsm_pkt)
    notification_fsm_pkt = notification_fsm_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(notification_fsm_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_a_v4 += len(notification_fsm)

# ACK for FSM Error
notification_fsm_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(notification_fsm_ack) < 60:
    pad_len = 60 - len(notification_fsm_ack)
    notification_fsm_ack = notification_fsm_ack/Padding(load=b'\x00' * pad_len)
pkts.append(notification_fsm_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay

print(f"  Added {10} packets for BGP NOTIFICATION messages")


# ------------ Scenario 11: Normal Traffic ------------
print("[*] Generating normal BGP traffic patterns...")

# Get session parameters from the main AS pair
src_ipv4 = ip_allocations[main_src_as]["interfaces"][main_dst_as]
dst_ipv4 = ip_allocations[main_dst_as]["interfaces"][main_src_as]

src_ip_id += 1
# Check we're still in range
if src_ip_id > NORMAL_TRAFFIC_ID_RANGE[1]:
    src_ip_id = NORMAL_TRAFFIC_ID_RANGE[0]

dst_ip_id += 1
# Check we're still in range
if dst_ip_id > NORMAL_TRAFFIC_ID_RANGE[1]:
    dst_ip_id = NORMAL_TRAFFIC_ID_RANGE[0]

# Get prefixes to announce in normal traffic
normal_prefixes = ip_allocations[main_src_as]["announced_prefixes"]
print(f"  Generating normal traffic for {len(normal_prefixes)} prefixes")

# 1. Regular KEEPALIVE message exchange
print("  Generating KEEPALIVE messages")
keepalive_v4 = BGPHeader(type=4)/BGPKeepAlive()
keepalive_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/keepalive_v4
if len(keepalive_pkt) < 60:
    pad_len = 60 - len(keepalive_pkt)
    keepalive_pkt = keepalive_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(keepalive_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_a_v4 += len(keepalive_v4)

# ACK for KEEPALIVE
keepalive_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(keepalive_ack) < 60:
    pad_len = 60 - len(keepalive_ack)
    keepalive_ack = keepalive_ack/Padding(load=b'\x00' * pad_len)
pkts.append(keepalive_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
dst_ip_id += 1

# KEEPALIVE from the other direction
keepalive_v4_reply = BGPHeader(type=4)/BGPKeepAlive()
keepalive_reply_pkt = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="PA", seq=seq_b_v4, ack=seq_a_v4, window=16384)/keepalive_v4_reply
if len(keepalive_reply_pkt) < 60:
    pad_len = 60 - len(keepalive_reply_pkt)
    keepalive_reply_pkt = keepalive_reply_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(keepalive_reply_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
seq_b_v4 += len(keepalive_v4_reply)

# ACK for reply KEEPALIVE
keepalive_reply_ack = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+1)/TCP(sport=sport, dport=dport, flags="A", seq=seq_a_v4, ack=seq_b_v4, window=16384)
if len(keepalive_reply_ack) < 60:
    pad_len = 60 - len(keepalive_reply_ack)
    keepalive_reply_ack = keepalive_reply_ack/Padding(load=b'\x00' * pad_len)
pkts.append(keepalive_reply_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)  # Add delay
src_ip_id += 2

# 2. Regular stable route announcements
print("  Generating stable route announcements")
# Ensure origin attribute is defined (IGP = 0)
origin = BGPPathAttr(type_flags=0x40, type_code=1)
origin.attribute = BGPPAOrigin(origin=0)

# Regular AS_PATH attribute
as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
as_path_segment = BGPPAASPath()
segment = BGPPAASPath.ASPathSegment(
    segment_type=2,  # AS_SEQUENCE
    segment_length=1,
    segment_value=[main_src_as]
)
as_path_segment.segments = [segment]
as_path_attr.attribute = as_path_segment

# NEXT_HOP attribute
next_hop_attr_v4 = BGPPathAttr(type_flags=0x40, type_code=3)
next_hop_attr_v4.attribute = BGPPANextHop(next_hop=src_ipv4)

# MED attribute
med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
med_attr.attribute = BGPPAMultiExitDisc(med=100)

# LOCAL_PREF attribute
local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
local_pref_attr.attribute = BGPPALocalPref(local_pref=200)

# COMMUNITIES attribute
communities_list = []
communities_list.append(BGPPACommunity(community=0xFFFFFF01))  # NO_EXPORT
communities_list.append(BGPPACommunity(community=main_src_as<<16|200))
communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
communities_attr.attribute = communities_list

# Announce each prefix in a separate update
for i, prefix in enumerate(normal_prefixes):
    normal_update = BGPHeader(type=2)/BGPUpdate()
    normal_update.path_attr = [
        origin,
        as_path_attr,
        next_hop_attr_v4,
        med_attr,
        local_pref_attr,
        communities_attr
    ]
    normal_update.nlri = [BGPNLRI_IPv4(prefix=prefix)]

    normal_update_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+i)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/normal_update
    if len(normal_update_pkt) < 60:
        pad_len = 60 - len(normal_update_pkt)
        normal_update_pkt = normal_update_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(normal_update_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    seq_a_v4 += len(normal_update)

    # ACK for update
    normal_update_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id+i)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
    if len(normal_update_ack) < 60:
        pad_len = 60 - len(normal_update_ack)
        normal_update_ack = normal_update_ack/Padding(load=b'\x00' * pad_len)
    pkts.append(normal_update_ack)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)

src_ip_id += len(normal_prefixes)
dst_ip_id += len(normal_prefixes)

# 3. Routes from the other direction (peer to source)
print("  Generating peer-to-source route announcements")

# Get prefixes from the destination AS
peer_prefixes = ip_allocations[main_dst_as]["announced_prefixes"]
if not peer_prefixes:
    # If no prefixes defined, create some
    peer_prefixes = [f"172.16.{main_dst_as}.0/24", f"172.17.{main_dst_as}.0/24"]

# Modify attributes for reverse direction
peer_as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
peer_as_path_segment = BGPPAASPath()
peer_segment = BGPPAASPath.ASPathSegment(
    segment_type=2,  # AS_SEQUENCE
    segment_length=1,
    segment_value=[main_dst_as]  # From peer AS
)
peer_as_path_segment.segments = [peer_segment]
peer_as_path_attr.attribute = peer_as_path_segment

# NEXT_HOP from peer
peer_next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
peer_next_hop_attr.attribute = BGPPANextHop(next_hop=dst_ipv4)

# Announce prefixes from peer
for i, prefix in enumerate(peer_prefixes):
    peer_update = BGPHeader(type=2)/BGPUpdate()
    peer_update.path_attr = [
        origin,
        peer_as_path_attr,
        peer_next_hop_attr,
        med_attr,
        local_pref_attr,
        communities_attr
    ]
    peer_update.nlri = [BGPNLRI_IPv4(prefix=prefix)]

    peer_update_pkt = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id+i)/TCP(sport=dport, dport=sport, flags="PA", seq=seq_b_v4, ack=seq_a_v4, window=16384)/peer_update
    if len(peer_update_pkt) < 60:
        pad_len = 60 - len(peer_update_pkt)
        peer_update_pkt = peer_update_pkt/Padding(load=b'\x00' * pad_len)
    pkts.append(peer_update_pkt)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
    seq_b_v4 += len(peer_update)

    # ACK for peer update
    peer_update_ack = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id+i)/TCP(sport=sport, dport=dport, flags="A", seq=seq_a_v4, ack=seq_b_v4, window=16384)
    if len(peer_update_ack) < 60:
        pad_len = 60 - len(peer_update_ack)
        peer_update_ack = peer_update_ack/Padding(load=b'\x00' * pad_len)
    pkts.append(peer_update_ack)
    apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)

src_ip_id += len(peer_prefixes)
dst_ip_id += len(peer_prefixes)

# 4. Final KEEPALIVE messages to show stable session
print("  Generating final KEEPALIVE messages for stable session")
final_keepalive = BGPHeader(type=4)/BGPKeepAlive()
final_keepalive_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a_v4, ack=seq_b_v4, window=16384)/final_keepalive
if len(final_keepalive_pkt) < 60:
    pad_len = 60 - len(final_keepalive_pkt)
    final_keepalive_pkt = final_keepalive_pkt/Padding(load=b'\x00' * pad_len)
pkts.append(final_keepalive_pkt)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)
seq_a_v4 += len(final_keepalive)

# ACK for final KEEPALIVE
final_keepalive_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b_v4, ack=seq_a_v4, window=16384)
if len(final_keepalive_ack) < 60:
    pad_len = 60 - len(final_keepalive_ack)
    final_keepalive_ack = final_keepalive_ack/Padding(load=b'\x00' * pad_len)
pkts.append(final_keepalive_ack)
apply_delay(is_attack=False, distribution=DELAY_DISTRIBUTION)

total_packets = 4 + (2 * len(normal_prefixes)) + (2 * len(peer_prefixes))
print(f"  Added {total_packets} packets for normal BGP traffic")

# ------------ Scenario 12: additional  Traffic ------------

def generate_additional_normal_traffic(topology, ip_allocations, pkts, target_updates=180):
    """Generate a controlled amount of normal BGP traffic to balance dataset"""
    print(f"[*] Generating up to {target_updates} normal BGP updates...")
    
    # Keep track of the total updates generated
    total_updates_generated = 0
    
    # Find valid AS numbers from the topology
    if len(topology) >= 2:
        main_as_list = list(topology.keys())
        main_src_as = main_as_list[0]  # Use first AS as source
        main_dst_as = main_as_list[1]  # Use second AS as destination
        
        print(f"  Using AS{main_src_as} and AS{main_dst_as} for normal traffic")
    else:
        print("Error: Need at least 2 ASes in the topology")
        return 0
    
    # Get session parameters
    try:
        src_ipv4 = ip_allocations[main_src_as]["interfaces"][main_dst_as]
        dst_ipv4 = ip_allocations[main_dst_as]["interfaces"][main_src_as]
    except KeyError:
        print("Error: Interface information not found in ip_allocations")
        # Use fallback values if interface information is not available
        src_ipv4 = f"10.{main_src_as//256}.{main_src_as%256}.1"
        dst_ipv4 = f"10.{main_dst_as//256}.{main_dst_as%256}.1"
        print(f"  Using fallback IP addresses: {src_ipv4} and {dst_ipv4}")
    
    # Create MAC addresses
    #src_mac = "00:" + ":".join([f"{random.randint(0, 255):02x}" for _ in range(5)])
    #dst_mac = "00:" + ":".join([f"{random.randint(0, 255):02x}" for _ in range(5)])
    
    # Generate port numbers
    sport = random.randint(30000, 65000)
    dport = 179  # Standard BGP port
    
    # Initialize IP IDs and sequence numbers
    src_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
    dst_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
    seq_a = random.randint(1000, 10000)
    seq_b = random.randint(1000, 10000)
    
    # Generate a diverse set of prefixes for normal announcements
    # Start with existing prefixes from topology if available, otherwise create some
    try:
        normal_prefixes = list(ip_allocations[main_src_as]["announced_prefixes"])
    except (KeyError, TypeError):
        normal_prefixes = []
    
    # If not enough prefixes, add more
    if len(normal_prefixes) < 50:  # Ensure we have enough prefixes for our target
        # Add more diverse prefixes
        for i in range(1, 15):
            for j in range(0, 10, 2):
                prefix = f"10.{i}.{j}.0/24"
                if prefix not in normal_prefixes:
                    normal_prefixes.append(prefix)
        
        for i in range(1, 15):
            prefix = f"192.168.{i}.0/24"
            if prefix not in normal_prefixes:
                normal_prefixes.append(prefix)
    
    # Add some IPv6 prefixes
    ipv6_prefixes = []
    for i in range(1, 4):
        ipv6_prefixes.append(f"2001:db8:{i}::/48")
    
    print(f"  Generated {len(normal_prefixes)} IPv4 prefixes and {len(ipv6_prefixes)} IPv6 prefixes")
    
    # Create standard path attributes for normal traffic
    # ORIGIN - IGP
    origin = BGPPathAttr(type_flags=0x40, type_code=1)
    origin.attribute = BGPPAOrigin(origin=0)
    
    # AS_PATH - normal path
    as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
    as_path_segment = BGPPAASPath()
    segment = BGPPAASPath.ASPathSegment(
        segment_type=2,  # AS_SEQUENCE
        segment_length=1,
        segment_value=[main_src_as]
    )
    as_path_segment.segments = [segment]
    as_path_attr.attribute = as_path_segment
    
    # NEXT_HOP
    next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
    next_hop_attr.attribute = BGPPANextHop(next_hop=src_ipv4)
    
    # MED
    med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
    med_attr.attribute = BGPPAMultiExitDisc(med=100)
    
    # LOCAL_PREF
    local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
    local_pref_attr.attribute = BGPPALocalPref(local_pref=200)
    
    # COMMUNITIES
    communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
    communities_list = []
    communities_list.append(BGPPACommunity(community=main_src_as<<16|100))
    communities_list.append(BGPPACommunity(community=main_src_as<<16|200))
    communities_attr.attribute = communities_list
    
    # 1. Generate regular announcements (bulk of normal traffic)
    print("  Generating regular route announcements...")
    announcements_count = 0
    
    # Announce IPv4 prefixes in batches to be more realistic
    batch_size = 7  # Announce 7 prefixes per UPDATE
    batch_count = min(target_updates, len(normal_prefixes))
    
    for batch_index in range(batch_count):
        if total_updates_generated >= target_updates:
            break
            
        # Calculate batch start
        batch_start = (batch_index * batch_size) % len(normal_prefixes)
        
        # Get batch of prefixes
        batch_prefixes = normal_prefixes[batch_start:batch_start + batch_size]
        if not batch_prefixes:  # In case we run out of prefixes
            batch_prefixes = normal_prefixes[:batch_size]
        
        # Create UPDATE message
        update = BGPHeader(type=2)/BGPUpdate()
        update.path_attr = [
            origin,
            as_path_attr,
            next_hop_attr,
            med_attr,
            local_pref_attr,
            communities_attr
        ]
        
        # Add all prefixes in this batch to NLRI
        update.nlri = [BGPNLRI_IPv4(prefix=prefix) for prefix in batch_prefixes]
        
        # Make sure IP ID is in normal range
        src_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
        
        # Create packet
        update_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a, ack=seq_b, window=16384)/update
        if len(update_pkt) < 60:
            pad_len = 60 - len(update_pkt)
            update_pkt = update_pkt/Padding(load=b'\x00' * pad_len)
        
        pkts.append(update_pkt)  # Add to global pkts list
        announcements_count += len(batch_prefixes)
        total_updates_generated += 1  # Count each UPDATE as one
        
        # Update sequence number
        seq_a += len(update)
        
        # ACK for update
        dst_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
        update_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b, ack=seq_a, window=16384)
        if len(update_ack) < 60:
            pad_len = 60 - len(update_ack)
            update_ack = update_ack/Padding(load=b'\x00' * pad_len)
        
        pkts.append(update_ack)  # Add to global pkts list
        
        # Add a slight delay between batches for realism
        try:
            apply_delay(is_attack=False, distribution="normal")
        except:
            # If delay function fails, continue without delay
            pass
    
    # 2. Generate some withdrawals if we haven't reached the target yet
    if total_updates_generated < target_updates:
        print("  Generating normal withdrawals...")
        withdraw_count = 0
        
        # Choose a subset of prefixes to withdraw
        max_withdrawals = target_updates - total_updates_generated  # Remaining updates we can generate
        num_withdraws = min(int(len(normal_prefixes) * 0.2), max_withdrawals * 3)  # Withdraw up to 20% of prefixes
        prefixes_to_withdraw = random.sample(normal_prefixes, num_withdraws)
        
        # Create batches of withdrawals
        withdraw_batch_size = 3
        for batch_start in range(0, len(prefixes_to_withdraw), withdraw_batch_size):
            if total_updates_generated >= target_updates:
                break
                
            batch_prefixes = prefixes_to_withdraw[batch_start:batch_start + withdraw_batch_size]
            
            # Create UPDATE with withdrawals
            withdraw = BGPHeader(type=2)/BGPUpdate()
            withdraw.path_attr = []  # Empty path attributes for a withdrawal
            
            # Different approach to set withdrawn_routes
            withdraw_nlri_list = []
            for prefix in batch_prefixes:
                withdraw_nlri_list.append(BGPNLRI_IPv4(prefix=prefix))
            
            withdraw.withdrawn_routes = withdraw_nlri_list
            withdraw.nlri = []       # Empty NLRI for a withdrawal
            
            # Make sure IP ID is in normal range
            src_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
            
            # Create packet
            withdraw_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a, ack=seq_b, window=16384)/withdraw
            if len(withdraw_pkt) < 60:
                pad_len = 60 - len(withdraw_pkt)
                withdraw_pkt = withdraw_pkt/Padding(load=b'\x00' * pad_len)
            
            pkts.append(withdraw_pkt)  # Add to global pkts list
            withdraw_count += len(batch_prefixes)
            total_updates_generated += 1  # Count each UPDATE as one
            
            # Update sequence number
            seq_a += len(withdraw)
            
            # ACK for withdrawal
            dst_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
            withdraw_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b, ack=seq_a, window=16384)
            if len(withdraw_ack) < 60:
                pad_len = 60 - len(withdraw_ack)
                withdraw_ack = withdraw_ack/Padding(load=b'\x00' * pad_len)
            
            pkts.append(withdraw_ack)  # Add to global pkts list
            
            # Add a slight delay for realism
            try:
                apply_delay(is_attack=False, distribution="DELAY_DISTRIBUTION")
            except:
                # If delay function fails, continue without delay
                pass
    
    # 3. Add some KEEPALIVE messages for realism if we still haven't reached the target
    if total_updates_generated < target_updates:
        print("  Adding KEEPALIVE messages...")
        keepalive_count = min(target_updates - total_updates_generated, 10)  # Add up to 10 KEEPALIVEs
        
        for _ in range(keepalive_count):  # Add KEEPALIVEs
            # Make sure IP ID is in normal range
            src_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
            
            # Create KEEPALIVE
            keepalive = BGPHeader(type=4)/BGPKeepAlive()
            keepalive_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=sport, dport=dport, flags="PA", seq=seq_a, ack=seq_b, window=16384)/keepalive
            if len(keepalive_pkt) < 60:
                pad_len = 60 - len(keepalive_pkt)
                keepalive_pkt = keepalive_pkt/Padding(load=b'\x00' * pad_len)
            
            pkts.append(keepalive_pkt)  # Add to global pkts list
            total_updates_generated += 1  # Count each KEEPALIVE as one
            
            # Update sequence number
            seq_a += len(keepalive)
            
            # ACK for KEEPALIVE
            dst_ip_id = random.randint(NORMAL_TRAFFIC_ID_RANGE[0], NORMAL_TRAFFIC_ID_RANGE[1])
            keepalive_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ipv4, dst=src_ipv4, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dport, dport=sport, flags="A", seq=seq_b, ack=seq_a, window=16384)
            if len(keepalive_ack) < 60:
                pad_len = 60 - len(keepalive_ack)
                keepalive_ack = keepalive_ack/Padding(load=b'\x00' * pad_len)
            
            pkts.append(keepalive_ack)  # Add to global pkts list
            
            # Add a longer delay for KEEPALIVEs
            try:
                apply_delay(is_attack=False, distribution="DELAY_DISTRIBUTION")
            except:
                # If delay function fails, continue without delay
                pass
    
    # Total number of packets added
    normal_packets_added = total_updates_generated * 2  # x2 for ACKs
    
    print(f"Added {normal_packets_added} normal packets:")
    print(f"  - {announcements_count} route announcements in {batch_count} UPDATE packets")
    print(f"  - {withdraw_count if 'withdraw_count' in locals() else 0} route withdrawals")
    print(f"  - {keepalive_count if 'keepalive_count' in locals() else 0} KEEPALIVE messages")
    print(f"  - Generated a total of {total_updates_generated} normal BGP messages")
    
    return total_updates_generated

# Generate additional normal traffic
print("\n[*] Generating additional normal BGP traffic to balance dataset")
normal_updates_added = generate_additional_normal_traffic(topology, ip_allocations, pkts, target_updates=150) 

# Create directories if they don't exist
os.makedirs("/Users/shadimoteali/PhD/BGP_Traffic_Generation/pcaps", exist_ok=True)

# Write to pcap
wrpcap("/Users/shadimoteali/PhD/BGP_Traffic_Generation/pcaps/realistic_bgp_complete_scenarios.pcap", pkts)
print(f"✅ Wrote pcaps/realistic_bgp_complete_scenarios.pcap with comprehensive BGP scenarios (added {normal_updates_added} normal updates)")

[+] Generating realistic BGP updates for IPv4 session...
[*] Generating ORIGIN change scenario (Extended)...
  Using prefix 203.0.113.0/24 for ORIGIN Change scenario
  Added 18 packets for extended ORIGIN change scenario
[*] Generating extended AS_PATH modification scenario...
  Using prefix 203.0.113.0/24 and others for AS_PATH modification scenario
  Generating initial KEEPALIVE exchanges...
  Generating basic AS path prepending variations...
    - Sent update with 1x prepending
    - Sent update with 2x prepending
    - Sent update with 3x prepending
    - Sent update with 4x prepending
    - Sent update with 5x prepending
    - Sent update with 6x prepending
    - Sent update with 7x prepending
  Announcing multiple prefixes with AS path variations...
    - Sent update for prefix 203.0.113.0/24 with 1 transit ASes
    - Sent update for prefix 203.0.113.0/24 with 2 transit ASes
    - Sent update for prefix 203.0.113.0/24 with 3 transit ASes
    - Sent update for prefix 198.51.100.0/

In [124]:
# ======================================
# PART 7: Anormal BGP Update Scenarios
# ======================================
print("[+] Generating Anormal BGP updates for IPv4 session...")

# ------------ Scenario 1: Prefix hijack ------------

def generate_prefix_hijacking_attack(topology, ip_allocations):
    """Generate BGP prefix hijacking attack packets with specific target prefixes - extended version"""
    attack_packets = []
    
    # Use the specified AS numbers for attacker and victim
    attacker_as = 39224  # Attacker AS
    victim_as = 47066    # Victim AS
    
    print(f"[+] Generating extended prefix hijacking attack: AS{attacker_as} hijacking AS{victim_as}'s prefixes")
    
    # Use the specified target prefixes for the attack
    # Original prefixes
    original_target_prefixes = ["203.0.113.0/24", "198.51.100.0/24", "192.0.2.0/24"]
    
    # Add more specific prefixes for each original prefix to increase packet count
    target_prefixes = []
    for prefix in original_target_prefixes:
        target_prefixes.append(prefix)  # Original prefix
        # Add more specific prefixes (subdivisions of the original prefix)
        ip, mask = prefix.split('/')
        mask = int(mask)
        if mask < 30:  # Ensure we don't exceed /32
            # Add two more specific prefixes
            octets = ip.split('.')
            if mask == 24:
                # For a /24, add two /25s
                target_prefixes.append(f"{ip}/{mask+1}")  # First half
                # Second half - increment the last octet appropriately
                new_last_octet = int(octets[3]) + 128  # For /24 to /25, add 128
                target_prefixes.append(f"{octets[0]}.{octets[1]}.{octets[2]}.{new_last_octet}/{mask+1}")
    
    print(f"Target prefixes for hijacking: {target_prefixes}")
    
    # Use specific router IDs as provided
    attacker_router_id = "192.168.19.1"
    victim_router_id = "192.168.186.1"
    
    # Use specific source IPv4
    source_ipv4 = "10.51.90.5"
    
    # For each neighbor of the attacker, send a hijacked announcement
    # If we don't have specific neighbors, we'll create a reasonable attack pattern
    neighbors = topology.get(attacker_as, {}).get("neighbors", [])
    if not neighbors:
        # If not found in topology, use a reasonable set of neighbors for the attack
        neighbors = [neighbor for neighbor in topology.keys() if neighbor != attacker_as and neighbor != victim_as]
        if not neighbors:
            # Fallback: Create more fictional neighbors to increase packet count
            neighbors = [12345, 23456, 34567, 45678, 56789, 67890, 78901, 89012]
    
    # Helper function to create MAC addresses based on IP
    def create_mac_from_ip(ip):
        octets = list(map(int, ip.split('.')))
        return f"00:{octets[0]:02x}:{octets[1]:02x}:{octets[2]:02x}:{octets[3]:02x}:00"
    
    # Helper function to send a BGP UPDATE with a hijacked prefix
    def send_hijacked_update(src_ip, dst_ip, prefix, as_path=None, with_ack=True):
        nonlocal attack_packets
        
        # Create MAC addresses
        src_mac = create_mac_from_ip(src_ip)
        dst_mac = create_mac_from_ip(dst_ip)
        
        # Generate port numbers
        src_port = random.randint(30000, 65000)
        dst_port = 179  # Standard BGP port
        
        # Initialize sequence numbers
        seq_a = random.randint(1000, 10000)
        seq_b = random.randint(1000, 10000)
        
        # Create IP ID
        src_ip_id = random.randint(PREFIX_HIJACK_ID_RANGE[0], PREFIX_HIJACK_ID_RANGE[1])
        
        # Create path attributes for the hijack
        # ORIGIN - set to IGP (0) to appear legitimate
        origin = BGPPathAttr(type_flags=0x40, type_code=1)
        origin.attribute = BGPPAOrigin(origin=0)  # IGP = 0
        
        # AS_PATH - showing the attacker as origin or using provided AS path
        as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
        as_path_segment = BGPPAASPath()
        
        if as_path is None:
            # Default path with just the attacker
            segment = BGPPAASPath.ASPathSegment(
                segment_type=2,  # AS_SEQUENCE
                segment_length=1,
                segment_value=[attacker_as]  # Only include attacker's AS
            )
        else:
            # Use provided AS path
            segment = BGPPAASPath.ASPathSegment(
                segment_type=2,  # AS_SEQUENCE
                segment_length=len(as_path),
                segment_value=as_path
            )
        
        as_path_segment.segments = [segment]
        as_path_attr.attribute = as_path_segment
        
        # NEXT_HOP - attacker's router
        next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
        next_hop_attr.attribute = BGPPANextHop(next_hop=src_ip)
        
        # MED - low value to appear more attractive
        med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
        med_attr.attribute = BGPPAMultiExitDisc(med=100)
        
        # LOCAL_PREF - high value to appear more attractive
        local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
        local_pref_attr.attribute = BGPPALocalPref(local_pref=300)  # Higher than normal
        
        # COMMUNITIES attribute - to make it look more legitimate
        communities_list = []
        communities_list.append(BGPPACommunity(community=attacker_as<<16|100))
        communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
        communities_attr.attribute = communities_list
        
        # Create the hijack UPDATE message
        hijack_update = BGPHeader(type=2)/BGPUpdate()
        hijack_update.path_attr = [
            origin,
            as_path_attr,
            next_hop_attr,
            med_attr,
            local_pref_attr,
            communities_attr
        ]
        
        # Add the hijacked prefix to NLRI
        hijack_update.nlri = [BGPNLRI_IPv4(prefix=prefix)]
        
        # Create the packet with the specific ID
        hijack_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ip, dst=dst_ip, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=src_port, dport=dst_port, flags="PA", seq=seq_a, ack=seq_b, window=16384)/hijack_update

        if len(hijack_pkt) < 60:
            pad_len = 60 - len(hijack_pkt)
            hijack_pkt = hijack_pkt/Padding(load=b'\x00' * pad_len)
        
        attack_packets.append(hijack_pkt)
        
        # If requested, generate an ACK for the hijack announcement
        if with_ack:
            dst_ip_id = random.randint(PREFIX_HIJACK_ID_RANGE[0], PREFIX_HIJACK_ID_RANGE[1])
            hijack_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ip, dst=src_ip, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dst_port, dport=src_port, flags="A", seq=seq_b, ack=seq_a + len(hijack_update), window=16384)

            if len(hijack_ack) < 60:
                pad_len = 60 - len(hijack_ack)
                hijack_ack = hijack_ack/Padding(load=b'\x00' * pad_len)
            
            attack_packets.append(hijack_ack)
            
        return len(hijack_update), hijack_update.nlri[0].prefix
    
    # Helper function to send a withdrawal for a prefix
    def send_withdrawal(src_ip, dst_ip, prefix):
        nonlocal attack_packets
        
        # Create MAC addresses
        src_mac = create_mac_from_ip(src_ip)
        dst_mac = create_mac_from_ip(dst_ip)
        
        # Generate port numbers
        src_port = random.randint(30000, 65000)
        dst_port = 179  # Standard BGP port
        
        # Initialize sequence numbers
        seq_a = random.randint(1000, 10000)
        seq_b = random.randint(1000, 10000)
        
        # Create IP ID
        src_ip_id = random.randint(PREFIX_HIJACK_ID_RANGE[0], PREFIX_HIJACK_ID_RANGE[1])
        
        # Create the withdrawal UPDATE message
        withdrawal_update = BGPHeader(type=2)/BGPUpdate()
        withdrawal_update.withdrawn_routes = [BGPNLRI_IPv4(prefix=prefix)]
        
        # Create the packet
        withdrawal_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ip, dst=dst_ip, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=src_port, dport=dst_port, flags="PA", seq=seq_a, ack=seq_b, window=16384)/withdrawal_update

        if len(withdrawal_pkt) < 60:
            pad_len = 60 - len(withdrawal_pkt)
            withdrawal_pkt = withdrawal_pkt/Padding(load=b'\x00' * pad_len)
        
        attack_packets.append(withdrawal_pkt)
        
        # Generate an ACK for the withdrawal
        dst_ip_id = random.randint(PREFIX_HIJACK_ID_RANGE[0], PREFIX_HIJACK_ID_RANGE[1])
        withdrawal_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ip, dst=src_ip, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dst_port, dport=src_port, flags="A", seq=seq_b, ack=seq_a + len(withdrawal_update), window=16384)

        if len(withdrawal_ack) < 60:
            pad_len = 60 - len(withdrawal_ack)
            withdrawal_ack = withdrawal_ack/Padding(load=b'\x00' * pad_len)
        
        attack_packets.append(withdrawal_ack)
        
        return len(withdrawal_update)
    
    # Helper function to send a KEEPALIVE
    def send_keepalive(src_ip, dst_ip):
        nonlocal attack_packets
        
        # Create MAC addresses
        src_mac = create_mac_from_ip(src_ip)
        dst_mac = create_mac_from_ip(dst_ip)
        
        # Generate port numbers
        src_port = random.randint(30000, 65000)
        dst_port = 179  # Standard BGP port
        
        # Initialize sequence numbers
        seq_a = random.randint(1000, 10000)
        seq_b = random.randint(1000, 10000)
        
        # Create IP ID
        src_ip_id = random.randint(PREFIX_HIJACK_ID_RANGE[0], PREFIX_HIJACK_ID_RANGE[1])
        
        # Create the KEEPALIVE message
        keepalive = BGPHeader(type=4)  # Type 4 = KEEPALIVE
        
        # Create the packet
        keepalive_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ip, dst=dst_ip, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=src_port, dport=dst_port, flags="PA", seq=seq_a, ack=seq_b, window=16384)/keepalive

        if len(keepalive_pkt) < 60:
            pad_len = 60 - len(keepalive_pkt)
            keepalive_pkt = keepalive_pkt/Padding(load=b'\x00' * pad_len)
        
        attack_packets.append(keepalive_pkt)
        
        # Generate an ACK for the KEEPALIVE
        dst_ip_id = random.randint(PREFIX_HIJACK_ID_RANGE[0], PREFIX_HIJACK_ID_RANGE[1])
        keepalive_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ip, dst=src_ip, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dst_port, dport=src_port, flags="A", seq=seq_b, ack=seq_a + len(keepalive), window=16384)

        if len(keepalive_ack) < 60:
            pad_len = 60 - len(keepalive_ack)
            keepalive_ack = keepalive_ack/Padding(load=b'\x00' * pad_len)
        
        attack_packets.append(keepalive_ack)
        
        return len(keepalive)
    
    # For each neighbor and each target prefix
    for neighbor in neighbors:
        if neighbor == victim_as:
            # Skip sending the hijacked route to the actual owner
            continue
            
        print(f"  Sending hijacked announcements to AS{neighbor}...")
        
        # Get interface IPs for this session - fixed to check ip_allocations not topology
        if (attacker_as in ip_allocations and 
            "interfaces" in ip_allocations[attacker_as] and 
            neighbor in ip_allocations[attacker_as]["interfaces"]):
            src_ipv4 = ip_allocations[attacker_as]["interfaces"][neighbor]
            dst_ipv4 = ip_allocations[neighbor]["interfaces"][attacker_as]
        else:
            # Use the specified source IPv4 and create a reasonable destination
            src_ipv4 = source_ipv4
            dst_ipv4 = f"10.{neighbor//256}.{neighbor%256}.1"
        
        # Start with some KEEPALIVE messages to establish context
        for _ in range(2):
            send_keepalive(src_ipv4, dst_ipv4)
            
        # Send a hijack for each target prefix
        for prefix in target_prefixes:
            # Basic hijack with just attacker in AS_PATH
            update_len, prefix_sent = send_hijacked_update(src_ipv4, dst_ipv4, prefix)
            print(f"    - Sent hijacked announcement for {prefix_sent}")
        
        # Send a KEEPALIVE after the basic announcements
        send_keepalive(src_ipv4, dst_ipv4)
        
        # For some neighbors, try advanced AS path manipulation attacks
        if neighbors.index(neighbor) % 3 == 0:  # Every 3rd neighbor gets more complex attacks
            # Try path prepending variations to make the route look more legitimate
            for prefix in original_target_prefixes[:2]:  # Just use the first two original prefixes
                # Create a fake transit AS path to hide the attacker
                transit_as1 = attacker_as + 1000
                transit_as2 = attacker_as + 2000
                
                # Version 1: One transit AS
                send_hijacked_update(src_ipv4, dst_ipv4, prefix, [attacker_as, transit_as1])
                
                # Version 2: Two transit ASes
                send_hijacked_update(src_ipv4, dst_ipv4, prefix, [attacker_as, transit_as1, transit_as2])
                
                # Add a KEEPALIVE between different path variations
                send_keepalive(src_ipv4, dst_ipv4)
            
            # Try a "sub-prefix" attack - hijack more specific prefixes
            for prefix in original_target_prefixes:
                ip, mask = prefix.split('/')
                mask = int(mask)
                if mask < 24:  # Ensure we don't exceed /24 in this specific attack
                    # Send more specific prefixes
                    specific_prefix = f"{ip}/{mask+8}"  # Much more specific
                    send_hijacked_update(src_ipv4, dst_ipv4, specific_prefix)
                    print(f"    - Sent sub-prefix hijack for {specific_prefix}")
        
        # For some neighbors, perform withdrawal and re-announcement to simulate route flapping
        if neighbors.index(neighbor) % 2 == 0:  # Every 2nd neighbor gets withdrawal patterns
            for prefix in original_target_prefixes:
                # Withdraw the prefix
                send_withdrawal(src_ipv4, dst_ipv4, prefix)
                print(f"    - Sent withdrawal for {prefix}")
                
                # Re-announce it with a slightly different AS path
                transit_as = attacker_as + 500
                send_hijacked_update(src_ipv4, dst_ipv4, prefix, [attacker_as, transit_as])
                print(f"    - Sent re-announcement for {prefix} with transit AS{transit_as}")
        
        # Send a final KEEPALIVE
        send_keepalive(src_ipv4, dst_ipv4)
    
    print(f"Generated {len(attack_packets)} packets for extended prefix hijacking attack")
    
    # Save attack packets to separate PCAP
    pcap_dir = "/Users/shadimoteali/PhD/BGP_Traffic_Generation/pcaps"
    os.makedirs(pcap_dir, exist_ok=True)
    
    attack_pcap_file = f"{pcap_dir}/bgp_prefix_hijacking_attack_extended.pcap"
    wrpcap(attack_pcap_file, attack_packets)
    print(f"Saved attack packets to {attack_pcap_file}")
    
    return attack_packets

# Generate the prefix hijacking attack
print("\n[+] Generating extended prefix hijacking attack with specified target prefixes")
hijack_packets = generate_prefix_hijacking_attack(topology, ip_allocations)

# Add attack packets to the comprehensive pcap
if len(hijack_packets) > 0:
    # Path to the comprehensive pcap file
    comprehensive_pcap = "/Users/shadimoteali/PhD/BGP_Traffic_Generation/pcaps/realistic_bgp_complete_scenarios.pcap"
    
    try:
        # Read the existing pcap file
        existing_packets = rdpcap(comprehensive_pcap)
        
        # Extract the actual packets from the PacketList object
        # and combine with our attack packets
        combined_packets = existing_packets.res + hijack_packets
        
        # Write back the combined pcap
        wrpcap(comprehensive_pcap, combined_packets)
        print(f"Added {len(hijack_packets)} attack packets to comprehensive pcap file")
    except FileNotFoundError:
        # If the file doesn't exist yet, just save our attack packets
        wrpcap(comprehensive_pcap, hijack_packets)
        print(f"Created new comprehensive pcap file with {len(hijack_packets)} attack packets")


    


# ------------ Scenario 2: Path manipulation ------------

def generate_path_manipulation_attack(topology, ip_allocations):
    """Generate BGP path manipulation attack packets - extended version
    
    This attack falsely advertises a better path to a destination than actually exists,
    potentially creating a man-in-the-middle situation.
    """
    attack_packets = []
    
    # Use the specified AS numbers for attacker and victim
    attacker_as = 39224  # Attacker AS
    victim_as = 47066    # Victim AS
    
    print(f"[+] Generating extended path manipulation attack: AS{attacker_as} manipulating path to AS{victim_as}")
    
    # Use the specified target prefixes for the attack
    original_target_prefixes = ["203.0.113.0/24", "198.51.100.0/24", "192.0.2.0/24"]
    
    # Add more specific prefixes for each original prefix to increase packet count
    target_prefixes = []
    for prefix in original_target_prefixes:
        target_prefixes.append(prefix)  # Original prefix
        # Add more specific prefixes (subdivisions of the original prefix)
        ip, mask = prefix.split('/')
        mask = int(mask)
        if mask < 30:  # Ensure we don't exceed /32
            # Add two more specific prefixes
            octets = ip.split('.')
            if mask == 24:
                # For a /24, add two /25s
                target_prefixes.append(f"{ip}/{mask+1}")  # First half
                # Second half - increment the last octet appropriately
                new_last_octet = int(octets[3]) + 128  # For /24 to /25, add 128
                target_prefixes.append(f"{octets[0]}.{octets[1]}.{octets[2]}.{new_last_octet}/{mask+1}")
    
    print(f"Target prefixes for path manipulation: {target_prefixes}")
    
    # Use specific source IPv4
    source_ipv4 = "10.51.90.5"
    
    # For each neighbor of the attacker, send a manipulated path announcement
    neighbors = topology.get(attacker_as, {}).get("neighbors", [])
    if not neighbors:
        # If not found in topology, use a reasonable set of neighbors for the attack
        neighbors = [neighbor for neighbor in topology.keys() if neighbor != attacker_as and neighbor != victim_as]
        if not neighbors:
            # Fallback: Create more fictional neighbors to increase packet count
            neighbors = [12345, 23456, 34567, 45678, 56789, 67890]
    
    # Helper function to create MAC addresses based on IP
    def create_mac_from_ip(ip):
        octets = list(map(int, ip.split('.')))
        return f"00:{octets[0]:02x}:{octets[1]:02x}:{octets[2]:02x}:{octets[3]:02x}:00"
    
    # Helper function to send a BGP UPDATE with path manipulation
    def send_manipulated_update(src_ip, dst_ip, prefix, as_path, with_ack=True):
        nonlocal attack_packets
        
        # Create MAC addresses
        src_mac = create_mac_from_ip(src_ip)
        dst_mac = create_mac_from_ip(dst_ip)
        
        # Generate port numbers
        src_port = random.randint(30000, 65000)
        dst_port = 179  # Standard BGP port
        
        # Initialize sequence numbers
        seq_a = random.randint(1000, 10000)
        seq_b = random.randint(1000, 10000)
        
        # Create IP ID
        src_ip_id = random.randint(PATH_MANIP_ID_RANGE[0], PATH_MANIP_ID_RANGE[1])
        
        # Create path attributes for the manipulation
        # ORIGIN - set to IGP (0) to appear legitimate
        origin = BGPPathAttr(type_flags=0x40, type_code=1)
        origin.attribute = BGPPAOrigin(origin=0)  # IGP = 0
        
        # AS_PATH - showing the falsified path
        as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
        as_path_segment = BGPPAASPath()
        segment = BGPPAASPath.ASPathSegment(
            segment_type=2,  # AS_SEQUENCE
            segment_length=len(as_path),
            segment_value=as_path
        )
        as_path_segment.segments = [segment]
        as_path_attr.attribute = as_path_segment
        
        # NEXT_HOP - attacker's router
        next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
        next_hop_attr.attribute = BGPPANextHop(next_hop=src_ip)
        
        # MED - low value to appear more attractive
        med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
        med_attr.attribute = BGPPAMultiExitDisc(med=100)
        
        # LOCAL_PREF - high value to appear more attractive
        local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
        local_pref_attr.attribute = BGPPALocalPref(local_pref=300)  # Higher than normal
        
        # COMMUNITIES attribute - to make it look more legitimate
        communities_list = []
        communities_list.append(BGPPACommunity(community=attacker_as<<16|100))
        communities_list.append(BGPPACommunity(community=victim_as<<16|100))
        communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
        communities_attr.attribute = communities_list
        
        # Create the manipulation UPDATE message
        manip_update = BGPHeader(type=2)/BGPUpdate()
        manip_update.path_attr = [
            origin,
            as_path_attr,
            next_hop_attr,
            med_attr,
            local_pref_attr,
            communities_attr
        ]
        
        # Add the target prefix to NLRI
        manip_update.nlri = [BGPNLRI_IPv4(prefix=prefix)]
        
        # Create the packet
        manip_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ip, dst=dst_ip, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=src_port, dport=dst_port, flags="PA", seq=seq_a, ack=seq_b, window=16384)/manip_update

        if len(manip_pkt) < 60:
            pad_len = 60 - len(manip_pkt)
            manip_pkt = manip_pkt/Padding(load=b'\x00' * pad_len)
        
        attack_packets.append(manip_pkt)
        
        # If requested, generate an ACK for the manipulation announcement
        if with_ack:
            dst_ip_id = random.randint(PATH_MANIP_ID_RANGE[0], PATH_MANIP_ID_RANGE[1])
            manip_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ip, dst=src_ip, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dst_port, dport=src_port, flags="A", seq=seq_b, ack=seq_a + len(manip_update), window=16384)

            if len(manip_ack) < 60:
                pad_len = 60 - len(manip_ack)
                manip_ack = manip_ack/Padding(load=b'\x00' * pad_len)
            
            attack_packets.append(manip_ack)
        
        return len(manip_update), prefix
    
    # Helper function to send a multi-prefix manipulated update
    def send_multi_prefix_manip(src_ip, dst_ip, prefixes, as_path):
        nonlocal attack_packets
        
        # Create MAC addresses
        src_mac = create_mac_from_ip(src_ip)
        dst_mac = create_mac_from_ip(dst_ip)
        
        # Generate port numbers
        src_port = random.randint(30000, 65000)
        dst_port = 179  # Standard BGP port
        
        # Initialize sequence numbers
        seq_a = random.randint(1000, 10000)
        seq_b = random.randint(1000, 10000)
        
        # Create IP ID
        src_ip_id = random.randint(PATH_MANIP_ID_RANGE[0], PATH_MANIP_ID_RANGE[1])
        
        # Create path attributes for the manipulation
        origin = BGPPathAttr(type_flags=0x40, type_code=1)
        origin.attribute = BGPPAOrigin(origin=0)  # IGP = 0
        
        as_path_attr = BGPPathAttr(type_flags=0x40, type_code=2)
        as_path_segment = BGPPAASPath()
        segment = BGPPAASPath.ASPathSegment(
            segment_type=2, segment_length=len(as_path), segment_value=as_path
        )
        as_path_segment.segments = [segment]
        as_path_attr.attribute = as_path_segment
        
        next_hop_attr = BGPPathAttr(type_flags=0x40, type_code=3)
        next_hop_attr.attribute = BGPPANextHop(next_hop=src_ip)
        
        med_attr = BGPPathAttr(type_flags=0x80, type_code=4)
        med_attr.attribute = BGPPAMultiExitDisc(med=100)
        
        local_pref_attr = BGPPathAttr(type_flags=0x40, type_code=5)
        local_pref_attr.attribute = BGPPALocalPref(local_pref=300)
        
        communities_list = []
        communities_list.append(BGPPACommunity(community=attacker_as<<16|100))
        communities_list.append(BGPPACommunity(community=victim_as<<16|100))
        communities_attr = BGPPathAttr(type_flags=0x40|0x80, type_code=8)
        communities_attr.attribute = communities_list
        
        # Create the manipulation UPDATE message
        multi_update = BGPHeader(type=2)/BGPUpdate()
        multi_update.path_attr = [
            origin,
            as_path_attr,
            next_hop_attr,
            med_attr,
            local_pref_attr,
            communities_attr
        ]
        
        # Add all prefixes to NLRI
        multi_update.nlri = [BGPNLRI_IPv4(prefix=p) for p in prefixes]
        
        # Create the packet
        multi_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ip, dst=dst_ip, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=src_port, dport=dst_port, flags="PA", seq=seq_a, ack=seq_b, window=16384)/multi_update

        if len(multi_pkt) < 60:
            pad_len = 60 - len(multi_pkt)
            multi_pkt = multi_pkt/Padding(load=b'\x00' * pad_len)
        
        attack_packets.append(multi_pkt)
        
        # Generate an ACK for the manipulation announcement
        dst_ip_id = random.randint(PATH_MANIP_ID_RANGE[0], PATH_MANIP_ID_RANGE[1])
        multi_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ip, dst=src_ip, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dst_port, dport=src_port, flags="A", seq=seq_b, ack=seq_a + len(multi_update), window=16384)

        if len(multi_ack) < 60:
            pad_len = 60 - len(multi_ack)
            multi_ack = multi_ack/Padding(load=b'\x00' * pad_len)
        
        attack_packets.append(multi_ack)
        
        return len(multi_update)
    
    # Helper function to send a withdrawal for a prefix
    def send_withdrawal(src_ip, dst_ip, prefix):
        nonlocal attack_packets
        
        # Create MAC addresses
        src_mac = create_mac_from_ip(src_ip)
        dst_mac = create_mac_from_ip(dst_ip)
        
        # Generate port numbers
        src_port = random.randint(30000, 65000)
        dst_port = 179  # Standard BGP port
        
        # Initialize sequence numbers
        seq_a = random.randint(1000, 10000)
        seq_b = random.randint(1000, 10000)
        
        # Create IP ID
        src_ip_id = random.randint(PATH_MANIP_ID_RANGE[0], PATH_MANIP_ID_RANGE[1])
        
        # Create the withdrawal UPDATE message
        withdrawal_update = BGPHeader(type=2)/BGPUpdate()
        withdrawal_update.withdrawn_routes = [BGPNLRI_IPv4(prefix=prefix)]
        
        # Create the packet
        withdrawal_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ip, dst=dst_ip, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=src_port, dport=dst_port, flags="PA", seq=seq_a, ack=seq_b, window=16384)/withdrawal_update

        if len(withdrawal_pkt) < 60:
            pad_len = 60 - len(withdrawal_pkt)
            withdrawal_pkt = withdrawal_pkt/Padding(load=b'\x00' * pad_len)
        
        attack_packets.append(withdrawal_pkt)
        
        # Generate an ACK for the withdrawal
        dst_ip_id = random.randint(PATH_MANIP_ID_RANGE[0], PATH_MANIP_ID_RANGE[1])
        withdrawal_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ip, dst=src_ip, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dst_port, dport=src_port, flags="A", seq=seq_b, ack=seq_a + len(withdrawal_update), window=16384)

        if len(withdrawal_ack) < 60:
            pad_len = 60 - len(withdrawal_ack)
            withdrawal_ack = withdrawal_ack/Padding(load=b'\x00' * pad_len)
        
        attack_packets.append(withdrawal_ack)
        
        return len(withdrawal_update)
    
    # Helper function to send a KEEPALIVE
    def send_keepalive(src_ip, dst_ip):
        nonlocal attack_packets
        
        # Create MAC addresses
        src_mac = create_mac_from_ip(src_ip)
        dst_mac = create_mac_from_ip(dst_ip)
        
        # Generate port numbers
        src_port = random.randint(30000, 65000)
        dst_port = 179  # Standard BGP port
        
        # Initialize sequence numbers
        seq_a = random.randint(1000, 10000)
        seq_b = random.randint(1000, 10000)
        
        # Create IP ID
        src_ip_id = random.randint(PATH_MANIP_ID_RANGE[0], PATH_MANIP_ID_RANGE[1])
        
        # Create the KEEPALIVE message
        keepalive = BGPHeader(type=4)  # Type 4 = KEEPALIVE
        
        # Create the packet
        keepalive_pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ip, dst=dst_ip, ttl=1, flags="DF", tos=0xC0, id=src_ip_id)/TCP(sport=src_port, dport=dst_port, flags="PA", seq=seq_a, ack=seq_b, window=16384)/keepalive

        if len(keepalive_pkt) < 60:
            pad_len = 60 - len(keepalive_pkt)
            keepalive_pkt = keepalive_pkt/Padding(load=b'\x00' * pad_len)
        
        attack_packets.append(keepalive_pkt)
        
        # Generate an ACK for the KEEPALIVE
        dst_ip_id = random.randint(PATH_MANIP_ID_RANGE[0], PATH_MANIP_ID_RANGE[1])
        keepalive_ack = Ether(src=dst_mac, dst=src_mac)/IP(src=dst_ip, dst=src_ip, ttl=1, flags=0, tos=0xC0, id=dst_ip_id)/TCP(sport=dst_port, dport=src_port, flags="A", seq=seq_b, ack=seq_a + len(keepalive), window=16384)

        if len(keepalive_ack) < 60:
            pad_len = 60 - len(keepalive_ack)
            keepalive_ack = keepalive_ack/Padding(load=b'\x00' * pad_len)
        
        attack_packets.append(keepalive_ack)
        
        return len(keepalive)
    
    # Create a few transit ASes for path manipulation variations
    transit_ases = [
        attacker_as + 1000,  # Transit AS 1
        attacker_as + 2000,  # Transit AS 2
        attacker_as + 3000,  # Transit AS 3
        attacker_as + 4000,  # Transit AS 4
    ]
    
    # For each neighbor and each target prefix
    for neighbor in neighbors:
        if neighbor == victim_as:
            # Skip sending the manipulated path to the actual owner
            continue
            
        print(f"  Sending path manipulations to AS{neighbor}...")
        
        # Get interface IPs for this session
        if (attacker_as in ip_allocations and 
            "interfaces" in ip_allocations[attacker_as] and 
            neighbor in ip_allocations[attacker_as]["interfaces"]):
            src_ipv4 = ip_allocations[attacker_as]["interfaces"][neighbor]
            dst_ipv4 = ip_allocations[neighbor]["interfaces"][attacker_as]
        else:
            # Use the specified source IPv4 and create a reasonable destination
            src_ipv4 = source_ipv4
            dst_ipv4 = f"10.{neighbor//256}.{neighbor%256}.1"
        
        # Start with some KEEPALIVE messages
        for _ in range(2):
            send_keepalive(src_ipv4, dst_ipv4)
        
        # SECTION 1: Basic path manipulation
        print("    - Sending basic path manipulations (attacker->victim)...")
        # This is the basic attack - falsely claiming a direct path to victim
        basic_as_path = [attacker_as, victim_as]
        
        # Send the basic manipulation for each prefix
        for prefix in target_prefixes:
            update_len, prefix_sent = send_manipulated_update(src_ipv4, dst_ipv4, prefix, basic_as_path)
            print(f"      - Sent basic path manipulation for {prefix_sent}")
        
        # Add a KEEPALIVE
        send_keepalive(src_ipv4, dst_ipv4)
        
        # SECTION 2: Advanced path manipulations with different fake paths
        print("    - Sending advanced path manipulations with fake transit ASes...")
        
        # Several false AS paths claiming various beneficial paths
        manipulated_paths = [
            # False path showing a path via one transit AS
            [attacker_as, transit_ases[0], victim_as],
            
            # False path showing a path via multiple transit ASes
            [attacker_as, transit_ases[0], transit_ases[1], victim_as],
            
            # False path showing a very favorable path
            [attacker_as, victim_as, victim_as],  # Double victim_as to make it appear more preferred
        ]
        
        # Apply each manipulated path to selected prefixes
        for path_idx, fake_path in enumerate(manipulated_paths):
            # Use first 3 prefixes for the first path, next 3 for the next path, etc.
            start_idx = (path_idx * 3) % len(target_prefixes)
            end_idx = min(start_idx + 3, len(target_prefixes))
            selected_prefixes = target_prefixes[start_idx:end_idx]
            
            for prefix in selected_prefixes:
                update_len, prefix_sent = send_manipulated_update(src_ipv4, dst_ipv4, prefix, fake_path)
                path_str = " -> ".join([f"AS{asn}" for asn in fake_path])
                print(f"      - Sent path manipulation with fake path {path_str} for {prefix_sent}")
            
            # Add a KEEPALIVE after each set
            send_keepalive(src_ipv4, dst_ipv4)
        
        # SECTION 3: Multi-prefix updates with path manipulation
        print("    - Sending multi-prefix updates with path manipulation...")
        
        # Send all original prefixes in a single update with manipulated path
        send_multi_prefix_manip(src_ipv4, dst_ipv4, original_target_prefixes, 
                               [attacker_as, transit_ases[2], victim_as])
        print(f"      - Sent multi-prefix manipulation for {len(original_target_prefixes)} prefixes")
        
        # SECTION 4: Path manipulation combined with withdrawal/re-announcement
        print("    - Performing withdrawal and re-announcement with path manipulation...")
        
        # For a subset of prefixes, withdraw and then re-announce with different paths
        for prefix in original_target_prefixes:
            # First withdraw the prefix
            send_withdrawal(src_ipv4, dst_ipv4, prefix)
            print(f"      - Withdrew prefix {prefix}")
            
            # Re-announce with a new manipulated path
            new_path = [attacker_as, transit_ases[3], victim_as]
            update_len, prefix_sent = send_manipulated_update(src_ipv4, dst_ipv4, prefix, new_path)
            print(f"      - Re-announced {prefix_sent} with new manipulated path")
        
        # Add a KEEPALIVE after the withdrawal/re-announcement
        send_keepalive(src_ipv4, dst_ipv4)
        
        # SECTION 5: Path poisoning (include ASes that shouldn't be in the path)
        # This is an advanced attack to make the route more attractive by suggesting
        # the victim is closer than it actually is
        print("    - Performing path poisoning attacks...")
        
        # Create some interesting poisoned paths
        poisoned_paths = [
            # Path including the neighbor AS (to make it more trusted)
            [attacker_as, neighbor, victim_as],
            
            # Path suggesting it's already traversing major transit networks
            [attacker_as, 7018, 3356, victim_as],  # AT&T and Level3 ASNs
            
            # Path suggesting very short path through popular IXPs
            [attacker_as, 6695, victim_as]  # DE-CIX ASN
        ]
        
        for path_idx, poisoned_path in enumerate(poisoned_paths):
            # Use a different prefix for each poisoned path
            prefix_idx = path_idx % len(original_target_prefixes)
            prefix = original_target_prefixes[prefix_idx]
            
            update_len, prefix_sent = send_manipulated_update(src_ipv4, dst_ipv4, prefix, poisoned_path)
            path_str = " -> ".join([f"AS{asn}" for asn in poisoned_path])
            print(f"      - Sent poisoned path {path_str} for {prefix_sent}")
        
        # Final KEEPALIVE
        send_keepalive(src_ipv4, dst_ipv4)
    
    print(f"Generated {len(attack_packets)} packets for extended path manipulation attack")
    
    # Save attack packets to separate PCAP
    pcap_dir = "/Users/shadimoteali/PhD/BGP_Traffic_Generation/pcaps"
    os.makedirs(pcap_dir, exist_ok=True)
    
    attack_pcap_file = f"{pcap_dir}/bgp_path_manipulation_attack_extended.pcap"
    wrpcap(attack_pcap_file, attack_packets)
    print(f"Saved attack packets to {attack_pcap_file}")
    
    return attack_packets

# Generate the path manipulation attack
print("\n[+] Generating extended path manipulation attack with falsified AS paths")
path_manip_packets = generate_path_manipulation_attack(topology, ip_allocations)

# Add attack packets to the comprehensive pcap
if len(path_manip_packets) > 0:
    # Path to the comprehensive pcap file
    comprehensive_pcap = "/Users/shadimoteali/PhD/BGP_Traffic_Generation/pcaps/realistic_bgp_complete_scenarios.pcap"
    
    try:
        # Read the existing pcap file
        existing_packets = rdpcap(comprehensive_pcap)
        
        # Extract the actual packets from the PacketList object
        # and combine with our attack packets
        combined_packets = existing_packets.res + path_manip_packets
        
        # Write back the combined pcap
        wrpcap(comprehensive_pcap, combined_packets)
        print(f"Added {len(path_manip_packets)} path manipulation packets to comprehensive pcap file")
    except FileNotFoundError:
        # If the file doesn't exist yet, just save our attack packets
        wrpcap(comprehensive_pcap, path_manip_packets)
        print(f"Created comprehensive pcap file with {len(path_manip_packets)} path manipulation packets")    





# ------------ Scenario 3#: DOS ------------
def generate_bgp_dos_attack(topology, ip_allocations):
    """Generate BGP Denial of Service attack with UPDATE messages only"""
    attack_packets = []
    
    # Attacker and victim AS numbers
    attacker_as = 39224
    victim_as = 47066
    
    # Log start of process
    print(f"[+] Generating simplified BGP DoS attack: AS{attacker_as} flooding AS{victim_as}")
    
    # Create fixed IP addresses if needed
    src_ipv4 = "10.39.22.4"  # Fixed IP based on attacker AS
    dst_ipv4 = "10.47.6.6"   # Fixed IP based on victim AS
    
    # Create fixed MAC addresses
    src_mac = "00:11:22:33:44:55"
    dst_mac = "aa:bb:cc:dd:ee:ff"
    
    # Fixed port numbers
    sport = 50000
    dport = 179
    
    # Initial sequence number
    seq_num = 10000
    ack_num = 20000
    
    # Number of UPDATE messages to generate
    num_updates = 420
    
    print(f"  Generating {num_updates} UPDATE messages...")
    
    # Create UPDATE messages
    for i in range(num_updates):
        # Use correct IP ID from DOS_ATTACK_ID_RANGE
        ip_id = DOS_ATTACK_ID_RANGE[0] + i % (DOS_ATTACK_ID_RANGE[1] - DOS_ATTACK_ID_RANGE[0])
        
        # Print first few IP IDs for verification
        if i < 5:
            print(f" DEBUG: Using IP ID: 0x{ip_id:X} (should be in range (0x{DOS_ATTACK_ID_RANGE[0]:X}, 0x{DOS_ATTACK_ID_RANGE[1]:X}))")
        
        # Create basic path attributes
        origin = BGPPathAttr(type_flags=0x40, type_code=1)
        origin.attribute = BGPPAOrigin(origin=0)
        
        as_path = BGPPathAttr(type_flags=0x40, type_code=2)
        as_path_seg = BGPPAASPath()
        segment = BGPPAASPath.ASPathSegment(segment_type=2, segment_length=1, segment_value=[attacker_as])
        as_path_seg.segments = [segment]
        as_path.attribute = as_path_seg
        
        next_hop = BGPPathAttr(type_flags=0x40, type_code=3)
        next_hop.attribute = BGPPANextHop(next_hop=src_ipv4)
        
        # Create UPDATE message with a unique prefix for each message
        update = BGPHeader(type=2)/BGPUpdate()
        update.path_attr = [origin, as_path, next_hop]
        update.nlri = [BGPNLRI_IPv4(prefix=f"192.168.{i//256}.{i%256}/32")]
        
        # Create packet with correct IP ID in the DoS range
        pkt = Ether(src=src_mac, dst=dst_mac)/IP(src=src_ipv4, dst=dst_ipv4, ttl=1, id=ip_id)/TCP(sport=sport, dport=dport, seq=seq_num, ack=ack_num, flags="PA")/update
        
        # Add packet to list
        attack_packets.append(pkt)
        
        # Increment sequence number
        seq_num += len(update)
    
    print(f"Generated {len(attack_packets)} DoS attack packets with IP IDs in range {DOS_ATTACK_ID_RANGE}")
    
    # Save to separate pcap
    pcap_dir = "/Users/shadimoteali/PhD/BGP_Traffic_Generation/pcaps"
    os.makedirs(pcap_dir, exist_ok=True)
    attack_pcap = f"{pcap_dir}/bgp_dos_attack.pcap"
    wrpcap(attack_pcap, attack_packets)
    print(f"Saved DoS attack packets to {attack_pcap}")
    
    return attack_packets

# After calling the function, add packets to the comprehensive pcap
dos_attack_packets = generate_bgp_dos_attack(topology, ip_allocations)

# Add to comprehensive pcap
if dos_attack_packets:
    comprehensive_pcap = "/Users/shadimoteali/PhD/BGP_Traffic_Generation/pcaps/realistic_bgp_complete_scenarios.pcap"
    
    # Read existing packets
    try:
        existing_packets = rdpcap(comprehensive_pcap)
        # FIX: Use list() to convert PacketList to a regular list
        combined_packets = list(existing_packets) + dos_attack_packets
        
        # Write combined pcap
        wrpcap(comprehensive_pcap, combined_packets)
        print(f"Added {len(dos_attack_packets)} DoS packets to comprehensive pcap file")
    except Exception as e:
        print(f"Error adding packets to comprehensive pcap: {e}")

[+] Generating Anormal BGP updates for IPv4 session...

[+] Generating extended prefix hijacking attack with specified target prefixes
[+] Generating extended prefix hijacking attack: AS39224 hijacking AS47066's prefixes
Target prefixes for hijacking: ['203.0.113.0/24', '203.0.113.0/25', '203.0.113.128/25', '198.51.100.0/24', '198.51.100.0/25', '198.51.100.128/25', '192.0.2.0/24', '192.0.2.0/25', '192.0.2.128/25']
  Sending hijacked announcements to AS3010...
    - Sent hijacked announcement for 203.0.113.0/24
    - Sent hijacked announcement for 203.0.113.0/25
    - Sent hijacked announcement for 203.0.113.128/25
    - Sent hijacked announcement for 198.51.100.0/24
    - Sent hijacked announcement for 198.51.100.0/25
    - Sent hijacked announcement for 198.51.100.128/25
    - Sent hijacked announcement for 192.0.2.0/24
    - Sent hijacked announcement for 192.0.2.0/25
    - Sent hijacked announcement for 192.0.2.128/25
    - Sent withdrawal for 203.0.113.0/24
    - Sent re-announceme

In [125]:
# ======================================
# PART 8:BGP UPDATE PCAP to CSV Converter with Path Attribute Extraction
# ======================================


# Load the BGP layer
load_contrib('bgp')

# Define IP ID ranges for different traffic types (same as generation)
NORMAL_TRAFFIC_ID_RANGE = (1000, 29999)      # Regular BGP updates, keepalives
PREFIX_HIJACK_ID_RANGE = (30000, 39999)      # Prefix hijacking attacks
PATH_MANIP_ID_RANGE = (40000, 49999)         # Path manipulation attacks
DOS_ATTACK_ID_RANGE = (50000, 59999)         # DoS attacks
ROUTE_LEAK_ID_RANGE = (60000, 65535)         # Route leaks (if implemented)

def determine_traffic_type(ip_id):
    """Determine traffic type based on IP ID"""
    if NORMAL_TRAFFIC_ID_RANGE[0] <= ip_id <= NORMAL_TRAFFIC_ID_RANGE[1]:
        return "normal"
    elif PREFIX_HIJACK_ID_RANGE[0] <= ip_id <= PREFIX_HIJACK_ID_RANGE[1]:
        return "prefix_hijacking"
    elif PATH_MANIP_ID_RANGE[0] <= ip_id <= PATH_MANIP_ID_RANGE[1]:
        return "path_manipulation"
    elif DOS_ATTACK_ID_RANGE[0] <= ip_id <= DOS_ATTACK_ID_RANGE[1]:
        return "dos_attack"
    elif ROUTE_LEAK_ID_RANGE[0] <= ip_id <= ROUTE_LEAK_ID_RANGE[1]:
        return "route_leak"
    else:
        return "unknown"

def format_community(community_value):
    """Convert raw community value to ASN:value format"""
    try:
        value = int(community_value)
        asn = value >> 16
        comm_val = value & 0xFFFF
        return f"{asn}:{comm_val}"
    except:
        return str(community_value)

def extract_bgp_updates_to_csv(pcap_file, output_file):
    """Extract BGP UPDATE information from pcap to CSV with detailed attribute extraction"""
    print(f"[+] Reading pcap file: {pcap_file}")
    
    try:
        # Read pcap file
        packets = rdpcap(pcap_file)
        print(f"[+] Successfully read {len(packets)} packets")
        
        # Create output directory if it doesn't exist
        output_dir = os.path.dirname(output_file)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        # Open CSV file for writing
        with open(output_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            
            # Write header with Label column
            writer.writerow([
                'Type', 'Timestamp', 'Subtype', 'Peer_IP', 'Peer_ASN', 
                'Prefix', 'AS_Path', 'Origin', 'Next_Hop', 'MED', 
                'Local_Pref', 'Communities', 'Aggregator_Flag', 'Aggregator_ASN Aggregator_IP', 'Label'
            ])
            
            # Process each packet
            update_count = 0
            withdraw_count = 0
            ipv4_withdraw_count = 0
            ipv6_withdraw_count = 0
            
            # Debug counter for attributes
            debug_count = 0
            
            for i, pkt in enumerate(packets):
                # Skip non-IP packets
                if IP not in pkt or TCP not in pkt:
                    continue
                
                # Skip non-BGP TCP packets
                if pkt[TCP].dport != 179 and pkt[TCP].sport != 179:
                    continue
                    
                # Skip packets with no payload
                if len(pkt[TCP].payload) == 0:
                    continue
                
                # Try to extract BGP information
                try:
                    # Get BGP layer - Scapy should auto-parse the BGP layer
                    if BGPHeader in pkt and pkt[BGPHeader].type == 2:  # Type 2 is UPDATE
                        update_msg = pkt[BGPUpdate]
                        
                        # Get timestamp
                        try:
                            timestamp = datetime.datetime.fromtimestamp(float(pkt.time)).strftime('%Y-%m-%d %H:%M:%S.%f')
                        except:
                            timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
                        
                        # Get peer info
                        peer_ip = pkt[IP].src
                        peer_asn = ""  # Need to extract from AS_PATH or context
                        
                        # 1. Check for standard IPv4 withdrawals
                        if hasattr(update_msg, 'withdrawn_routes') and update_msg.withdrawn_routes:
                            for withdrawn in update_msg.withdrawn_routes:
                                try:
                                    prefix = withdrawn.prefix
                                    writer.writerow([
                                        'BGP', timestamp, 'WITHDRAW', peer_ip, peer_asn,
                                        prefix, "", "", "", "", "", "", "0", "", 
                                        determine_traffic_type(pkt[IP].id)
                                    ])
                                    withdraw_count += 1
                                    ipv4_withdraw_count += 1
                                except Exception as e:
                                    print(f"Error extracting standard withdrawal: {e}")
                        
                        # 2. Check for MP_UNREACH_NLRI (IPv6 withdrawals)
                        if hasattr(update_msg, 'path_attr') and update_msg.path_attr:
                            for attr in update_msg.path_attr:
                                # MP_UNREACH_NLRI has type code 15
                                if hasattr(attr, 'type_code') and attr.type_code == 15:
                                    try:
                                        # Extract the MP_UNREACH_NLRI attribute
                                        mp_unreach = attr.attribute
                                        
                                        # Get AFI and SAFI
                                        afi = mp_unreach.afi if hasattr(mp_unreach, 'afi') else "unknown"
                                        safi = mp_unreach.safi if hasattr(mp_unreach, 'safi') else "unknown"
                                        
                                        # Debug info for first few MP_UNREACH
                                        if debug_count < 3:
                                            debug_count += 1
                                            print(f"DEBUG MP_UNREACH: {dir(mp_unreach)}")
                                            if hasattr(mp_unreach, 'afi_safi_specific'):
                                                specific = mp_unreach.afi_safi_specific
                                                print(f"DEBUG MP_UNREACH SPECIFIC: {dir(specific)}")
                                        
                                        # Check if there's an AFI/SAFI specific part
                                        if hasattr(mp_unreach, 'afi_safi_specific'):
                                            specific = mp_unreach.afi_safi_specific
                                            
                                            # Check if this specific part has withdrawn routes
                                            if hasattr(specific, 'withdrawn_routes'):
                                                for withdrawn in specific.withdrawn_routes:
                                                    try:
                                                        prefix = withdrawn.prefix
                                                        writer.writerow([
                                                            'BGP', timestamp, f'WITHDRAW_MP_UNREACH_NLRI_AFI{afi}', peer_ip, peer_asn,
                                                            prefix, "", "", "", "", "", "", "0", "", 
                                                            determine_traffic_type(pkt[IP].id)
                                                        ])
                                                        withdraw_count += 1
                                                        ipv6_withdraw_count += 1
                                                    except Exception as e:
                                                        print(f"Error extracting MP_UNREACH withdrawal: {e}")
                                    except Exception as e:
                                        print(f"Error processing MP_UNREACH_NLRI: {e}")
                        
                        # 3. Check for announced routes
                        if hasattr(update_msg, 'nlri') and update_msg.nlri:
                            
                            # Initialize path attributes
                            as_path = ""
                            origin = ""
                            next_hop = ""
                            med = ""
                            local_pref = ""
                            communities = ""
                            aggregator_flag = "0"  # Use 0/1 instead of False/True
                            aggregator_info = ""
                            
                            # Extract path attributes
                            if hasattr(update_msg, 'path_attr') and update_msg.path_attr:
                                for attr in update_msg.path_attr:
                                    if not hasattr(attr, 'type_code'):
                                        continue
                                        
                                    # Debug: Print attribute details for first few packets
                                    if i < 3 and (attr.type_code in [7, 8]):
                                        print(f"DEBUG Packet {i} - Attr type {attr.type_code}: {dir(attr.attribute)}")
                                    
                                    if attr.type_code == 1:  # ORIGIN
                                        origin_codes = {0: 'IGP', 1: 'EGP', 2: 'INCOMPLETE'}
                                        try:
                                            origin_val = attr.attribute.origin
                                            origin = origin_codes.get(origin_val, str(origin_val))
                                        except:
                                            origin = "Unknown"
                                            
                                    elif attr.type_code == 2:  # AS_PATH
                                        segments = []
                                        try:
                                            # Check if segments attribute exists
                                            if hasattr(attr.attribute, 'segments'):
                                                segments_list = attr.attribute.segments
                                                for segment in segments_list:
                                                    if hasattr(segment, 'segment_type') and hasattr(segment, 'segment_value'):
                                                        segment_type = segment.segment_type
                                                        segment_values = segment.segment_value
                                                        
                                                        if segment_type == 1:  # AS_SET
                                                            as_set_str = '{' + ','.join(map(str, segment_values)) + '}'
                                                            segments.append(as_set_str)
                                                        elif segment_type == 2:  # AS_SEQUENCE
                                                            segments.extend(map(str, segment_values))
                                                
                                                # Set peer ASN if not already set and we have segment values
                                                if not peer_asn and segments:
                                                    peer_asn = segments[0]  # First ASN in path
                                            
                                            as_path = " ".join(segments)
                                        except Exception as e:
                                            as_path = f"ERROR: {str(e)}"
                                            
                                    elif attr.type_code == 3:  # NEXT_HOP
                                        try:
                                            if hasattr(attr.attribute, 'next_hop'):
                                                next_hop = attr.attribute.next_hop
                                        except:
                                            next_hop = "Unknown"
                                            
                                    elif attr.type_code == 4:  # MED
                                        try:
                                            if hasattr(attr.attribute, 'med'):
                                                med = str(attr.attribute.med)
                                        except:
                                            med = ""
                                            
                                    elif attr.type_code == 5:  # LOCAL_PREF
                                        try:
                                            if hasattr(attr.attribute, 'local_pref'):
                                                local_pref = str(attr.attribute.local_pref)
                                        except:
                                            local_pref = ""
                                            
                                    elif attr.type_code == 6:  # ATOMIC_AGGREGATE
                                        aggregator_flag = "1"  # Use 1 instead of True
                                        
                                    elif attr.type_code == 7:  # AGGREGATOR
                                        aggregator_flag = "1"  # Use 1 instead of True
                                        try:
                                            # Check different field naming patterns
                                            if hasattr(attr.attribute, 'aggregator_asn') and hasattr(attr.attribute, 'aggregator_ip'):
                                                aggregator_info = f"{attr.attribute.aggregator_asn} {attr.attribute.aggregator_ip}"
    
                                            else:
                                                # Try to find the right field names
                                                attr_fields = dir(attr.attribute)
                                                asn_field = next((f for f in attr_fields if "asn" in f.lower() or "as" == f.lower()), None)
                                                ip_field = next((f for f in attr_fields if "ip" in f.lower() or "addr" in f.lower()), None)
                                                
                                                if asn_field and ip_field:
                                                    asn_val = getattr(attr.attribute, asn_field)
                                                    ip_val = getattr(attr.attribute, ip_field)
                                                    aggregator_info = f"{asn_val} {ip_val}"
                                                else:
                                                    aggregator_info = str(attr.attribute)
                                        except Exception as e:
                                            aggregator_info = f"ERROR: {str(e)}"
                                            
                                    elif attr.type_code == 8:  # COMMUNITIES
                                        try:
                                            comm_list = []
                                            if hasattr(attr, 'attribute') and hasattr(attr.attribute, '__iter__'):
                                                for comm in attr.attribute:
                                                    if hasattr(comm, 'community'):
                                                        # Convert to ASN:value format
                                                        formatted_comm = format_community(comm.community)
                                                        comm_list.append(formatted_comm)
                                                communities = " ".join(comm_list)
                                            else:
                                                communities = str(attr.attribute)
                                        except Exception as e:
                                            communities = f"ERROR: {str(e)}"
                            
                            # Write each NLRI (announced prefix) as a separate row
                            for nlri in update_msg.nlri:
                                try:
                                    prefix = nlri.prefix
                                    writer.writerow([
                                        'BGP', timestamp, 'ANNOUNCE', peer_ip, peer_asn,
                                        prefix, as_path, origin, next_hop, med, 
                                        local_pref, communities, aggregator_flag, aggregator_info,
                                        determine_traffic_type(pkt[IP].id)
                                    ])
                                    update_count += 1
                                except Exception as e:
                                    # Skip if can't extract prefix
                                    print(f"Error processing NLRI prefix: {str(e)}")
                                    pass
                except Exception as e:
                    # Skip packets that can't be parsed
                    if i < 10:  # Only print errors for first few packets
                        print(f"Error parsing packet {i}: {e}")
                
                # Print progress every 1000 packets
                if (i+1) % 1000 == 0:
                    print(f"Processed {i+1} packets...")
            
        print(f"[+] Finished processing {len(packets)} total packets")
        print(f"[+] Found {update_count} announcements and {withdraw_count} withdrawals")
        print(f"    - IPv4 withdrawals: {ipv4_withdraw_count}")
        print(f"    - IPv6 withdrawals (MP_UNREACH_NLRI): {ipv6_withdraw_count}")
        print(f"[+] CSV file saved to {output_file}")
        
    except Exception as e:
        print(f"[!] Error processing pcap: {str(e)}")
        raise

if __name__ == "__main__":
    # Define input and output paths
    pcap_file = "/Users/shadimoteali/PhD/BGP_Traffic_Generation/pcaps/realistic_bgp_complete_scenarios.pcap"
    output_csv = "/Users/shadimoteali/PhD/BGP_Traffic_Generation/results/bgp_updates_analysis.csv"
    
    # Extract data
    extract_bgp_updates_to_csv(pcap_file, output_csv)
    
    # Print label statistics
    print("\n[+] Generating label statistics...")
    
    try:
        # Read the CSV to count labels
        with open(output_csv, 'r') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header
            
            label_counts = {
                "normal": 0,
                "prefix_hijacking": 0, 
                "path_manipulation": 0,
                "dos_attack": 0,
                "route_leak": 0,
                "unknown": 0
            }
            
            # Count each label
            for row in reader:
                label = row[-1]  # Last column is the label
                if label in label_counts:
                    label_counts[label] += 1
                else:
                    label_counts["unknown"] += 1
            
            # Print statistics
            print("\nTraffic Label Statistics:")
            print("========================")
            total = sum(label_counts.values())
            for label, count in label_counts.items():
                if count > 0:  # Only show non-zero counts
                    percentage = (count / total) * 100 if total > 0 else 0
                    print(f"{label}: {count} updates ({percentage:.2f}%)")
            print(f"Total: {total} updates")
                
    except Exception as e:
        print(f"[!] Error generating statistics: {str(e)}")

[+] Reading pcap file: /Users/shadimoteali/PhD/BGP_Traffic_Generation/pcaps/realistic_bgp_complete_scenarios.pcap
[+] Successfully read 2590 packets
DEBUG Packet 0 - Attr type 8: ['_PickleType', '__all_slots__', '__bool__', '__bytes__', '__class__', '__class_getitem__', '__contains__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__div__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__iterlen__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__nonzero__', '__orig_bases__', '__parameters__', '__rdiv__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__rtruediv__', '__setattr__', '__setitem__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__truediv__', '__weakref__', '_answered', '_command', '_do_summary', '_is_protocol', '_name', '_overload_fields', '_pkt', '